In [1]:
"""
ULTIMATE FRAUD DETECTION ENGINE v2.1 - RECALL-OPTIMIZED HYBRID SYSTEM
========================================================================================
ENHANCEMENTS ADDED TO ORIGINAL ROBUST SYSTEM:
1. Multi-Stage Bayesian Optimization (200+ trials per model with early stopping)
2. Recall-Focused Multi-Objective Optimization (F2-weighted scoring)
3. Advanced Stacking with multiple meta-learners
4. Dynamic ensemble strategy selection based on recall thresholds
5. Extended hyperparameter search spaces
6. Maintains ALL original 150+ features and robustness features
"""

import asyncio
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import multiprocessing as mp
import os
import gc

# ============================================================================
# CENTRALIZED THREAD MANAGEMENT
# ============================================================================
# Use half of CPU cores to prevent oversubscription
MAX_THREADS = max(1, os.cpu_count() // 2)

# Thread control for numerical libraries
os.environ['OMP_NUM_THREADS'] = str(MAX_THREADS)
os.environ['OPENBLAS_NUM_THREADS'] = str(MAX_THREADS)
os.environ['MKL_NUM_THREADS'] = str(MAX_THREADS)
os.environ['NUMEXPR_MAX_THREADS'] = str(MAX_THREADS)

# ============================================================================
# DEVICE DETECTION
# ============================================================================
try:
    import torch
    DEVICE_AVAILABLE = {
        'cuda': torch.cuda.is_available(),
        'mps': torch.backends.mps.is_available() if hasattr(torch.backends, 'mps') else False
    }
except ImportError:
    import subprocess
    # Fallback: check nvidia-smi for CUDA
    try:
        subprocess.run(['nvidia-smi'], capture_output=True, check=True)
        DEVICE_AVAILABLE = {'cuda': True, 'mps': False}
    except:
        DEVICE_AVAILABLE = {'cuda': False, 'mps': False}

def get_compute_device():
    """Auto-detect best available device"""
    if DEVICE_AVAILABLE['cuda']:
        return 'cuda', 'GPU (NVIDIA)'
    elif DEVICE_AVAILABLE['mps']:
        return 'mps', 'GPU (Apple Silicon)'
    else:
        return 'cpu', 'CPU'

DEVICE, DEVICE_NAME = get_compute_device()
print(f"🖥️  Compute device: {DEVICE_NAME}")
print(f"🔧 MAX_THREADS: {MAX_THREADS}")


import pandas as pd
import numpy as np
from datetime import datetime
import warnings
import joblib
import json
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from math import radians, sin, cos, sqrt, atan2
from scipy import stats
from scipy.stats import skew, kurtosis
from collections import defaultdict


# Core ML Libraries
from sklearn.model_selection import train_test_split, StratifiedKFold, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, 
    precision_recall_curve, average_precision_score, f1_score,
    accuracy_score, precision_score, recall_score, roc_curve, 
    matthews_corrcoef, cohen_kappa_score, balanced_accuracy_score,
    fbeta_score
)
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

# Gradient Boosting Ensemble
import xgboost as xgb
try:
    import lightgbm as lgb
    LIGHTGBM_AVAILABLE = True
except:
    LIGHTGBM_AVAILABLE = False
    print("⚠️ LightGBM not available")

try:
    import catboost as cb
    CATBOOST_AVAILABLE = True
except:
    CATBOOST_AVAILABLE = False
    print("⚠️ CatBoost not available")

# Network Analysis
try:
    import networkx as nx
    NETWORKX_AVAILABLE = True
except:
    NETWORKX_AVAILABLE = False
    print("⚠️ NetworkX not available (graph features disabled)")

# Bayesian Optimization
try:
    import optuna
    from optuna.samplers import TPESampler
    from optuna.pruners import SuccessiveHalvingPruner, MedianPruner
    OPTUNA_AVAILABLE = True
except:
    OPTUNA_AVAILABLE = False
    print("⚠️ Optuna not available (using default parameters)")

# Configuration
warnings.filterwarnings('ignore')
np.random.seed(42)
RANDOM_STATE = 42

def convert_to_serializable(obj):
    """Convert numpy types to Python native types"""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: convert_to_serializable(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_serializable(item) for item in obj]
    return obj


# ============================================================================
# HAVERSINE DISTANCE FUNCTION (for Impossible Travel Detection)
# ============================================================================
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points on Earth.
    Returns distance in kilometers.
    
    Used for impossible travel detection - if a user appears in two
    locations faster than physically possible (e.g., >900 km/h), it's fraud.
    
    Parameters:
    -----------
    lat1, lon1 : float - First location (latitude, longitude in degrees)
    lat2, lon2 : float - Second location (latitude, longitude in degrees)
    
    Returns:
    --------
    float : Distance in kilometers
    """
    R = 6371  # Earth's radius in km
    
    try:
        phi1 = radians(float(lat1))
        phi2 = radians(float(lat2))
        dphi = radians(float(lat2) - float(lat1))
        dlambda = radians(float(lon2) - float(lon1))
        
        a = sin(dphi / 2) ** 2 + cos(phi1) * cos(phi2) * sin(dlambda / 2) ** 2
        
        return 2 * R * atan2(sqrt(a), sqrt(1 - a))
    except:
        return 0.0


# ============================================================================
# RULE WEIGHTS (Calibrated based on fraud detection research)
# ============================================================================
RULE_WEIGHTS = {
    # =========================================================================
    # EXISTING RULES (Weights 10-50)
    # =========================================================================
    "new_device": 15,              # Transaction from unknown device
    "new_country": 20,             # Transaction from different country
    "high_amount": 25,             # Amount significantly above user average (>5x)
    "velocity_attack": 30,         # Too many transactions in short time (>=5 in 10min)
    "velocity_attack_extreme": 50, # Extreme velocity attack (>10 txns in 10min)
    "velocity_suspicious": 15,     # Suspicious velocity (3-4 txns in 10min)
    "impossible_travel": 40,       # Physically impossible location change (>900 km/h)
    "night_transaction": 10,       # Transaction during unusual hours (0-5 AM)
    "amount_anomaly_extreme": 35,  # Amount >10x user average
    "new_device_night_high": 45,   # Combination: new device + night + high amount
    "new_country_high_amount": 40, # Combination: new country + high amount
    "first_txn_high": 30,          # First transaction is unusually high
    "fraud_history": 50,           # User has past fraud incidents
    "rapid_burst": 25,             # >3 transactions in 1 minute
    "high_risk_merchant_night": 20, # Risky merchant category at night
    "country_mismatch": 35,        # Multiple country mismatches
    
    # =========================================================================
    # NEW RULES - PHASE 1: SEQUENTIAL PATTERN RULES
    # =========================================================================
    "card_testing_sequence": 85,   # Micro txns (<$15) followed by large txn (>$300)
    "merchant_category_hopping": 70, # 3+ high-risk categories in 1 hour
    
    # =========================================================================
    # NEW RULES - PHASE 1: PHYSICAL IMPOSSIBILITY RULES
    # =========================================================================
    "speed_of_light_violation": 98,  # Travel >1500 km/h (HARD BLOCK - impossible)
    "suspicious_travel": 50,         # Travel 500-900 km/h (rare but possible)
    "timezone_impossibility": 60,    # 2-5 AM local + country mismatch
    
    # =========================================================================
    # NEW RULES - PHASE 1: BUSINESS LOGIC RULES
    # =========================================================================
    "sanctioned_country_merchant": 90, # US/UK user + sanctioned country merchant
    "payment_method_mismatch": 65,   # Crypto merchant + card payment
    "refund_before_purchase": 98,    # Refund before purchase (HARD BLOCK - 100% fraud)
    
    # =========================================================================
    # NEW RULES - PHASE 1: IDENTITY COHERENCE RULES
    # =========================================================================
    "impossible_user_profile": 75,   # New signup + KYC failed + high value
    "device_fingerprint_chaos": 80,  # 5+ devices + 5+ IPs in 24 hours
    "email_country_mismatch": 40,    # Email domain (.ru) != billing country (US)
    
    # =========================================================================
    # NEW RULES - PHASE 2: REPEAT OFFENDER & MICRO TXN PATTERNS
    # =========================================================================
    "repeat_fraud_offender": 85,     # User with 2+ past fraud incidents (HARD BLOCK)
    "micro_txn_velocity": 85,        # 3+ micro transactions in 5 min (HARD BLOCK - card testing)
    "fraud_history_high": 65,        # User with fraud history + above-avg purchase
}


# ============================================================================
# RULE CONDITIONS (Declarative format - can be serialized to JSON)
# ============================================================================
# Each rule is defined as a dict with:
#   - field: The transaction field to check
#   - op: Operator (>, >=, <, <=, ==, between, in, and, or)
#   - value: The comparison value(s)
#
# For compound conditions, use 'and'/'or' operators with nested conditions
# ============================================================================
RULE_CONDITIONS = {
    # =========================================================================
    # TIER 1: CRITICAL RULES
    # =========================================================================
    "impossible_travel": {
        "field": "travel_speed_kmh",
        "op": ">",
        "value": 900
    },
    "fraud_history_high": {
        "op": "and",
        "conditions": [
            {"field": "user_past_fraud_count", "op": ">=", "value": 2},
            {"field": "amount", "op": ">", "value": 200}
        ]
    },
    
    # =========================================================================
    # TIER 2: HIGH-RISK COMBINATION RULES
    # =========================================================================
    "new_device_night_high": {
        "op": "and",
        "conditions": [
            {"field": "is_new_device", "op": "==", "value": 1},
            {"field": "is_night", "op": "==", "value": 1},
            {"field": "amount", "op": ">", "value": 500}
        ]
    },
    "new_country_high_amount": {
        "op": "and",
        "conditions": [
            {"field": "is_new_country", "op": "==", "value": 1},
            {"field": "amount", "op": ">", "value": 1000}
        ]
    },
    
    # =========================================================================
    # TIER 3: VELOCITY & BEHAVIORAL RULES (Graduated Thresholds)
    # =========================================================================
    "velocity_attack_extreme": {
        "field": "txns_last_10min",
        "op": ">",
        "value": 10
    },
    "velocity_attack": {
        "field": "txns_last_10min",
        "op": "between",
        "value": [5, 10]
    },
    "velocity_suspicious": {
        "field": "txns_last_10min",
        "op": "between",
        "value": [3, 4]
    },
    "rapid_burst": {
        "field": "is_burst_txn",
        "op": "==",
        "value": 1
    },
    "amount_anomaly_extreme": {
        "field": "amount_vs_user_avg",
        "op": ">",
        "value": 10
    },
    "high_amount": {
        "field": "amount_vs_user_avg",
        "op": ">",
        "value": 5
    },
    
    # =========================================================================
    # TIER 4: SINGLE INDICATOR RULES
    # =========================================================================
    "first_txn_high": {
        "op": "and",
        "conditions": [
            {"field": "user_total_txn_count", "op": "==", "value": 0},
            {"field": "amount", "op": ">", "value": 1000}
        ]
    },
    "country_mismatch_risk": {
        "op": "and",
        "conditions": [
            {"field": "country_mismatch_count", "op": ">=", "value": 2},
            {"field": "high_risk_country_count", "op": ">=", "value": 1}
        ]
    },
    "new_device": {
        "field": "is_new_device",
        "op": "==",
        "value": 1
    },
    "new_country": {
        "field": "is_new_country",
        "op": "==",
        "value": 1
    },
    "night_transaction": {
        "field": "is_night",
        "op": "==",
        "value": 1
    },
    "risky_merchant_night": {
        "op": "and",
        "conditions": [
            {"field": "is_high_risk_category", "op": "==", "value": 1},
            {"field": "is_night", "op": "==", "value": 1}
        ]
    },
    
    # =========================================================================
    # TIER 5: SEQUENTIAL PATTERN RULES
    # =========================================================================
    "card_testing_sequence": {
        "op": "and",
        "conditions": [
            {"field": "micro_txn_count_5min", "op": ">=", "value": 2},
            {"field": "amount", "op": ">", "value": 300},
            {"field": "time_since_last_micro_txn_min", "op": "<", "value": 5}
        ]
    },
    "merchant_category_hopping": {
        "field": "distinct_high_risk_categories_1hr",
        "op": ">=",
        "value": 3
    },
    
    # =========================================================================
    # TIER 6: PHYSICAL IMPOSSIBILITY RULES
    # =========================================================================
    "speed_of_light_violation": {
        "field": "travel_speed_kmh",
        "op": ">",
        "value": 1500
    },
    "suspicious_travel": {
        "field": "travel_speed_kmh",
        "op": "between",
        "value": [500, 900]
    },
    "timezone_impossibility": {
        "op": "and",
        "conditions": [
            {"field": "is_night", "op": "==", "value": 1},
            {"field": "is_new_country", "op": "==", "value": 1},
            {"field": "local_hour", "op": "between", "value": [2, 5]}
        ]
    },
    
    # =========================================================================
    # TIER 7: BUSINESS LOGIC RULES
    # =========================================================================
    "sanctioned_country_merchant": {
        "op": "and",
        "conditions": [
            {"field": "user_country", "op": "in", "value": ["US", "UK", "GB", "DE", "FR"]},
            {"field": "merchant_country", "op": "in", "value": ["KP", "IR", "SY", "CU", "VE", "RU", "BY"]}
        ]
    },
    "payment_method_mismatch": {
        "op": "and",
        "conditions": [
            {"field": "merchant_category", "op": "in", "value": ["crypto", "cryptocurrency", "digital_wallet"]},
            {"field": "payment_method", "op": "in", "value": ["credit_card", "debit_card", "card"]}
        ]
    },
    "refund_before_purchase": {
        "op": "and",
        "conditions": [
            {"field": "is_refund", "op": "==", "value": 1},
            {"field": "user_total_purchase_count", "op": "==", "value": 0}
        ]
    },
    
    # =========================================================================
    # TIER 8: IDENTITY COHERENCE RULES
    # =========================================================================
    "impossible_user_profile": {
        "op": "and",
        "conditions": [
            {"field": "account_age_days", "op": "<", "value": 7},
            {"field": "kyc_status", "op": "in", "value": ["failed", "pending", "not_submitted"]},
            {"field": "amount", "op": ">", "value": 500}
        ]
    },
    "device_fingerprint_chaos": {
        "op": "and",
        "conditions": [
            {"field": "distinct_devices_24hr", "op": ">=", "value": 5},
            {"field": "distinct_ips_24hr", "op": ">=", "value": 5}
        ]
    },
    "email_country_mismatch": {
        "op": "and",
        "conditions": [
            {"field": "email_domain_country", "op": "!=", "value": ""},
            {"field": "billing_country", "op": "!=", "value": ""},
            {"field": "_custom", "op": "field_ne", "value": ["email_domain_country", "billing_country"]}
        ]
    },
    
    # =========================================================================
    # TIER 9: REPEAT OFFENDER & MICRO TXN PATTERNS
    # =========================================================================
    "repeat_fraud_offender": {
        "field": "user_past_fraud_count",
        "op": ">=",
        "value": 2
    },
    "micro_txn_velocity": {
        "field": "micro_txn_count_5min",
        "op": ">=",
        "value": 3
    },
}


# ============================================================================
# PHASE 2: GRADUATED THRESHOLDS FOR TRAVEL SPEED
# ============================================================================
def compute_travel_risk(speed_kmh):
    """
    Graduated thresholds for travel speed detection.
    
    Instead of a binary cutoff, this function returns graduated risk scores
    based on how physically impossible the travel speed is.
    
    Parameters:
    -----------
    speed_kmh : float
        Calculated travel speed in kilometers per hour
    
    Returns:
    --------
    tuple: (score, reason)
        - score: int (0-95) risk score
        - reason: str or None - human readable reason
    
    Speed Thresholds:
    - >1500 km/h: Teleportation (impossible even by supersonic jet)
    - 900-1500 km/h: Impossible for commercial travel (max ~900 km/h)
    - 500-900 km/h: Suspicious but possible (private jet, connection)
    - <500 km/h: Normal travel
    """
    if speed_kmh > 1500:
        return 95, "Speed-of-Light Violation (teleportation detected)"
    elif speed_kmh > 900:
        return 80, "Impossible Travel (faster than commercial aircraft)"
    elif speed_kmh > 500:
        return 50, "Suspicious Travel (unusually fast)"
    else:
        return 0, None


# ============================================================================
# PHASE 3: CONTEXTUAL RULE WEIGHTING
# ============================================================================
def contextual_weight_night_transaction(base_weight, context):
    """
    Dynamically adjust night transaction rule weight based on context.
    
    Business users working late, entertainment purchases, etc. should
    have reduced weight, while financial transactions at night are riskier.
    
    Parameters:
    -----------
    base_weight : int
        The base rule weight (default 10 for night_transaction)
    context : dict
        Transaction context with keys like:
        - user_segment: 'consumer' | 'corporate' | 'vip'
        - merchant_category: category string
        - is_new_user: bool
        - amount: float
    
    Returns:
    --------
    int: Adjusted weight (capped at 95)
    
    Adjustment Logic:
    - Corporate users: 70% reduction (they work late)
    - Entertainment/Food: 50% reduction (normal at night)
    - Crypto/Money Transfer: 2x increase (high risk at night)
    - New user + high value: 3x increase (major red flag)
    """
    weight = float(base_weight)
    
    # Reduce for business users (they work late, international calls, etc.)
    user_segment = context.get('user_segment', 'consumer')
    if user_segment in ['corporate', 'business', 'enterprise']:
        weight *= 0.3
    elif user_segment == 'vip':
        weight *= 0.5  # VIP users have unusual patterns
    
    # Reduce for entertainment/food (expected at night)
    merchant_category = context.get('merchant_category', '').lower()
    if merchant_category in ['food', 'restaurant', 'entertainment', 'bar', 'club', 'gaming']:
        weight *= 0.5
    
    # Increase for financial transactions at night (high risk)
    if merchant_category in ['crypto', 'cryptocurrency', 'money_transfer', 'wire_transfer', 'forex']:
        weight *= 2.0
    
    # Massive increase for new user + late night + high value
    is_new_user = context.get('is_new_user', False) or context.get('account_age_days', 9999) < 30
    amount = context.get('amount', 0)
    if is_new_user and amount > 1000:
        weight *= 3.0
    elif is_new_user and amount > 500:
        weight *= 2.0
    
    # Cap at 95 to leave room for critical rules
    return int(min(95, max(0, weight)))



# FRAMEWORK COMPONENT 1: RULE-BASED ENGINE (Enhanced with User Profile)
# ============================================================================
class RuleBasedEngine:
    """
    Rule-Based Fraud Detection Engine
    
    A production-grade fraud detection system using expert-crafted rules.
    Provides fast (~10-30ms), explainable decisions that complement ML models.
    
    Key Detection Patterns:
    1. Impossible Travel - User can't be in two distant locations simultaneously
    2. Velocity Attack - Too many transactions in short timeframe (>5 in 10 min)
    3. New Device/Country - Unusual access patterns
    4. Amount Anomaly - Transaction significantly above user's average (>5x)
    5. Night Transactions - Unusual timing (0-5 AM local time)
    6. Combination Rules - Multiple weak signals combining to strong evidence
    
    Risk Levels:
    - LOW (0-30): Normal transaction
    - MEDIUM-LOW (31-50): Minor anomaly, monitor
    - MEDIUM (51-70): Suspicious, may require verification
    - HIGH (71-90): Likely fraud, block or verify
    - CRITICAL (91-100): Definite fraud, block immediately
    
    Usage:
    ------
    # Basic usage (with pre-computed features)
    engine = RuleBasedEngine()
    score, rules = engine.evaluate(transaction_dict)
    
    # With user profile (for real-time enrichment)
    user_profile = {"known_devices": [...], "home_country": "US", "avg_amount": 200}
    result = engine.evaluate_with_profile(transaction, user_profile)
    """
    
    def __init__(self):
        self.weights = RULE_WEIGHTS
        self.rules = []
        self._setup_rules()
    
    def _setup_rules(self):
        """Define expert fraud detection rules with calibrated weights"""
        self.rules = [
            # =================================================================
            # TIER 1: CRITICAL RULES (Immediate Fraud Indicators)
            # =================================================================
            
            # Rule 1: Impossible Travel (>900 km/h travel speed)
            # If user was in NYC and 10 mins later in London, that's physically impossible
            ('impossible_travel', 
             lambda x: x.get('travel_speed_kmh', 0) > 900, 
             self.weights['impossible_travel']),
            
            # Rule 2: Fraud History + High Amount
            ('fraud_history_high', 
             lambda x: (x.get('user_past_fraud_count', 0) >= 2 and x.get('amount', 0) > 200),
             self.weights['fraud_history']),
            
            # =================================================================
            # TIER 2: HIGH-RISK COMBINATION RULES
            # =================================================================
            
            # Rule 3: New Device + Night + High Amount (classic fraud pattern)
            ('new_device_night_high', lambda x: (
                x.get('is_new_device', 0) == 1 and 
                x.get('is_night', 0) == 1 and 
                x.get('amount', 0) > 500
            ), self.weights['new_device_night_high']),
            
            # Rule 4: New Country + High Amount
            ('new_country_high_amount', lambda x: (
                x.get('is_new_country', 0) == 1 and 
                x.get('amount', 0) > 1000
            ), self.weights['new_country_high_amount']),
            
            # =================================================================
            # TIER 3: VELOCITY & BEHAVIORAL RULES (Graduated Thresholds)
            # =================================================================
            
            # Rule 5a: Extreme Velocity Attack (>10 txns in 10 min) - CRITICAL
            ('velocity_attack_extreme', lambda x: x.get('txns_last_10min', 0) > 10, 
             self.weights['velocity_attack_extreme']),
            
            # Rule 5b: Velocity Attack (>=5 txns in 10 min) - HIGH RISK
            ('velocity_attack', lambda x: 5 <= x.get('txns_last_10min', 0) <= 10, 
             self.weights['velocity_attack']),
            
            # Rule 5c: Suspicious Velocity (3-4 txns in 10 min) - MODERATE
            ('velocity_suspicious', lambda x: 3 <= x.get('txns_last_10min', 0) < 5, 
             self.weights['velocity_suspicious']),
            
            # Rule 6: Rapid Burst (>3 txns in 1 min)
            ('rapid_burst', lambda x: x.get('is_burst_txn', 0) == 1, 
             self.weights['rapid_burst']),
            
            # Rule 7: Extreme Amount Anomaly (>10x user average)
            ('amount_anomaly_extreme', lambda x: x.get('amount_vs_user_avg', 1) > 10,
             self.weights['amount_anomaly_extreme']),
            
            # Rule 8: High Amount Anomaly (>5x user average) - From friend's code
            ('high_amount', lambda x: x.get('amount_vs_user_avg', 1) > 5,
             self.weights['high_amount']),
            
            # =================================================================
            # TIER 4: SINGLE INDICATOR RULES
            # =================================================================
            
            # Rule 9: First Transaction > $1000
            ('first_txn_high', lambda x: (
                x.get('user_total_txn_count', 0) == 0 and 
                x.get('amount', 0) > 1000
            ), self.weights['first_txn_high']),
            
            # Rule 10: Country Mismatch + High Risk
            ('country_mismatch_risk', lambda x: (
                x.get('country_mismatch_count', 0) >= 2 and 
                x.get('high_risk_country_count', 0) >= 1
            ), self.weights['country_mismatch']),
            
            # Rule 11: New Device (standalone)
            ('new_device', lambda x: x.get('is_new_device', 0) == 1,
             self.weights['new_device']),
            
            # Rule 12: New Country (standalone)
            ('new_country', lambda x: x.get('is_new_country', 0) == 1,
             self.weights['new_country']),
            
            # Rule 13: Night Transaction (0-5 AM) - From friend's code
            ('night_transaction', lambda x: x.get('is_night', 0) == 1,
             self.weights['night_transaction']),
            
            # Rule 14: High-Risk Merchant at Night
            ('risky_merchant_night', lambda x: (
                x.get('is_high_risk_category', 0) == 1 and 
                x.get('is_night', 0) == 1
            ), self.weights['high_risk_merchant_night']),
            
            # =================================================================
            # TIER 5: SEQUENTIAL PATTERN RULES (ML models miss these)
            # =================================================================
            
            # Rule 15: Card Testing Sequence
            # Pattern: 2+ micro transactions (<$15) in 5min followed by large (>$300)
            ('card_testing_sequence', lambda x: (
                x.get('micro_txn_count_5min', 0) >= 2 and 
                x.get('amount', 0) > 300 and
                x.get('time_since_last_micro_txn_min', 999) < 5
            ), self.weights['card_testing_sequence']),
            
            # Rule 16: Rapid Merchant Category Hopping
            # Pattern: 3+ different high-risk categories in 1 hour
            ('merchant_category_hopping', lambda x: (
                x.get('distinct_high_risk_categories_1hr', 0) >= 3
            ), self.weights['merchant_category_hopping']),
            
            # =================================================================
            # TIER 6: PHYSICAL IMPOSSIBILITY RULES (Graduated Thresholds)
            # =================================================================
            
            # Rule 17: Speed-of-Light Violation (>1500 km/h - completely impossible)
            ('speed_of_light_violation', lambda x: x.get('travel_speed_kmh', 0) > 1500, 
             self.weights['speed_of_light_violation']),
            
            # Rule 18: Suspicious Travel (500-900 km/h - rare but possible with fast jets)
            ('suspicious_travel', lambda x: (
                500 < x.get('travel_speed_kmh', 0) <= 900
            ), self.weights['suspicious_travel']),
            
            # Rule 19: Timezone Impossibility
            # Transaction at 2-5 AM user's local time + different country
            ('timezone_impossibility', lambda x: (
                x.get('is_night', 0) == 1 and
                x.get('is_new_country', 0) == 1 and
                x.get('local_hour', 12) >= 2 and
                x.get('local_hour', 12) <= 5
            ), self.weights['timezone_impossibility']),
            
            # =================================================================
            # TIER 7: BUSINESS LOGIC RULES (Domain knowledge)
            # =================================================================
            
            # Rule 20: Sanctioned Country + Merchant Combination
            # US/UK/EU user transacting with merchant in sanctioned country
            ('sanctioned_country_merchant', lambda x: (
                x.get('user_country', '').upper() in ['US', 'UK', 'GB', 'DE', 'FR'] and
                x.get('merchant_country', '').upper() in ['KP', 'IR', 'SY', 'CU', 'VE', 'RU', 'BY']
            ), self.weights['sanctioned_country_merchant']),
            
            # Rule 21: Payment Method Mismatch
            # Crypto/digital-only merchant accepting traditional card payment
            ('payment_method_mismatch', lambda x: (
                x.get('merchant_category', '').lower() in ['crypto', 'cryptocurrency', 'digital_wallet'] and
                x.get('payment_method', '').lower() in ['credit_card', 'debit_card', 'card']
            ), self.weights['payment_method_mismatch']),
            
            # Rule 22: Refund Before Purchase (100% fraud indicator)
            ('refund_before_purchase', lambda x: (
                x.get('is_refund', 0) == 1 and
                x.get('user_total_purchase_count', 1) == 0
            ), self.weights['refund_before_purchase']),
            
            # =================================================================
            # TIER 8: IDENTITY COHERENCE RULES (Profile consistency)
            # =================================================================
            
            # Rule 23: Impossible User Profile
            # New user + failed KYC + high value first transaction
            ('impossible_user_profile', lambda x: (
                x.get('account_age_days', 9999) < 7 and
                x.get('kyc_status', 'passed').lower() in ['failed', 'pending', 'not_submitted'] and
                x.get('amount', 0) > 500
            ), self.weights['impossible_user_profile']),
            
            # Rule 24: Device Fingerprint Chaos
            # 5+ different devices AND 5+ different IPs in 24 hours
            ('device_fingerprint_chaos', lambda x: (
                x.get('distinct_devices_24hr', 0) >= 5 and
                x.get('distinct_ips_24hr', 0) >= 5
            ), self.weights['device_fingerprint_chaos']),
            
            # Rule 25: Email-Country Mismatch
            # Email domain suggests different country than billing address
            ('email_country_mismatch', lambda x: (
                x.get('email_domain_country', '').upper() != '' and
                x.get('billing_country', '').upper() != '' and
                x.get('email_domain_country', '').upper() != x.get('billing_country', '').upper()
            ), self.weights['email_country_mismatch']),
        ]
    
    def evaluate(self, transaction_dict):
        """
        Evaluate all rules and return risk score (0-100).
        Uses the MAX score approach (most severe rule wins).
        
        Parameters:
        -----------
        transaction_dict : dict
            Transaction features as a dictionary
        
        Returns:
        --------
        tuple: (max_score, triggered_rules)
            - max_score: int (0-100)
            - triggered_rules: list of (rule_name, score) tuples
        """
        triggered_rules = []
        max_score = 0
        
        for rule_name, rule_func, score in self.rules:
            try:
                if rule_func(transaction_dict):
                    triggered_rules.append((rule_name, score))
                    max_score = max(max_score, score)
            except:
                pass
        
        return max_score, triggered_rules
    
    def evaluate_with_profile(self, transaction, user_profile):
        """
        Evaluate transaction with real-time user profile enrichment.
        This is the method from friend's implementation that uses user context.
        
        Parameters:
        -----------
        transaction : dict
            Raw transaction data with keys like:
            - device_id, country, amount, lat, lon, timestamp, hours_since_last_txn
        
        user_profile : dict
            User context with keys like:
            - known_devices: list of device IDs user has used before
            - home_country: user's primary country
            - avg_amount: user's average transaction amount
            - txns_last_10_min: count of transactions in last 10 minutes
            - last_lat, last_lon: last known location
        
        Returns:
        --------
        dict with:
            - rule_risk_score: 0-100 score
            - risk_level: LOW/MEDIUM-LOW/MEDIUM/HIGH/CRITICAL
            - triggered_rules: list of rule names
        """
        # Enrich transaction with user profile
        enriched = self._enrich_with_profile(transaction, user_profile)
        
        # Evaluate enriched transaction
        risk_score = 0
        triggered_rules = []
        
        # Rule 1: New Device (from friend's code)
        if transaction.get("device_id") not in user_profile.get("known_devices", []):
            risk_score += self.weights["new_device"]
            triggered_rules.append("New Device")
        
        # Rule 2: New Country (from friend's code)
        if transaction.get("country") != user_profile.get("home_country", ""):
            risk_score += self.weights["new_country"]
            triggered_rules.append("New Country")
        
        # Rule 3: High Amount vs User Average (from friend's code)
        avg_amount = user_profile.get("avg_amount", 0)
        if avg_amount > 0:
            amount_ratio = transaction.get("amount", 0) / avg_amount
            if amount_ratio > 5:
                risk_score += self.weights["high_amount"]
                triggered_rules.append("High Amount (>5x avg)")
        
        # Rule 4: Velocity Attack (from friend's code)
        if user_profile.get("txns_last_10_min", 0) > 5:
            risk_score += self.weights["velocity_attack"]
            triggered_rules.append("Velocity Attack")
        
        # Rule 5: Impossible Travel (from friend's code - uses haversine)
        if all(k in transaction for k in ['lat', 'lon']) and \
           all(k in user_profile for k in ['last_lat', 'last_lon']):
            distance = haversine(
                transaction['lat'], transaction['lon'],
                user_profile['last_lat'], user_profile['last_lon']
            )
            hours = max(transaction.get('hours_since_last_txn', 0.01), 0.01)
            travel_speed = distance / hours
            
            if travel_speed > 900:  # > 900 km/h is impossible
                risk_score += self.weights["impossible_travel"]
                triggered_rules.append("Impossible Travel")
        
        # Rule 6: Night Transaction (from friend's code)
        if 'timestamp' in transaction:
            try:
                hour = datetime.fromisoformat(str(transaction['timestamp'])).hour
                if 0 <= hour <= 5:
                    risk_score += self.weights["night_transaction"]
                    triggered_rules.append("Night Transaction")
            except:
                pass
        
        # Also run standard rule evaluation on enriched features
        standard_score, standard_rules = self.evaluate(enriched)
        
        # Combine scores (take max)
        final_score = max(risk_score, standard_score)
        all_rules = triggered_rules + [r[0] for r in standard_rules]
        
        return {
            "rule_risk_score": min(final_score, 100),
            "risk_level": self.classify_risk(final_score),
            "triggered_rules": list(set(all_rules))  # Remove duplicates
        }
    
    def _enrich_with_profile(self, txn, profile):
        """
        Add derived features based on user profile for standard rule evaluation.
        """
        enriched = txn.copy() if isinstance(txn, dict) else {}
        
        # Check if device is new
        if 'device_id' in txn and 'known_devices' in profile:
            enriched['is_new_device'] = 1 if txn['device_id'] not in profile['known_devices'] else 0
        
        # Check if country is new
        if 'country' in txn and 'home_country' in profile:
            enriched['is_new_country'] = 1 if txn['country'] != profile['home_country'] else 0
        
        # Compute amount vs user average
        if 'amount' in txn and 'avg_amount' in profile and profile['avg_amount'] > 0:
            enriched['amount_vs_user_avg'] = txn['amount'] / profile['avg_amount']
        
        # Check velocity
        if 'txns_last_10_min' in profile:
            enriched['txns_last_10min'] = profile['txns_last_10_min']
        
        # Compute travel speed if location data available
        if all(k in txn for k in ['lat', 'lon']) and \
           all(k in profile for k in ['last_lat', 'last_lon']):
            distance = haversine(txn['lat'], txn['lon'], 
                                profile['last_lat'], profile['last_lon'])
            hours = max(txn.get('hours_since_last_txn', 0.01), 0.01)
            enriched['travel_speed_kmh'] = distance / hours
        
        # Check if night transaction
        if 'timestamp' in txn:
            try:
                hour = datetime.fromisoformat(str(txn['timestamp'])).hour
                enriched['is_night'] = 1 if 0 <= hour <= 5 else 0
            except:
                pass
        
        return enriched
    
    def classify_risk(self, score):
        """
        Classify risk level based on cumulative score.
        From friend's implementation.
        """
        if score <= 30:
            return "LOW"
        elif score <= 50:
            return "MEDIUM-LOW"
        elif score <= 70:
            return "MEDIUM"
        elif score <= 90:
            return "HIGH"
        else:
            return "CRITICAL"
    
    def predict_batch(self, X, feature_names):
        """Evaluate rules on batch of transactions (handles DataFrame or numpy array)"""
        # Convert to numpy array if DataFrame
        if hasattr(X, 'values'):
            X_arr = X.values
        else:
            X_arr = np.asarray(X)
        
        scores = np.zeros(len(X_arr))
        
        for i in range(len(X_arr)):
            txn_dict = dict(zip(feature_names, X_arr[i]))
            score, _ = self.evaluate(txn_dict)
            scores[i] = score / 100.0  # Normalize to 0-1
        
        return scores
    
    def get_explanation(self, transaction_dict, user_profile=None):
        """
        Get a human-readable explanation of the fraud assessment.
        """
        if user_profile:
            result = self.evaluate_with_profile(transaction_dict, user_profile)
            score = result['rule_risk_score']
            risk_level = result['risk_level']
            rules = result['triggered_rules']
        else:
            score, rules = self.evaluate(transaction_dict)
            risk_level = self.classify_risk(score)
            rules = [r[0] for r in rules]
        
        explanation = []
        explanation.append(f"Risk Level: {risk_level} (Score: {score}/100)")
        
        if rules:
            explanation.append("Triggered Rules:")
            for rule in sorted(rules):
                weight = self.weights.get(rule.lower().replace(' ', '_'), '?')
                explanation.append(f"  - {rule}: +{weight} points")
        else:
            explanation.append("No fraud indicators detected.")
        
        return "\n".join(explanation)


# ============================================================================
# BAYESIAN HIGH-RISK IDENTIFIER (Neural Network for 60-84 Zone Detection)
# ============================================================================
class BayesianHighRiskIdentifier:
    """
    Bayesian Neural Network for High-Risk Transaction Identification
    
    This replaces the hardcoded 60-84 threshold range with a learned model
    that identifies transactions needing extra rule influence (35% blend).
    
    Architecture:
    - Input: Binary rule signals (25 rules → 25 features)
    - Hidden: 2 dense layers with dropout (Bayesian approximation)
    - Output: P(high_risk) + uncertainty via MC Dropout
    
    Training:
    - Positive: Fraud transactions with rule score 30-84 (gray zone frauds)
    - Negative: Legit transactions with rule score 30-84 (gray zone legits)
    - Goal: Learn which rule combinations in the gray zone indicate fraud
    
    Inference:
    - Returns is_high_risk (bool), confidence (0-1)
    - High confidence → trust the decision
    - Low confidence → let ML decide purely
    """
    
    def __init__(self, n_rules=25, hidden_dim=16, dropout_rate=0.3):
        self.n_rules = n_rules
        self.hidden_dim = hidden_dim
        self.dropout_rate = dropout_rate
        self.model = None
        self.is_fitted = False
        self.threshold = 0.5  # Decision threshold
        self.rule_names = []
    
    def _build_model(self):
        """Build a simple neural network using sklearn's MLPClassifier"""
        from sklearn.neural_network import MLPClassifier
        
        # MLPClassifier with similar architecture
        # Hidden layers simulate the BNN structure
        self.model = MLPClassifier(
            hidden_layer_sizes=(self.hidden_dim, self.hidden_dim // 2),
            activation='relu',
            solver='adam',
            alpha=0.01,  # L2 regularization (Bayesian prior approximation)
            max_iter=200,
            early_stopping=True,
            validation_fraction=0.15,
            random_state=RANDOM_STATE,
            verbose=False
        )
    
    def extract_rule_signals(self, X, feature_names, rule_engine):
        """
        Extract binary rule signals from transactions.
        
        Parameters:
        -----------
        X : np.ndarray
            Transaction features (unscaled)
        feature_names : list
            Feature names matching X columns
        rule_engine : RuleBasedEngine
            Rule engine with defined rules
            
        Returns:
        --------
        np.ndarray : Binary matrix (n_samples, n_rules)
        """
        if hasattr(X, 'values'):
            X_arr = X.values
        else:
            X_arr = np.asarray(X)
        
        n_samples = len(X_arr)
        n_rules = len(rule_engine.rules)
        
        # Extract binary signals for each rule
        rule_signals = np.zeros((n_samples, n_rules))
        
        for i in range(n_samples):
            txn_dict = dict(zip(feature_names, X_arr[i]))
            
            # Check each rule
            for j, (rule_name, rule_fn, weight) in enumerate(rule_engine.rules):
                try:
                    if rule_fn(txn_dict):
                        rule_signals[i, j] = 1.0
                except:
                    pass
        
        # Store rule names for interpretability
        self.rule_names = [r[0] for r in rule_engine.rules]
        
        return rule_signals
    
    def fit(self, X, feature_names, y, rule_engine, rule_scores=None):
        """
        Train the BNN to identify high-risk transactions in the gray zone.
        
        Only trains on transactions with rule scores in 30-84 range
        (the ambiguous zone where ML needs rule assistance).
        
        Parameters:
        -----------
        X : np.ndarray
            Transaction features (unscaled)
        feature_names : list
            Feature names
        y : np.ndarray
            Fraud labels (0/1)
        rule_engine : RuleBasedEngine
            Rule engine instance
        rule_scores : np.ndarray, optional
            Pre-computed rule scores (0-1 normalized)
        """
        print("\n   🧠 Training Bayesian High-Risk Identifier...")
        
        self._build_model()
        
        # Get rule scores if not provided
        if rule_scores is None:
            rule_scores = rule_engine.predict_batch(X, feature_names)
        
        # Filter to gray zone (30-84, i.e., 0.30-0.84 normalized)
        gray_zone_mask = (rule_scores >= 0.30) & (rule_scores < 0.85)
        n_gray = gray_zone_mask.sum()
        
        if n_gray < 100:
            print(f"      ⚠️ Only {n_gray} samples in gray zone, using full dataset")
            gray_zone_mask = np.ones(len(y), dtype=bool)
            n_gray = len(y)
        
        # Extract rule signals for gray zone
        rule_signals = self.extract_rule_signals(X, feature_names, rule_engine)
        
        X_train = rule_signals[gray_zone_mask]
        y_train = y[gray_zone_mask]
        
        # Balance the training data
        n_fraud = y_train.sum()
        n_legit = len(y_train) - n_fraud
        
        print(f"      Gray zone: {n_gray} samples ({n_fraud} fraud, {n_legit} legit)")
        
        # Train the model
        try:
            self.model.fit(X_train, y_train)
            self.is_fitted = True
            
            # ========== SELF-ADJUSTING THRESHOLD CALIBRATION ==========
            # Goal: Find threshold that targets 20-40% high-risk rate
            # while maximizing recall with decent precision
            
            train_proba = self.model.predict_proba(X_train)[:, 1]
            
            # DIAGNOSTIC: Check probability distribution
            unique_probs = np.unique(train_proba)
            print(f"      📊 Probability distribution: {len(unique_probs)} unique values")
            print(f"         Min={train_proba.min():.3f}, Max={train_proba.max():.3f}, Median={np.median(train_proba):.3f}")
            
            # If too few unique values, the model isn't discriminating well
            if len(unique_probs) < 10:
                print(f"      ⚠️ Low discrimination: only {len(unique_probs)} unique probability values")
            
            # Target parameters
            TARGET_HIGH_RISK_MIN = 0.20  # At least 20% should be high-risk (fraudish)
            TARGET_HIGH_RISK_MAX = 0.40  # At most 40% (avoid over-boosting)
            MIN_RECALL = 0.75  # Floor for recall
            PRECISION_CEILING = 0.70  # Don't sacrifice too much precision
            
            print(f"      🔄 Self-adjusting: Target high-risk rate 20-40%...")
            
            best_score = -1
            best_thresh = 0.5
            best_metrics = {}
            
            # Search for optimal threshold
            for thresh in np.linspace(0.3, 0.8, 51):
                pred = (train_proba >= thresh).astype(int)
                high_risk_rate = pred.mean()
                
                # Skip if outside target range
                if high_risk_rate < TARGET_HIGH_RISK_MIN or high_risk_rate > TARGET_HIGH_RISK_MAX:
                    continue
                
                # Calculate metrics
                rec = recall_score(y_train, pred, zero_division=0)
                prec = precision_score(y_train, pred, zero_division=0)
                f2 = fbeta_score(y_train, pred, beta=2, zero_division=0)
                
                # Score: Prioritize recall, penalize low precision
                # Recall weight: 2.0 (fraud detection priority)
                # Precision penalty if below ceiling
                score = rec * 2.0 + f2
                if prec < 0.30:  # Too many false positives
                    score *= 0.5
                
                if score > best_score:
                    best_score = score
                    best_thresh = thresh
                    best_metrics = {
                        'recall': rec,
                        'precision': prec,
                        'f2': f2,
                        'high_risk_rate': high_risk_rate
                    }
            
            # Fallback if no threshold in target range found
            if best_score < 0:
                print(f"      ⚠️ No threshold in target range, finding closest...")
                
                # IMPROVED FALLBACK: Find threshold closest to 30% high-risk rate
                # This is better than median which often gives 50%+ high-risk rate
                TARGET_HR = 0.30  # Target 30% high-risk (middle of 20-40% range)
                
                # Search wider range for fallback
                best_distance = float('inf')
                for thresh in np.linspace(0.1, 0.95, 86):
                    pred = (train_proba >= thresh).astype(int)
                    high_risk_rate = pred.mean()
                    
                    # Find threshold closest to target
                    distance = abs(high_risk_rate - TARGET_HR)
                    if distance < best_distance:
                        best_distance = distance
                        best_thresh = thresh
                        
                        rec = recall_score(y_train, pred, zero_division=0)
                        prec = precision_score(y_train, pred, zero_division=0)
                        f2 = fbeta_score(y_train, pred, beta=2, zero_division=0)
                        best_metrics = {
                            'recall': rec,
                            'precision': prec,
                            'f2': f2,
                            'high_risk_rate': high_risk_rate
                        }
                
                # If still can't get close to target, find EXACT threshold for 30% HR rate
                if best_distance > 0.10:  # More than 10% away from target
                    # Sort probabilities descending and pick threshold that gives 30% HR
                    sorted_proba = np.sort(train_proba)[::-1]  # Descending order
                    target_count = int(len(sorted_proba) * TARGET_HR)  # Top 30%
                    if target_count > 0 and target_count < len(sorted_proba):
                        best_thresh = sorted_proba[target_count - 1]  # Threshold at 30% mark
                    else:
                        best_thresh = sorted_proba[len(sorted_proba) // 3]  # Fallback
                    
                    pred = (train_proba >= best_thresh).astype(int)
                    actual_hr = pred.mean()
                    print(f"      ✅ Forced {TARGET_HR*100:.0f}% HR threshold={best_thresh:.3f} (actual HR={actual_hr*100:.1f}%)")
                best_metrics = {
                    'recall': recall_score(y_train, pred, zero_division=0),
                    'precision': precision_score(y_train, pred, zero_division=0),
                    'f2': fbeta_score(y_train, pred, beta=2, zero_division=0),
                    'high_risk_rate': pred.mean()
                }
            
            self.threshold = best_thresh
            
            # Report calibration results
            print(f"      ✅ BNN calibrated (threshold={self.threshold:.3f})")
            print(f"      High-risk rate: {best_metrics['high_risk_rate']*100:.1f}% (target: 20-40%)")
            print(f"      Gray zone: Recall={best_metrics['recall']*100:.1f}%, Prec={best_metrics['precision']*100:.1f}%")
            
        except Exception as e:
            print(f"      ❌ BNN training failed: {str(e)[:50]}")
            self.is_fitted = False
    
    def predict(self, X, feature_names, rule_engine, n_samples=10):
        """
        Predict high-risk probability with uncertainty estimation.
        
        Uses MC Dropout approximation: run multiple forward passes
        and measure variance as uncertainty.
        
        Parameters:
        -----------
        X : np.ndarray
            Transaction features (unscaled)
        feature_names : list
            Feature names
        rule_engine : RuleBasedEngine
            Rule engine instance
        n_samples : int
            Number of MC samples for uncertainty (default 10)
            
        Returns:
        --------
        dict with:
            - is_high_risk: np.ndarray (bool)
            - probability: np.ndarray (0-1)
            - uncertainty: np.ndarray (0-1, higher = less confident)
        """
        if not self.is_fitted:
            # Fallback: use rule score range 60-84
            return {
                'is_high_risk': np.zeros(len(X), dtype=bool),
                'probability': np.zeros(len(X)),
                'uncertainty': np.ones(len(X))  # High uncertainty
            }
        
        # Extract rule signals
        rule_signals = self.extract_rule_signals(X, feature_names, rule_engine)
        
        # Get predictions
        # For sklearn MLP, we can't do true MC dropout, so we use single prediction
        # with confidence from predict_proba
        probabilities = self.model.predict_proba(rule_signals)[:, 1]
        
        # Estimate uncertainty from probability (closer to 0.5 = higher uncertainty)
        uncertainty = 1 - 2 * np.abs(probabilities - 0.5)
        
        # Decision
        is_high_risk = probabilities >= self.threshold
        
        return {
            'is_high_risk': is_high_risk,
            'probability': probabilities,
            'uncertainty': uncertainty
        }
    
    def get_explanation(self, X_single, feature_names, rule_engine):
        """Get explanation for a single transaction's high-risk classification"""
        if not self.is_fitted:
            return "BNN not trained"
        
        rule_signals = self.extract_rule_signals(X_single.reshape(1, -1), feature_names, rule_engine)[0]
        triggered_rules = [self.rule_names[i] for i in range(len(rule_signals)) if rule_signals[i] > 0]
        
        result = self.predict(X_single.reshape(1, -1), feature_names, rule_engine)
        prob = result['probability'][0]
        unc = result['uncertainty'][0]
        
        explanation = [
            f"High-Risk Probability: {prob*100:.1f}%",
            f"Confidence: {(1-unc)*100:.1f}%",
            f"Decision: {'HIGH RISK' if result['is_high_risk'][0] else 'NORMAL'}",
            f"Triggered Rules: {', '.join(triggered_rules) if triggered_rules else 'None'}"
        ]
        
        return "\n".join(explanation)


# FRAMEWORK COMPONENT 2: AUTOENCODER ANOMALY DETECTOR
# ============================================================================
class AutoencoderAnomalyDetector:
    """
    Autoencoder for anomaly detection
    Learns to compress and reconstruct normal transactions
    High reconstruction error = anomaly
    """
    
    def __init__(self, input_dim=50, encoding_dim=10):
        self.input_dim = input_dim
        self.encoding_dim = encoding_dim
        self.encoder = None
        self.decoder = None
        self.threshold = None
        self.scaler = None
        self.is_fitted = False
    
    def _build_model(self):
        """Build simple autoencoder using sklearn's MLPRegressor"""
        from sklearn.neural_network import MLPRegressor
        
        # We'll use MLPRegressor as a simple autoencoder
        self.autoencoder = MLPRegressor(
            hidden_layer_sizes=(self.input_dim // 2, self.encoding_dim, self.input_dim // 2),
            activation='relu',
            solver='adam',
            alpha=0.001,
            max_iter=100,
            early_stopping=True,
            validation_fraction=0.1,
            random_state=RANDOM_STATE,
            verbose=False
        )
    
    def fit(self, X_normal):
        """Train autoencoder on normal (non-fraud) data only"""
        from sklearn.preprocessing import StandardScaler
        
        # Scale the data
        self.scaler = StandardScaler()
        X_scaled = self.scaler.fit_transform(X_normal)
        
        # Limit input dimension
        if X_scaled.shape[1] > self.input_dim:
            X_scaled = X_scaled[:, :self.input_dim]
        
        self._build_model()
        
        # Train autoencoder (input = output for reconstruction)
        self.autoencoder.fit(X_scaled, X_scaled)
        
        # Calculate reconstruction errors for threshold
        reconstructed = self.autoencoder.predict(X_scaled)
        errors = np.mean((X_scaled - reconstructed) ** 2, axis=1)
        
        # Set threshold at 95th percentile of normal errors
        self.threshold = np.percentile(errors, 95)
        self.is_fitted = True
        
        return self
    
    def predict(self, X):
        """Return anomaly scores (higher = more anomalous)"""
        if not self.is_fitted:
            return np.zeros(len(X))
        
        X_scaled = self.scaler.transform(X)
        if X_scaled.shape[1] > self.input_dim:
            X_scaled = X_scaled[:, :self.input_dim]
        
        reconstructed = self.autoencoder.predict(X_scaled)
        errors = np.mean((X_scaled - reconstructed) ** 2, axis=1)
        
        # Normalize to 0-1 range
        scores = np.clip(errors / (self.threshold + 1e-6), 0, 1)
        return scores


# ============================================================================
# FRAMEWORK COMPONENT 3: GRAPH FEATURE EXTRACTOR
# ============================================================================
class GraphFeatureExtractor:
    """
    Extract graph-based features from device/IP/merchant relationships
    Detects fraud rings and money laundering patterns
    """
    
    def __init__(self):
        self.device_stats = {}
        self.ip_stats = {}
        self.merchant_stats = {}
        self.user_device_map = {}
        self.is_fitted = False
    
    def fit(self, df):
        """Build graph statistics from training data"""
        # Device statistics: how many users, fraud rate
        if 'device_id' in df.columns:
            device_groups = df.groupby('device_id').agg({
                'user_id': 'nunique',
                'is_fraud': ['sum', 'count', 'mean']
            })
            device_groups.columns = ['shared_users', 'fraud_count', 'txn_count', 'fraud_rate']
            self.device_stats = device_groups.to_dict('index')
        
        # IP statistics (if available)
        if 'ip_address' in df.columns:
            ip_groups = df.groupby('ip_address').agg({
                'user_id': 'nunique',
                'is_fraud': ['sum', 'count', 'mean']
            })
            ip_groups.columns = ['shared_users', 'fraud_count', 'txn_count', 'fraud_rate']
            self.ip_stats = ip_groups.to_dict('index')
        
        # Merchant statistics
        if 'merchant_id' in df.columns:
            merchant_groups = df.groupby('merchant_id').agg({
                'is_fraud': ['sum', 'count', 'mean']
            })
            merchant_groups.columns = ['fraud_count', 'txn_count', 'fraud_rate']
            self.merchant_stats = merchant_groups.to_dict('index')
        
        # User-device mapping for connected fraudster detection
        if 'device_id' in df.columns and 'user_id' in df.columns:
            fraud_devices = set(df[df['is_fraud'] == 1]['device_id'].unique())
            self.fraud_devices = fraud_devices
        
        self.is_fitted = True
        return self
    
    def transform(self, df):
        """Add graph features to dataframe"""
        df = df.copy()
        
        if not self.is_fitted:
            # Default values if not fitted
            df['device_shared_users'] = 0
            df['device_fraud_rate'] = 0
            df['ip_shared_users'] = 0
            df['ip_fraud_rate'] = 0
            df['merchant_fraud_rate'] = 0
            df['uses_fraud_device'] = 0
            return df
        
        # Device features
        if 'device_id' in df.columns and self.device_stats:
            df['device_shared_users'] = df['device_id'].map(
                lambda x: self.device_stats.get(x, {}).get('shared_users', 0)
            ).fillna(0)
            df['device_fraud_rate'] = df['device_id'].map(
                lambda x: self.device_stats.get(x, {}).get('fraud_rate', 0)
            ).fillna(0)
            df['device_fraud_count'] = df['device_id'].map(
                lambda x: self.device_stats.get(x, {}).get('fraud_count', 0)
            ).fillna(0)
            # Flag if device was used in fraud before
            df['uses_fraud_device'] = df['device_id'].isin(
                getattr(self, 'fraud_devices', set())
            ).astype(int)
        else:
            df['device_shared_users'] = 0
            df['device_fraud_rate'] = 0
            df['device_fraud_count'] = 0
            df['uses_fraud_device'] = 0
        
        # IP features
        if 'ip_address' in df.columns and self.ip_stats:
            df['ip_shared_users'] = df['ip_address'].map(
                lambda x: self.ip_stats.get(x, {}).get('shared_users', 0)
            ).fillna(0)
            df['ip_fraud_rate'] = df['ip_address'].map(
                lambda x: self.ip_stats.get(x, {}).get('fraud_rate', 0)
            ).fillna(0)
        else:
            df['ip_shared_users'] = 0
            df['ip_fraud_rate'] = 0
        
        # Merchant features
        if 'merchant_id' in df.columns and self.merchant_stats:
            df['merchant_fraud_rate'] = df['merchant_id'].map(
                lambda x: self.merchant_stats.get(x, {}).get('fraud_rate', 0)
            ).fillna(0)
        else:
            df['merchant_fraud_rate'] = 0
        
        return df


# ============================================================================
# RISK STRATIFICATION MATRIX
# ============================================================================
def get_risk_tier(score):
    """Convert risk score (0-100) to risk tier and action"""
    if score <= 30:
        return 'LOW', 'Auto-approve', '🟢'
    elif score <= 50:
        return 'MEDIUM-LOW', 'Enhanced monitoring', '🟡'
    elif score <= 70:
        return 'MEDIUM', 'Add to watchlist', '🟠'
    elif score <= 90:
        return 'HIGH', 'Manual review', '🔴'
    else:
        return 'CRITICAL', 'Instant block', '⛔'


class UltimateFraudDetectionEngine:
    """
    ULTIMATE Fraud Detection System combining:
    - 150+ Advanced Features
    - Multi-Model Ensemble (XGBoost + LightGBM + CatBoost)
    - Graph Network Analysis
    - Bayesian Optimization with Multi-Objective Search
    - Adversarial Validation
    - Calibrated Predictions
    - SHAP Explainability
    """
    
    def __init__(self, data_path):
        print(f"📂 Loading data from {data_path}")
        # Note: Depending on file type, you might need pd.read_csv or pd.read_excel
        if data_path.endswith('.csv'):
            try:
                self.df = pd.read_csv(data_path)
            except:
                print("   ⚠️ Using Python engine for CSV parsing...")
                self.df = pd.read_csv(data_path, engine='python')
        else:
            self.df = pd.read_excel(data_path)
        
        # Force datetime conversion
        date_columns = ['timestamp', 'user_signup_date']
        for col in date_columns:
            if col in self.df.columns:
                try:
                    self.df[col] = pd.to_datetime(self.df[col], errors='coerce')
                    print(f"   ✓ Converted {col} to datetime")
                except Exception as e:
                    print(f"   ⚠ Warning: {col}: {e}")
        
        # Sort by timestamp for time-aware features
        if 'timestamp' in self.df.columns:
            self.df = self.df.sort_values('timestamp').reset_index(drop=True)
        
        # Initialize storage
        self.X_train = None
        self.X_val = None
        self.X_test = None
        self.y_train = None
        self.y_val = None
        self.y_test = None
        self.X_train_scaled = None
        self.X_val_scaled = None
        self.X_test_scaled = None
        self.feature_names = None
        self.scaler = None
        self.models = {}  # Ensemble of models
        self.optimal_threshold = 0.5
        self.feature_importance = None
        self.anomaly_scores = None
        self.graph_features = None
        
        print(f"✅ Loaded: {self.df.shape}")
        if 'is_fraud' in self.df.columns:
            print(f"💰 Fraud rate: {self.df['is_fraud'].mean()*100:.2f}%")
    
    def calculate_haversine_distance(self, lat1, lon1, lat2, lon2):
        """Haversine distance in km"""
        R = 6371.0
        try:
            lat1, lon1, lat2, lon2 = map(radians, [float(lat1), float(lon1), float(lat2), float(lon2)])
            dlon = lon2 - lon1
            dlat = lat2 - lat1
            a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
            c = 2 * atan2(sqrt(a), sqrt(1 - a))
            return R * c
        except:
            return 0.0
    
    def create_graph_features(self, df):
        """
        ADVANCED GRAPH/NETWORK FEATURES
        Creates bipartite graph: Users <-> Devices/IPs/Merchants
        """
        if not NETWORKX_AVAILABLE:
            return df
        
        print("   🕸️ Creating Graph Network features...")
        
        # User-Device Graph - Optimized with vectorized operations
        if 'user_id' in df.columns and 'device_id' in df.columns:
            # Create edge dataframe for vectorized graph construction
            edge_df = pd.DataFrame({
                'source': 'U_' + df['user_id'].astype(str),
                'target': 'D_' + df['device_id'].astype(str)
            })
            G_device = nx.from_pandas_edgelist(edge_df, 'source', 'target')
            
            # Degree centrality
            centrality = nx.degree_centrality(G_device)
            df['user_device_centrality'] = df['user_id'].apply(lambda x: centrality.get(f"U_{x}", 0))
            
            # Clustering coefficient
            clustering = nx.clustering(G_device)
            df['user_device_clustering'] = df['user_id'].apply(lambda x: clustering.get(f"U_{x}", 0))
        
        # User-IP Graph
        if 'user_id' in df.columns and 'ip_address' in df.columns:
            ip_counts = df.groupby('ip_address')['user_id'].nunique().to_dict()
            df['users_per_ip'] = df['ip_address'].map(ip_counts).fillna(1)
            df['is_suspicious_ip'] = (df['users_per_ip'] > 10).astype(int)
        
        # User-Merchant Graph
        if 'user_id' in df.columns and 'merchant_id' in df.columns:
            merchant_diversity = df.groupby('user_id')['merchant_id'].nunique().to_dict()
            df['user_merchant_diversity'] = df['user_id'].map(merchant_diversity).fillna(1)
        
        return df
    
    def advanced_feature_engineering_ultimate(self):
        """
        ULTIMATE FEATURE ENGINEERING - 150+ Features
        Combines best from both systems + new innovations
        """
        print("\n🔧 ULTIMATE Feature Engineering (150+ features)...")
        df = self.df.copy()
        
        # ============ 1. TEMPORAL FEATURES (40+) ============
        print("   ⏰ Temporal features...")
        if 'timestamp' in df.columns:
            # Basic time components
            df['hour'] = df['timestamp'].dt.hour
            df['day_of_week'] = df['timestamp'].dt.dayofweek
            df['day_of_month'] = df['timestamp'].dt.day
            df['month'] = df['timestamp'].dt.month
            df['year'] = df['timestamp'].dt.year
            df['quarter'] = df['timestamp'].dt.quarter
            df['week_of_year'] = df['timestamp'].dt.isocalendar().week
            df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
            df['is_month_start'] = df['timestamp'].dt.is_month_start.astype(int)
            df['is_month_end'] = df['timestamp'].dt.is_month_end.astype(int)
            
            # Advanced time categories
            df['is_night'] = ((df['hour'] >= 22) | (df['hour'] <= 5)).astype(int)
            df['is_business_hours'] = ((df['hour'] >= 9) & (df['hour'] <= 17)).astype(int)
            df['is_late_night'] = ((df['hour'] >= 23) | (df['hour'] <= 3)).astype(int)
            df['is_peak_hours'] = ((df['hour'] >= 10) & (df['hour'] <= 14)).astype(int)
            
            # Cyclical encoding (sine/cosine)
            df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
            df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
            df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
            df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
            df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
            df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
            df['day_of_month_sin'] = np.sin(2 * np.pi * df['day_of_month'] / 30)
            df['day_of_month_cos'] = np.cos(2 * np.pi * df['day_of_month'] / 30)
            
            # Time since epoch (normalized)
            df['timestamp_epoch'] = df['timestamp'].astype(np.int64) // 10**9
            df['timestamp_normalized'] = (df['timestamp_epoch'] - df['timestamp_epoch'].min()) / \
                                         (df['timestamp_epoch'].max() - df['timestamp_epoch'].min() + 1)
        
        # User tenure
        if 'timestamp' in df.columns and 'user_signup_date' in df.columns:
            df['user_tenure_days'] = (df['timestamp'] - df['user_signup_date']).dt.total_seconds() / 86400
            df['user_tenure_days'] = df['user_tenure_days'].fillna(0).clip(0, 10000)
            df['user_tenure_weeks'] = df['user_tenure_days'] / 7
            df['user_tenure_months'] = df['user_tenure_days'] / 30
            df['user_tenure_log'] = np.log1p(df['user_tenure_days'])
            df['user_tenure_sqrt'] = np.sqrt(df['user_tenure_days'])
            df['is_new_user'] = (df['user_tenure_days'] < 30).astype(int)
            df['is_very_new_user'] = (df['user_tenure_days'] < 7).astype(int)
            df['is_established_user'] = (df['user_tenure_days'] > 365).astype(int)
        
        # ============ 2. AMOUNT FEATURES (30+) ============
        print("   💵 Amount features...")
        if 'amount' in df.columns:
            df['amount_log'] = np.log1p(df['amount'])
            df['amount_sqrt'] = np.sqrt(df['amount'])
            df['amount_squared'] = df['amount'] ** 2
            df['amount_cubed'] = df['amount'] ** 3
            
            # Statistical transformations
            df['amount_percentile'] = df['amount'].rank(pct=True)
            df['amount_percentile_bin'] = pd.cut(df['amount_percentile'], bins=10, labels=False)
            
            # Multiple binning strategies
            df['amount_range_fine'] = pd.cut(df['amount'], 
                bins=[0, 25, 50, 100, 150, 200, 300, 500, 750, 1000, float('inf')],
                labels=range(10)).astype(float)
            
            # Round number detection
            df['is_round_amount'] = (df['amount'] % 10 == 0).astype(int)
            df['is_very_round_amount'] = (df['amount'] % 100 == 0).astype(int)
            df['is_ultra_round_amount'] = (df['amount'] % 1000 == 0).astype(int)
            
            # Global statistics
            df['amount_vs_global_mean'] = df['amount'] / (df['amount'].mean() + 1e-6)
            df['amount_vs_global_median'] = df['amount'] / (df['amount'].median() + 1e-6)
            df['amount_zscore_global'] = (df['amount'] - df['amount'].mean()) / (df['amount'].std() + 1e-6)
            
            # Rolling statistics (time-aware)
            if 'timestamp' in df.columns:
                df['amount_rolling_mean_10'] = df['amount'].rolling(window=10, min_periods=1).mean()
                df['amount_rolling_std_10'] = df['amount'].rolling(window=10, min_periods=1).std().fillna(0)
                df['amount_vs_rolling_mean'] = df['amount'] / (df['amount_rolling_mean_10'] + 1e-6)
        
        # ============ 3. USER BEHAVIORAL FEATURES (50+) ============
        print("   👤 User behavioral features...")
        
        required_cols = ['user_total_txn_amount', 'user_total_txn_count', 'user_past_fraud_count', 'days_since_last_txn']
        for col in required_cols:
            if col not in df.columns:
                df[col] = 0

        df['user_avg_txn_amount'] = df['user_total_txn_amount'] / (df['user_total_txn_count'] + 1)
        df['user_txn_frequency'] = 1 / (df['days_since_last_txn'] + 1)
        
        if 'amount' in df.columns:
            df['amount_vs_user_avg'] = df['amount'] / (df['user_avg_txn_amount'] + 1e-6)
            df['amount_deviation_user'] = np.abs(df['amount'] - df['user_avg_txn_amount'])
            
            # Per-user advanced statistics
            user_stats = df.groupby('user_id').agg({
                'amount': ['mean', 'std', 'min', 'max', 'median', 'sum', 'count', 
                          lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), 
                          lambda x: skew(x, nan_policy='omit'), 
                          lambda x: kurtosis(x, nan_policy='omit')]
            }).fillna(0)
            user_stats.columns = ['user_amount_mean', 'user_amount_std', 'user_amount_min', 
                                  'user_amount_max', 'user_amount_median', 'user_amount_sum', 
                                  'user_txn_count_new', 'user_amount_q25', 'user_amount_q75',
                                  'user_amount_skew', 'user_amount_kurt']
            df = df.merge(user_stats, on='user_id', how='left')
            
            # Z-scores and IQR
            df['amount_zscore'] = (df['amount'] - df['user_amount_mean']) / (df['user_amount_std'] + 1e-6)
            df['amount_zscore'] = df['amount_zscore'].fillna(0).clip(-10, 10)
            df['amount_zscore_abs'] = np.abs(df['amount_zscore'])
            df['amount_is_outlier'] = (df['amount_zscore_abs'] > 3).astype(int)
            
            # IQR-based outlier detection
            df['user_iqr'] = df['user_amount_q75'] - df['user_amount_q25']
            df['amount_iqr_outlier'] = ((df['amount'] < (df['user_amount_q25'] - 1.5 * df['user_iqr'])) | 
                                     (df['amount'] > (df['user_amount_q75'] + 1.5 * df['user_iqr']))).astype(int)
            
            # Coefficient of variation
            df['user_amount_cv'] = df['user_amount_std'] / (df['user_amount_mean'] + 1e-6)
        
        # Fraud history
        df['user_fraud_ratio'] = df['user_past_fraud_count'] / (df['user_total_txn_count'] + 1)
        df['user_has_fraud_history'] = (df['user_past_fraud_count'] > 0).astype(int)
        df['user_fraud_count_log'] = np.log1p(df['user_past_fraud_count'])
        
        # Advanced velocity features
        if 'user_id' in df.columns and 'timestamp' in df.columns:
            df['time_since_prev_txn'] = df.groupby('user_id')['timestamp'].diff().dt.total_seconds().fillna(99999)
            df['is_rapid_txn'] = (df['time_since_prev_txn'] < 60).astype(int)
            df['is_burst_txn'] = (df['time_since_prev_txn'] < 10).astype(int)
            df['time_since_prev_log'] = np.log1p(df['time_since_prev_txn'])
            
            # Running velocity metrics
            df['user_running_velocity'] = df.groupby('user_id')['time_since_prev_txn'].transform(
                lambda x: x.rolling(5, min_periods=1).mean()
            )
        
        # ============ 4. LOCATION FEATURES (40+) ============
        print("   🌍 Location features...")
        
        loc_cols = ['billing_country', 'shipping_country', 'user_country', 'merchant_country']
        for col in loc_cols:
            if col not in df.columns:
                df[col] = 'UNKNOWN'

        # Country matches
        df['billing_shipping_match'] = (df['billing_country'] == df['shipping_country']).astype(int)
        df['user_billing_match'] = (df['user_country'] == df['billing_country']).astype(int)
        df['user_merchant_match'] = (df['user_country'] == df['merchant_country']).astype(int)
        df['all_countries_match'] = ((df['billing_shipping_match']) & 
                                     (df['user_billing_match']) & 
                                     (df['user_merchant_match'])).astype(int)
        
        df['country_mismatch_count'] = 4 - (df['billing_shipping_match'] + 
                                            df['user_billing_match'] + 
                                            df['user_merchant_match'] + 
                                            (df['billing_country'] == df['merchant_country']).astype(int))
        
        # High-risk countries
        high_risk_countries = ['BR', 'NG', 'RU', 'CN', 'VE', 'IR', 'KP', 'SD', 'SY', 'PK']
        for col in ['billing_country', 'shipping_country', 'merchant_country', 'user_country']:
            df[f'is_high_risk_{col}'] = df[col].isin(high_risk_countries).astype(int)
        
        df['high_risk_country_count'] = sum([df[f'is_high_risk_{col}'] for col in 
                                             ['billing_country', 'shipping_country', 
                                              'merchant_country', 'user_country']])
        
        # Country-level statistics
        country_stats = df.groupby('billing_country').agg({
            'amount': ['mean', 'median', 'std'],
            'is_fraud': ['mean', 'sum', 'count']
        }).fillna(0)
        country_stats.columns = ['country_avg_amount', 'country_median_amount', 
                                 'country_std_amount', 'country_fraud_rate', 
                                 'country_fraud_count', 'country_txn_count']
        df = df.merge(country_stats, on='billing_country', how='left')
        
        df['amount_vs_country_avg'] = df['amount'] / (df['country_avg_amount'] + 1e-6)
        df['country_is_high_fraud'] = (df['country_fraud_rate'] > 0.05).astype(int)
        
        # ============ 5. MERCHANT FEATURES (25+) ============
        print("   🏪 Merchant features...")
        
        if 'merchant_id' in df.columns:
            merchant_stats = df.groupby('merchant_id').agg({
                'amount': ['mean', 'std', 'min', 'max', 'median', 'count'],
                'is_fraud': ['mean', 'sum']
            }).fillna(0)
            merchant_stats.columns = ['merchant_avg_amount', 'merchant_std_amount', 
                                      'merchant_min_amount', 'merchant_max_amount',
                                      'merchant_median_amount', 'merchant_txn_count',
                                      'merchant_fraud_rate', 'merchant_fraud_count']
            df = df.merge(merchant_stats, on='merchant_id', how='left')
            
            df['amount_vs_merchant_avg'] = df['amount'] / (df['merchant_avg_amount'] + 1e-6)
            df['merchant_risk_score'] = df['merchant_fraud_rate'] * 100
            df['merchant_is_high_risk'] = (df['merchant_fraud_rate'] > 0.1).astype(int)
            
            # Merchant Z-score
            df['amount_zscore_merchant'] = ((df['amount'] - df['merchant_avg_amount']) / 
                                            (df['merchant_std_amount'] + 1e-6)).fillna(0).clip(-10, 10)
        
        # ============ 6. DEVICE & CHANNEL FEATURES (20+) ============
        print("   📱 Device & Channel features...")
        
        if 'device_type' in df.columns:
            df['is_mobile'] = (df['device_type'] == 'mobile').astype(int)
            df['is_desktop'] = (df['device_type'] == 'desktop').astype(int)
        
        if 'channel' in df.columns:
            channel_stats = df.groupby('channel').agg({
                'amount': ['mean', 'median'],
                'is_fraud': 'mean'
            }).fillna(0)
            channel_stats.columns = ['channel_avg_amount', 'channel_median_amount', 'channel_fraud_rate']
            df = df.merge(channel_stats, on='channel', how='left')
            
            df['amount_vs_channel_avg'] = df['amount'] / (df['channel_avg_amount'] + 1e-6)
            df['is_high_risk_channel'] = (df['channel_fraud_rate'] > 0.05).astype(int)
        
        # ============ 7. GRAPH NETWORK FEATURES (10+) ============
        df = self.create_graph_features(df)
        
        # ============ 8. RISK FLAGS & COMPOSITE SCORES (30+) ============
        print("   ⚠️ Risk flags & composite scores...")
        
        if 'kyc_status' in df.columns:
            df['kyc_risk_flag'] = (df['kyc_status'] != 'verified').astype(int)
        
        # Multi-factor risk score
        df['composite_risk_score'] = (
            df.get('high_risk_country_count', 0) * 2 +
            df.get('kyc_risk_flag', 0) * 3 +
            df.get('user_fraud_ratio', 0) * 10 +
            df.get('is_rapid_txn', 0) * 2 +
            df.get('is_night', 0) * 1 +
            (df['amount'] > df['amount'].quantile(0.9)).astype(int) * 2
        )
        
        # Suspicious patterns
        df['suspicious_new_user_high_amount'] = (
            (df.get('user_tenure_days', 999) < 30) * (df['amount'] > df['amount'].quantile(0.75)).astype(int)
        )
        
        df['night_international_high_amount'] = (
            df.get('is_night', 0) * (df.get('country_mismatch_count', 0) > 0) *
            (df['amount'] > df['amount'].quantile(0.7)).astype(int)
        )
        
        # ============ 9. ENHANCED FEATURES FOR BALANCE (NEW) ============
        print("   🎯 Enhanced balance features...")
        
        # === 9.1 Percentile-Based Features ===
        if 'amount' in df.columns and 'user_id' in df.columns:
            # Percentile rank of amount within user's transactions
            df['amount_user_percentile'] = df.groupby('user_id')['amount'].rank(pct=True)
            
            # Is amount in user's top 5%, 10%, 25%?
            df['amount_user_top5pct'] = (df['amount_user_percentile'] > 0.95).astype(int)
            df['amount_user_top10pct'] = (df['amount_user_percentile'] > 0.90).astype(int)
            df['amount_user_top25pct'] = (df['amount_user_percentile'] > 0.75).astype(int)
        
        # === 9.2 Enhanced Behavioral Stability ===
        if 'user_amount_std' in df.columns and 'user_amount_mean' in df.columns:
            # Coefficient of variation (already exists, but add more)
            df['user_spending_stability'] = 1 / (df['user_amount_cv'] + 1)  # Higher = more stable
            
            # Amount range relative to mean
            if 'user_amount_max' in df.columns and 'user_amount_min' in df.columns:
                df['user_amount_range_ratio'] = ((df['user_amount_max'] - df['user_amount_min']) / 
                                                 (df['user_amount_mean'] + 1e-6))
        
        # === 9.3 Enhanced Domain-Specific Risk Signals ===
        if 'amount' in df.columns:
            # Just-below-threshold amounts (common fraud pattern)
            df['is_just_below_100'] = ((df['amount'] >= 95) & (df['amount'] < 100)).astype(int)
            df['is_just_below_500'] = ((df['amount'] >= 490) & (df['amount'] < 500)).astype(int)
            df['is_just_below_1000'] = ((df['amount'] >= 990) & (df['amount'] < 1000)).astype(int)
            df['is_just_below_threshold'] = (df['is_just_below_100'] | 
                                             df['is_just_below_500'] | 
                                             df['is_just_below_1000']).astype(int)
            
            # Amount spike detection (more sensitive)
            if 'user_amount_mean' in df.columns and 'user_amount_std' in df.columns:
                df['amount_spike_2std'] = (df['amount'] > (df['user_amount_mean'] + 2 * df['user_amount_std'])).astype(int)
                df['amount_spike_3std'] = (df['amount'] > (df['user_amount_mean'] + 3 * df['user_amount_std'])).astype(int)
        
        # === 9.4 First-Transaction Flags ===
        if 'merchant_id' in df.columns and 'user_id' in df.columns:
            # First transaction with this merchant for this user
            df['user_merchant_pair'] = df['user_id'].astype(str) + '_' + df['merchant_id'].astype(str)
            df['is_first_merchant_txn'] = (~df.duplicated(subset='user_merchant_pair', keep='first')).astype(int)
            
        # ============ 10. ENHANCED FEATURE INTERACTIONS ============
        print("   🔗 Feature interactions...")
        
        if 'amount' in df.columns and 'user_tenure_days' in df.columns:
            df['amount_x_tenure'] = df['amount'] * df['user_tenure_days']
            df['amount_div_tenure'] = df['amount'] / (df['user_tenure_days'] + 1)
        
        if 'merchant_fraud_rate' in df.columns and 'country_fraud_rate' in df.columns:
            df['merchant_x_country_risk'] = df['merchant_fraud_rate'] * df['country_fraud_rate']
        
        # === NEW: High-Impact Interactions ===
        # Amount × user fraud history (powerful interaction)
        if 'amount' in df.columns and 'user_fraud_ratio' in df.columns:
            df['amount_x_fraud_ratio'] = df['amount'] * df['user_fraud_ratio']
            df['amount_x_has_fraud_hist'] = df['amount'] * df.get('user_has_fraud_history', 0)
        
        # New user × high risk indicators
        if 'is_new_user' in df.columns:
            df['new_user_x_high_risk_country'] = df['is_new_user'] * df.get('high_risk_country_count', 0)
            df['new_user_x_night'] = df['is_new_user'] * df.get('is_night', 0)
            df['new_user_x_country_mismatch'] = df['is_new_user'] * df.get('country_mismatch_count', 0)
            df['new_user_x_high_amount'] = df['is_new_user'] * (df['amount'] > df['amount'].quantile(0.75)).astype(int)
        
        # Night × country mismatch (common fraud pattern)
        if 'is_night' in df.columns and 'country_mismatch_count' in df.columns:
            df['night_x_country_mismatch'] = df['is_night'] * df['country_mismatch_count']
        
        # Amount zscore × tenure (new users with unusual amounts)
        if 'amount_zscore' in df.columns and 'user_tenure_days' in df.columns:
            df['zscore_div_tenure'] = df['amount_zscore'] / (df['user_tenure_log'] + 1)
        
        # ============ 11. PATTERN-ALIGNED FEATURES (NEW - Maps to Dataset Fraud Patterns) ============
        print("   🎯 Pattern-aligned features (matching fraud injection logic)...")
        
        # === Pattern 10 & 14: New User / First Transaction Detection ===
        if 'user_signup_date' in df.columns and 'timestamp' in df.columns:
            try:
                # Convert if needed
                if not pd.api.types.is_datetime64_any_dtype(df['user_signup_date']):
                    df['user_signup_date'] = pd.to_datetime(df['user_signup_date'], errors='coerce')
                if not pd.api.types.is_datetime64_any_dtype(df['timestamp']):
                    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
                
                # Days since signup
                df['days_since_signup'] = (df['timestamp'] - df['user_signup_date']).dt.days.fillna(365)
                df['is_very_new_user'] = (df['days_since_signup'] < 7).astype(int)
                df['is_new_user_7_30'] = ((df['days_since_signup'] >= 7) & (df['days_since_signup'] < 30)).astype(int)
                
                # Pattern 10: New user + high value (fraud injection: days < 7 AND amount > 500)
                df['is_new_user_high_value'] = (
                    (df['days_since_signup'] < 7) & (df['amount'] > 500)
                ).astype(int)
            except Exception as e:
                print(f"      ⚠️ Days since signup calculation failed: {e}")
        
        # Pattern 14: First transaction ever (fraud injection: total_transactions == 0 AND amount > 1000)
        if 'user_total_txn_count' in df.columns:
            df['is_first_ever_txn'] = (df['user_total_txn_count'] == 0).astype(int)
            df['is_first_txn_high_amount'] = (
                (df['user_total_txn_count'] == 0) & (df['amount'] > 1000)
            ).astype(int)
            df['is_first_txn_medium_amount'] = (
                (df['user_total_txn_count'] == 0) & (df['amount'] > 500)
            ).astype(int)
        
        # === Pattern 13: Late Night High Value ===
        # Fraud injection: 2 <= hour <= 5 AND amount > 2000
        if 'hour' in df.columns:
            df['is_late_night'] = ((df['hour'] >= 2) & (df['hour'] <= 5)).astype(int)
            df['is_late_night_high_value'] = (
                ((df['hour'] >= 2) & (df['hour'] <= 5)) & (df['amount'] > 2000)
            ).astype(int)
            df['is_late_night_medium_value'] = (
                ((df['hour'] >= 2) & (df['hour'] <= 5)) & (df['amount'] > 1000)
            ).astype(int)
        
        # === Pattern 8: Geo Mismatch Combined ===
        # Fraud injection: billing_country IN HIGH_RISK AND billing_country != user_country
        if 'is_high_risk_billing_country' in df.columns and 'user_billing_match' in df.columns:
            df['is_geo_mismatch_fraud'] = (
                (df['is_high_risk_billing_country'] == 1) & 
                (df['user_billing_match'] == 0)
            ).astype(int)
        
        # === Pattern 1: Stolen Card ===
        # Fraud injection: amount > 800 AND billing_country IN HIGH_RISK AND NOT in user's visited countries
        if 'is_high_risk_billing_country' in df.columns:
            df['is_stolen_card_pattern'] = (
                (df['amount'] > 800) & 
                (df.get('is_high_risk_billing_country', 0) == 1) &
                (df.get('user_billing_match', 1) == 0)
            ).astype(int)
        
        # === Pattern 16: Micro Testing ===
        # Fraud injection: amount < 5 AND total_transactions < 3
        if 'user_total_txn_count' in df.columns:
            df['is_micro_txn'] = (df['amount'] < 5).astype(int)
            df['is_micro_new_user'] = (
                (df['amount'] < 5) & (df['user_total_txn_count'] < 3)
            ).astype(int)
        
        # === Pattern 3 & 15: High-Risk Merchant Categories ===
        # Fraud injection: category IN [Gift Cards, Crypto, Money Transfer, Gambling]
        if 'merchant_category' in df.columns:
            high_risk_cats = ['Gift Cards', 'Crypto', 'Money Transfer', 'Gambling', 'Adult']
            df['is_high_risk_merchant_cat'] = df['merchant_category'].isin(high_risk_cats).astype(int)
            
            # Pattern 15: Gift card + new user or high amount
            df['is_gift_card_risky'] = (
                (df['merchant_category'].isin(['Gift Cards', 'Crypto', 'Money Transfer'])) &
                ((df['amount'] > 300) | (df.get('user_total_txn_count', 100) < 5))
            ).astype(int)
        
        # === Pattern 12: Device Sharing ===
        # Fraud injection: len(device.users_sharing) > 3
        if 'device_id' in df.columns and 'user_id' in df.columns:
            device_user_counts = df.groupby('device_id')['user_id'].transform('nunique')
            df['device_user_count'] = device_user_counts
            df['is_shared_device'] = (df['device_user_count'] > 3).astype(int)
            df['is_very_shared_device'] = (df['device_user_count'] > 5).astype(int)
        
        # === Pattern 2: Velocity Attack (Enhanced) ===
        # Fraud injection: days_since_last_txn < 0.007 (< 10 minutes)
        if 'days_since_last_txn' in df.columns:
            df['is_velocity_attack'] = (df['days_since_last_txn'] < 0.007).astype(int)  # < 10 min
            df['is_very_rapid'] = (df['days_since_last_txn'] < 0.0035).astype(int)  # < 5 min
        
        # === Pattern 17: Card Testing Sequence ===
        # Fraud injection: last 2 txns < $15 AND current > $300
        if 'user_id' in df.columns and 'timestamp' in df.columns:
            # Sort by user and time to get previous amounts
            df = df.sort_values(['user_id', 'timestamp']).reset_index(drop=True)
            df['prev_amount_1'] = df.groupby('user_id')['amount'].shift(1)
            df['prev_amount_2'] = df.groupby('user_id')['amount'].shift(2)
            
            # Card testing pattern
            df['is_card_testing'] = (
                (df['prev_amount_1'].fillna(999) < 15) & 
                (df['prev_amount_2'].fillna(999) < 15) & 
                (df['amount'] > 300)
            ).astype(int)
            
            # Amount spike after small transactions
            df['amount_spike_ratio'] = df['amount'] / (df['prev_amount_1'].fillna(1) + 1)
            df['is_amount_spike_20x'] = (df['amount_spike_ratio'] > 20).astype(int)
            df['is_amount_spike_50x'] = (df['amount_spike_ratio'] > 50).astype(int)
        
        # === Pattern 11: Impossible Travel ===
        # Fraud injection: speed > 900 km/h
        if all(col in df.columns for col in ['latitude', 'longitude', 'user_id', 'timestamp']):
            try:
                df = df.sort_values(['user_id', 'timestamp']).reset_index(drop=True)
                df['prev_lat'] = df.groupby('user_id')['latitude'].shift(1)
                df['prev_lon'] = df.groupby('user_id')['longitude'].shift(1)
                df['prev_time'] = df.groupby('user_id')['timestamp'].shift(1)
                
                # Approximate distance using Haversine formula
                def haversine_vectorized(lat1, lon1, lat2, lon2):
                    R = 6371  # Earth's radius in km
                    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
                    dlat = lat2 - lat1
                    dlon = lon2 - lon1
                    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
                    c = 2 * np.arcsin(np.sqrt(np.clip(a, 0, 1)))
                    return R * c
                
                mask = df['prev_lat'].notna() & df['prev_lon'].notna()
                df.loc[mask, 'travel_distance_km'] = haversine_vectorized(
                    df.loc[mask, 'prev_lat'], df.loc[mask, 'prev_lon'],
                    df.loc[mask, 'latitude'], df.loc[mask, 'longitude']
                )
                df['travel_distance_km'] = df['travel_distance_km'].fillna(0)
                
                # Time difference in hours
                df['time_diff_hours'] = (df['timestamp'] - df['prev_time']).dt.total_seconds() / 3600
                df['time_diff_hours'] = df['time_diff_hours'].fillna(999)
                
                # Speed in km/h
                df['travel_speed_kmh'] = df['travel_distance_km'] / (df['time_diff_hours'] + 0.001)
                df['is_impossible_travel'] = (df['travel_speed_kmh'] > 900).astype(int)
                df['is_suspicious_travel'] = (df['travel_speed_kmh'] > 500).astype(int)
                
                # Cleanup temp columns
                df.drop(['prev_lat', 'prev_lon', 'prev_time'], axis=1, inplace=True, errors='ignore')
            except Exception as e:
                print(f"      ⚠️ Impossible travel calculation failed: {e}")
                df['is_impossible_travel'] = 0
                df['is_suspicious_travel'] = 0
        
        # === Pattern 4: Account Takeover Score ===
        # Fraud injection: device != primary_device AND failed_logins > 2 AND late night
        if 'failed_login_attempts_last_24h' in df.columns:
            df['account_takeover_score'] = (
                (df['failed_login_attempts_last_24h'] > 2).astype(int) * 3 +
                df.get('is_late_night', 0) * 2 +
                df.get('is_night', 0) * 1
            )
            df['is_account_takeover_risk'] = (df['account_takeover_score'] >= 4).astype(int)
        
        # === COMPOSITE PATTERN SCORES ===
        print("   📊 Creating composite pattern scores...")
        
        # Master fraud pattern score (combines all pattern indicators)
        pattern_cols = [
            'is_new_user_high_value', 'is_first_txn_high_amount', 'is_late_night_high_value',
            'is_geo_mismatch_fraud', 'is_stolen_card_pattern', 'is_micro_new_user',
            'is_gift_card_risky', 'is_shared_device', 'is_velocity_attack',
            'is_card_testing', 'is_impossible_travel', 'is_account_takeover_risk'
        ]
        
        # Sum of all pattern flags (each pattern adds 1)
        df['pattern_match_count'] = 0
        for col in pattern_cols:
            if col in df.columns:
                df['pattern_match_count'] += df[col].fillna(0)
        
        # Weighted pattern score
        pattern_weights = {
            'is_impossible_travel': 10,
            'is_card_testing': 8,
            'is_account_takeover_risk': 7,
            'is_stolen_card_pattern': 6,
            'is_first_txn_high_amount': 5,
            'is_new_user_high_value': 4,
            'is_late_night_high_value': 4,
            'is_geo_mismatch_fraud': 4,
            'is_velocity_attack': 3,
            'is_gift_card_risky': 3,
            'is_shared_device': 2,
            'is_micro_new_user': 2
        }
        
        df['weighted_pattern_score'] = 0
        for col, weight in pattern_weights.items():
            if col in df.columns:
                df['weighted_pattern_score'] += df[col].fillna(0) * weight
        
        # High risk flag based on pattern score
        df['is_multi_pattern_risk'] = (df['pattern_match_count'] >= 2).astype(int)
        df['is_high_pattern_score'] = (df['weighted_pattern_score'] >= 8).astype(int)
        
        # ============ 12. ONE-HOT ENCODING ============
        print("   🔤 One-hot encoding...")
        
        low_card_categoricals = [
            'currency', 'transaction_type', 'channel', 'payment_method',
            'device_type', 'browser', 'merchant_category', 'kyc_status'
        ]
        
        for col in low_card_categoricals:
            if col in df.columns:
                try:
                    dummies = pd.get_dummies(df[col], prefix=col, drop_first=True, dtype=int)
                    df = pd.concat([df, dummies], axis=1)
                except Exception as e:
                    print(f"      ✗ {col}: {e}")
        
        # ============ 12. FEATURE SELECTION ============
        print("   ✂️ Feature selection...")
        
        high_card_categoricals = [
            'user_id', 'merchant_id', 'device_id', 'ip_address',
            'email', 'billing_city', 'shipping_city', 'device_os',
            'user_agent', 'browser_version', 'merchant_name',
            'user_merchant_pair'  # Added from new features
        ]
        
        exclude_cols = [
            'transaction_id', 'timestamp', 'user_signup_date',
            'is_fraud', 'fraud_type', 'latitude', 'longitude'
        ]
        exclude_cols.extend(low_card_categoricals)
        exclude_cols.extend(high_card_categoricals)
        
        feature_cols = [col for col in df.columns 
                        if col not in exclude_cols and pd.api.types.is_numeric_dtype(df[col])]
        
        print(f"   ✅ Total features: {len(feature_cols)}")
        
        return df, feature_cols
    
    def prepare_data_with_time_aware_split(self):
        """Time-aware train/val/test split (70:15:15)"""
        print("\n📊 Preparing time-aware data splits (70:15:15)...")
        
        df_engineered, feature_cols = self.advanced_feature_engineering_ultimate()
        self.feature_names = feature_cols
        
        X = df_engineered[feature_cols].copy()
        y = df_engineered['is_fraud'].copy()
        
        X = X.fillna(0).replace([np.inf, -np.inf], 0)
        
        print(f"\n📊 Feature matrix: {X.shape}")
        print(f"💰 Fraud rate: {y.mean()*100:.2f}%")
        
        # Time-aware split (70:15:15)
        n = len(X)
        train_end = int(n * 0.70)
        val_end = int(n * 0.85)
        
        X_train = X.iloc[:train_end]
        X_val = X.iloc[train_end:val_end]
        X_test = X.iloc[val_end:]
        y_train = y.iloc[:train_end]
        y_val = y.iloc[train_end:val_end]
        y_test = y.iloc[val_end:]
        
        self.X_train, self.X_val, self.X_test = X_train, X_val, X_test
        self.y_train, self.y_val, self.y_test = y_train, y_val, y_test
        
        # Multi-stage scaling: RobustScaler + QuantileTransformer
        print("\n🔄 Applying hybrid scaling (Robust + Quantile)...")
        self.scaler = RobustScaler()
        X_train_robust = self.scaler.fit_transform(self.X_train)
        X_val_robust = self.scaler.transform(self.X_val)
        X_test_robust = self.scaler.transform(self.X_test)
        
        # Apply QuantileTransformer for Gaussian-like distribution
        self.quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=RANDOM_STATE)
        self.X_train_scaled = self.quantile_transformer.fit_transform(X_train_robust)
        self.X_val_scaled = self.quantile_transformer.transform(X_val_robust)
        self.X_test_scaled = self.quantile_transformer.transform(X_test_robust)
        
        print(f"\n✅ Data splits:")
        print(f"   Train: {self.X_train.shape} (fraud: {self.y_train.mean()*100:.2f}%)")
        print(f"   Val:   {self.X_val.shape} (fraud: {self.y_val.mean()*100:.2f}%)")
        print(f"   Test:  {self.X_test.shape} (fraud: {self.y_test.mean()*100:.2f}%)")
        
        return self.X_train_scaled, self.X_val_scaled, self.X_test_scaled
    
    def verify_class_distribution(self):
        """
        Verify class distribution after time-aware split.
        
        Note: No resampling is performed because:
        1. The dataset has ~24% fraud rate (already reasonably balanced)
        2. Time-aware split preserves natural distribution across train/val/test
        3. Class imbalance is handled via scale_pos_weight in boosting models
        """
        print(f"\n📊 Class Distribution Verification...")
        print(f"   Data: {self.X_train_scaled.shape}, Fraud: {self.y_train.mean()*100:.2f}%")
        print("   ✓ Using existing distribution (balanced via model weights)")
    
    # REMOVED: Isolation Forest anomaly detection - no longer used in pipeline
    # def detect_anomalies(self):
    #     """Isolation Forest for anomaly reporting (No Filtering)"""
    #     print("\\n🔍 Running Isolation Forest for anomaly detection...")
    #     
    #     iso_forest = IsolationForest(
    #         contamination=0.1,
    #         random_state=RANDOM_STATE,
    #         n_jobs=-1
    #     )
    #     
    #     # Fit on training data
    #     anomaly_labels = iso_forest.fit_predict(self.X_train_scaled)
    #     self.anomaly_scores = iso_forest.score_samples(self.X_train_scaled)
    #     
    #     n_anomalies = (anomaly_labels == -1).sum()
    #     print(f"   Detected {n_anomalies} anomalies in training set (Reporting Only)")
    
    def train_ensemble_models(self):
        """Train ensemble of XGBoost, LightGBM, CatBoost"""
        print("\n" + "="*80)
        print("🎯 TRAINING ENSEMBLE MODELS")
        print("="*80)
        
        fraud_ratio = np.sum(self.y_train == 0) / np.sum(self.y_train == 1)
        
        # === 1. XGBoost ===
        print("\n[1/3] Training XGBoost...")
        xgb_params = {
            'objective': 'binary:logistic',
            'eval_metric': 'aucpr',
            'scale_pos_weight': fraud_ratio,
            'n_estimators': 1600,
            'learning_rate': 0.062,
            'max_depth': 12,
            'min_child_weight': 6,
            'subsample': 0.715,
            'colsample_bytree': 0.707,
            'gamma': 0.971,
            'reg_alpha': 1.003,
            'reg_lambda': 2.545,
            'random_state': RANDOM_STATE,
            'device': 'cuda',           # ADD THIS (Forces NVIDIA GPU)
            'n_jobs': -1,
            'tree_method': 'hist'
        }
        
        self.models['xgboost'] = xgb.XGBClassifier(**xgb_params)
        self.models['xgboost'].fit(
            self.X_train_scaled, self.y_train,
            eval_set=[(self.X_val_scaled, self.y_val)],
            early_stopping_rounds=50,
            verbose=False
        )
        print("   ✓ XGBoost trained")
        
        # === 2. LightGBM ===
        if LIGHTGBM_AVAILABLE:
            print("\n[2/3] Training LightGBM...")
            lgb_params = {
                'objective': 'binary',
                'metric': 'auc',
                'scale_pos_weight': fraud_ratio,
                'n_estimators': 1500,
                'learning_rate': 0.05,
                'max_depth': 10,
                'num_leaves': 31,
                'subsample': 0.8,
                'colsample_bytree': 0.8,
                'random_state': RANDOM_STATE,
                'n_jobs': -1,
                'verbosity': -1
            }
            
            self.models['lightgbm'] = lgb.LGBMClassifier(**lgb_params)
            self.models['lightgbm'].fit(
                self.X_train_scaled, self.y_train,
                eval_set=[(self.X_val_scaled, self.y_val)],
                callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)]
            )
            print("   ✓ LightGBM trained")
        
        # === 3. CatBoost ===
        if CATBOOST_AVAILABLE:
            print("\n[3/3] Training CatBoost...")
            cat_params = {
                'loss_function': 'Logloss',
                'eval_metric': 'AUC',
                'scale_pos_weight': fraud_ratio,
                'iterations': 1500,
                'learning_rate': 0.05,
                'depth': 10,
                'random_seed': RANDOM_STATE,
                'task_type': 'CPU',  # Changed from GPU to CPU
                'verbose': False
            }
            
            self.models['catboost'] = cb.CatBoostClassifier(**cat_params)
            self.models['catboost'].fit(
                self.X_train_scaled, self.y_train,
                eval_set=(self.X_val_scaled, self.y_val),
                early_stopping_rounds=50,
                verbose=False
            )
            print("   ✓ CatBoost trained")
        
        print(f"\n✅ Trained {len(self.models)} models in ensemble")
        
        # Calculate feature importance
        if 'xgboost' in self.models:
            self.feature_importance = pd.DataFrame({
                'feature': self.feature_names,
                'importance': self.models['xgboost'].feature_importances_
            }).sort_values('importance', ascending=False)
    
    def optimize_ensemble_weights_bayesian(self, n_trials=100):
        """
        BAYESIAN OPTIMIZATION FOR ENSEMBLE WEIGHTS
        
        Objective: Maximize F2 Score (recall-weighted) with Precision >= 60%
        
        F2 Score Formula: F2 = (1 + 4) × (P × R) / (4P + R)
        - Recall is weighted 4x more than precision
        - Precision floor of 60% prevents excessive false positives
        
        Uses known optimal weights as warm start for faster convergence.
        """
        if not OPTUNA_AVAILABLE:
            print("❌ Optuna not installed - using equal weights")
            return None
        
        print("\n" + "="*80)
        print("🧬 BAYESIAN OPTIMIZATION FOR ENSEMBLE WEIGHTS")
        print("   Objective: Maximize F2 Score | Constraint: Precision >= 60%")
        print("="*80)
        
        # ============= WARM START WEIGHTS =============
        # Best weights from previous F1-focused optimization (used as starting point)
        WARM_START_WEIGHTS = {
            'xgboost': 0.5329,
            'lightgbm': 0.1890,
            'catboost': 0.2781
        }
        
        # Get base predictions from all models
        base_predictions = {}
        for name, model in self.models.items():
            base_predictions[name] = model.predict_proba(self.X_val_scaled)[:, 1]
        
        model_names = list(base_predictions.keys())
        
        # Precision floor constraint
        PRECISION_FLOOR = 0.60
        
        def objective(trial):
            """
            Objective: Maximize F2 Score with Precision >= 60% constraint
            
            F2 heavily weights recall (catches more fraud) while the
            precision floor prevents flagging too many legitimate transactions.
            """
            # Suggest weights that sum to 1
            if len(model_names) == 1:
                weights = [1.0]
            elif len(model_names) == 2:
                w1 = trial.suggest_float('weight_0', 0.1, 0.9)
                weights = [w1, 1 - w1]
            else:
                # For 3 models, sample 2 weights and derive the third
                w1 = trial.suggest_float('weight_0', 0.1, 0.8)
                w2 = trial.suggest_float('weight_1', 0.1, 0.8)
                # Normalize to sum to 1
                total = w1 + w2
                if total >= 0.95:
                    w1 = w1 / (total + 0.1)
                    w2 = w2 / (total + 0.1)
                w3 = max(0.05, 1.0 - w1 - w2)
                weights = [w1, w2, w3][:len(model_names)]
            
            # Normalize weights to sum exactly to 1
            weights = np.array(weights)
            weights = weights / weights.sum()
            
            # Compute weighted ensemble prediction
            ensemble_pred = np.zeros(len(self.y_val))
            for i, name in enumerate(model_names):
                ensemble_pred += weights[i] * base_predictions[name]
            
            # Try multiple thresholds for robustness
            best_score = -1.0
            for thresh in [0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]:
                y_pred = (ensemble_pred > thresh).astype(int)
                
                # Calculate metrics
                f2 = fbeta_score(self.y_val, y_pred, beta=2)
                prec = precision_score(self.y_val, y_pred, zero_division=0)
                
                # ============= F2 + PRECISION CONSTRAINT =============
                # If precision is below floor, apply harsh penalty
                if prec < PRECISION_FLOOR:
                    # Penalty scales with how far below the floor we are
                    penalty = (PRECISION_FLOOR - prec) * 2.0
                    score = f2 - penalty
                else:
                    # Pure F2 optimization when precision is acceptable
                    # Small bonus for precision above floor (up to 5%)
                    precision_bonus = min(0.05, (prec - PRECISION_FLOOR) * 0.1)
                    score = f2 + precision_bonus
                
                best_score = max(best_score, score)
            
            return best_score
        
        print(f"🔍 Optimizing weights for {len(model_names)} models...")
        print(f"   Models: {', '.join(model_names)}")
        print(f"   Precision Floor: {PRECISION_FLOOR*100:.0f}%")
        
        study = optuna.create_study(
            direction='maximize',
            sampler=TPESampler(seed=RANDOM_STATE, n_startup_trials=20)
        )
        
        # ============= WARM START =============
        # Enqueue trial with known good weights for faster convergence
        if len(model_names) >= 2:
            try:
                warm_params = {}
                if 'xgboost' in WARM_START_WEIGHTS and 'lightgbm' in WARM_START_WEIGHTS:
                    warm_params['weight_0'] = WARM_START_WEIGHTS.get('xgboost', 0.33)
                    warm_params['weight_1'] = WARM_START_WEIGHTS.get('lightgbm', 0.33)
                    study.enqueue_trial(warm_params)
                    print(f"   🔥 Warm start: XGB={warm_params['weight_0']:.4f}, LGB={warm_params['weight_1']:.4f}")
            except Exception as e:
                print(f"   ⚠️ Warm start skipped: {e}")
        
        study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
        
        # Extract optimal weights
        optimal_weights = []
        if len(model_names) == 1:
            optimal_weights = [1.0]
        elif len(model_names) == 2:
            w1 = study.best_params['weight_0']
            optimal_weights = [w1, 1 - w1]
        else:
            w1 = study.best_params['weight_0']
            w2 = study.best_params['weight_1']
            w3 = max(0.05, 1.0 - w1 - w2)
            optimal_weights = [w1, w2, w3][:len(model_names)]
        
        # Normalize
        optimal_weights = np.array(optimal_weights)
        optimal_weights = optimal_weights / optimal_weights.sum()
        
        print(f"\n🏆 Optimal Ensemble Weights (F2 + Precision >= 60%):")
        for name, weight in zip(model_names, optimal_weights):
            print(f"   {name:12s}: {weight:.4f} ({weight*100:.2f}%)")
        print(f"\n   Best Score: {study.best_value:.6f}")
        
        # Store optimal weights
        self.optimal_weights = dict(zip(model_names, optimal_weights))
        
        # Validate on validation set with optimal weights
        ensemble_pred = np.zeros(len(self.y_val))
        for name, weight in self.optimal_weights.items():
            ensemble_pred += weight * base_predictions[name]
        
        # Find best threshold with optimal weights (optimizing for F2)
        best_f2 = 0
        best_thresh = 0.5
        for thresh in np.linspace(0.2, 0.7, 51):
            y_pred = (ensemble_pred > thresh).astype(int)
            f2 = fbeta_score(self.y_val, y_pred, beta=2)
            prec = precision_score(self.y_val, y_pred, zero_division=0)
            # Only consider thresholds with acceptable precision
            if prec >= PRECISION_FLOOR and f2 > best_f2:
                best_f2 = f2
                best_thresh = thresh
        
        y_pred_optimal = (ensemble_pred > best_thresh).astype(int)
        
        print(f"\n📊 Validation Set Performance with Optimized Weights:")
        print(f"   Threshold: {best_thresh:.2f}")
        print(f"   Accuracy:  {accuracy_score(self.y_val, y_pred_optimal)*100:.2f}%")
        print(f"   Precision: {precision_score(self.y_val, y_pred_optimal, zero_division=0)*100:.2f}%")
        print(f"   Recall:    {recall_score(self.y_val, y_pred_optimal)*100:.2f}%")
        print(f"   F1-Score:  {f1_score(self.y_val, y_pred_optimal)*100:.2f}%")
        print(f"   F2-Score:  {fbeta_score(self.y_val, y_pred_optimal, beta=2)*100:.2f}%")
        
        return self.optimal_weights
    
    def get_ensemble_predictions(self, X, mode='optimized'):
        """
        Get ensemble predictions with multiple modes
        Uses calibrated models when available.
        
        Modes:
        - 'bouncer': Rule Engine as early-exit filter (FASTEST, catches obvious fraud)
        - 'optimized': Use Bayesian-optimized weights (best)
        - 'enhanced': Use all framework components (Rule+Autoencoder+ML)
        - 'weighted': Use predefined weights based on model complexity
        - 'average': Simple average
        - 'voting': Hard voting (majority)
        - 'stacking': Meta-learning approach
        """
        
        # ========= BOUNCER MODE: Soft Cascade with Dynamic Boost =========
        if mode == 'bouncer':
            """
            SOFT CASCADE with DYNAMIC BOOST (ML Engineer's Formula):
            
            1. Rule Engine computes risk score (0-100) using UNSCALED data
            2. Score >= 85: HARD BLOCK (definite fraud)
            3. Score 60-84: HIGH RISK → 35% rule + 65% ML blend
            4. Score < 60: LOW RISK → 10% rule + 90% ML blend
            """
            final_predictions = np.zeros(len(X))
            HARD_BLOCK_THRESHOLD = 0.85  # Score >= 85 = definite fraud
            HIGH_RISK_THRESHOLD = 0.60   # Score >= 60 = high risk
            
            if hasattr(self, 'rule_engine') and hasattr(self, 'feature_names'):
                # CRITICAL: Rule Engine needs UNSCALED data (original feature values)
                # Determine which unscaled dataset matches the input shape
                X_unscaled = None
                if hasattr(self, 'X_val') and len(X) == len(self.X_val):
                    X_unscaled = self.X_val
                elif hasattr(self, 'X_test') and len(X) == len(self.X_test):
                    X_unscaled = self.X_test
                elif hasattr(self, 'X_train') and len(X) == len(self.X_train):
                    X_unscaled = self.X_train
                
                if X_unscaled is not None:
                    rule_scores = self.rule_engine.predict_batch(X_unscaled, self.feature_names)
                else:
                    # Fallback: try inverse transform if scaler exists
                    if hasattr(self, 'scaler') and self.scaler is not None:
                        try:
                            X_unscaled = self.scaler.inverse_transform(X)
                            rule_scores = self.rule_engine.predict_batch(X_unscaled, self.feature_names)
                        except:
                            # Last resort: use scaled data (rules may not fire correctly)
                            rule_scores = self.rule_engine.predict_batch(X, self.feature_names)
                    else:
                        rule_scores = self.rule_engine.predict_batch(X, self.feature_names)
                
                # ============= HARD BLOCK (Score >= 85) =============
                hard_block_mask = rule_scores >= HARD_BLOCK_THRESHOLD
                n_hard_blocked = hard_block_mask.sum()
                final_predictions[hard_block_mask] = 1.0
                
                # ============= SOFT CASCADE (Score < 98) =============
                soft_mask = ~hard_block_mask
                n_soft = soft_mask.sum()
                
                if n_soft > 0:
                    X_soft = X[soft_mask]
                    soft_rule_scores = rule_scores[soft_mask]
                    
                    # Get ML predictions for soft cascade transactions
                    models_to_use = getattr(self, 'calibrated_models', self.models)
                    
                    if hasattr(self, 'optimal_weights') and self.optimal_weights:
                        ml_preds = np.zeros(len(X_soft))
                        for name, model in models_to_use.items():
                            weight = self.optimal_weights.get(name, 0)
                            if weight > 0:
                                ml_preds += weight * model.predict_proba(X_soft)[:, 1]
                    else:
                        pred_list = [model.predict_proba(X_soft)[:, 1] for model in models_to_use.values()]
                        ml_preds = np.mean(pred_list, axis=0) if pred_list else np.zeros(len(X_soft))
                    
                    # Add autoencoder boost if available
                    if hasattr(self, 'autoencoder') and self.autoencoder.is_fitted:
                        ae_scores = self.autoencoder.predict(X_soft)
                        ml_preds = ml_preds * 0.85 + ae_scores * 0.15
                    
                    # ============= DYNAMIC BOOST FORMULA (BNN-Enhanced) =============
                    # Use BNN to identify high-risk transactions (replaces hardcoded threshold)
                    # High risk (BNN predicts fraud): 35% rule, 65% ML
                    # Low risk (BNN predicts legit): 10% rule, 90% ML
                    
                    if hasattr(self, 'high_risk_identifier') and self.high_risk_identifier.is_fitted:
                        # Use BNN for high-risk identification
                        X_soft_unscaled = X_unscaled[soft_mask] if X_unscaled is not None else None
                        
                        if X_soft_unscaled is not None:
                            bnn_result = self.high_risk_identifier.predict(
                                X_soft_unscaled, 
                                self.feature_names, 
                                self.rule_engine
                            )
                            high_risk_mask = bnn_result['is_high_risk']
                            high_risk_proba = bnn_result['probability']
                            high_risk_uncertainty = bnn_result['uncertainty']
                        else:
                            # Fallback to hardcoded threshold
                            high_risk_mask = soft_rule_scores >= HIGH_RISK_THRESHOLD
                    else:
                        # Fallback: use hardcoded threshold
                        high_risk_mask = soft_rule_scores >= HIGH_RISK_THRESHOLD
                    
                    blended = np.zeros(len(X_soft))
                    
                    # High risk transactions (BNN says fraud-like patterns)
                    if high_risk_mask.sum() > 0:
                        boost_weight = 0.35
                        blended[high_risk_mask] = (
                            ml_preds[high_risk_mask] * (1 - boost_weight) +
                            soft_rule_scores[high_risk_mask] * boost_weight
                        )
                    
                    # Low risk transactions
                    low_risk_mask = ~high_risk_mask
                    if low_risk_mask.sum() > 0:
                        boost_weight = 0.10
                        blended[low_risk_mask] = (
                            ml_preds[low_risk_mask] * (1 - boost_weight) +
                            soft_rule_scores[low_risk_mask] * boost_weight
                        )
                    
                    final_predictions[soft_mask] = np.clip(blended, 0, 1)
                
                # Store cascade statistics
                n_high_risk = high_risk_mask.sum() if 'high_risk_mask' in dir() else 0
                self._cascade_stats = {
                    'total': len(X),
                    'hard_blocked': int(n_hard_blocked),
                    'high_risk_soft': int(n_high_risk),
                    'low_risk': int(len(X) - n_hard_blocked - n_high_risk),
                    'hard_block_rate': n_hard_blocked / len(X) * 100
                }
                
                # Log stats
                if not getattr(self, '_bouncer_stats_logged', False):
                    print(f"   🚨 Hard Blocked (score>=85): {n_hard_blocked}/{len(X)} ({n_hard_blocked/len(X)*100:.1f}%)")
                    print(f"   ⚡ High Risk Blend (35% rule, BNN): {n_high_risk}")
                    print(f"   📊 Low Risk Blend (10% rule): {len(X) - n_hard_blocked - n_high_risk}")
                    self._bouncer_stats_logged = True
                
                return final_predictions
            else:
                print("   ⚠️ Rule engine not available, falling back to optimized mode")
                mode = 'optimized'
        
        # ========= Standard Modes =========
        predictions = {}
        
        # Use calibrated models if available, otherwise raw models
        models_to_use = getattr(self, 'calibrated_models', self.models)
        
        for name, model in models_to_use.items():
            pred_proba = model.predict_proba(X)[:, 1]
            predictions[name] = pred_proba
        
        # Always add Autoencoder predictions (for 'optimized' mode with 5-model weights)
        if hasattr(self, 'autoencoder') and self.autoencoder.is_fitted:
            predictions['autoencoder'] = self.autoencoder.predict(X)
        
        # Add framework component scores if mode is enhanced
        if mode == 'enhanced':
            # Get autoencoder anomaly scores
            if hasattr(self, 'autoencoder') and self.autoencoder.is_fitted:
                autoencoder_scores = self.autoencoder.predict(X)
                predictions['autoencoder'] = autoencoder_scores
            
            # Get rule engine scores
            if hasattr(self, 'rule_engine') and hasattr(self, 'feature_names'):
                rule_scores = self.rule_engine.predict_batch(X, self.feature_names)
                predictions['rule_engine'] = rule_scores
            
            # Weighted combination: ML models (80%) + Framework (20%)
            ml_weight = 0.80
            framework_weight = 0.20
            
            ml_models = ['xgboost', 'lightgbm', 'catboost', 'extratrees']
            framework_models = ['autoencoder', 'rule_engine']
            
            ml_preds = [predictions[m] for m in ml_models if m in predictions]
            fw_preds = [predictions[m] for m in framework_models if m in predictions]
            
            ml_avg = np.mean(ml_preds, axis=0) if ml_preds else np.zeros(len(X))
            fw_avg = np.mean(fw_preds, axis=0) if fw_preds else np.zeros(len(X))
            
            return ml_weight * ml_avg + framework_weight * fw_avg
        
        if mode == 'optimized' and hasattr(self, 'optimal_weights'):
            # Use Bayesian-optimized weights
            ensemble_pred = np.zeros(len(X))
            for name, weight in self.optimal_weights.items():
                if name in predictions:
                    ensemble_pred += weight * predictions[name]
            return ensemble_pred
        
        elif mode == 'weighted':
            # Predefined weights for 4-model ensemble
            weight_map = {
                'xgboost': 0.30,     # Strong baseline
                'lightgbm': 0.30,    # Fast and accurate  
                'catboost': 0.20,    # Good with categorical
                'extratrees': 0.20   # Diversity
            }
            ensemble_pred = np.zeros(len(X))
            total_weight = 0
            for name, pred in predictions.items():
                weight = weight_map.get(name, 1.0 / len(predictions))
                ensemble_pred += weight * pred
                total_weight += weight
            return ensemble_pred / total_weight
        
        elif mode == 'average':
            # Simple average
            return np.mean(list(predictions.values()), axis=0)
        
        elif mode == 'voting':
            # Hard voting (majority)
            votes = np.array([pred > 0.5 for pred in predictions.values()])
            return np.mean(votes, axis=0)
        
        elif mode == 'stacking':
            # Stacking: use predictions as features for meta-model
            if hasattr(self, 'meta_model'):
                stacked_features = np.column_stack(list(predictions.values()))
                return self.meta_model.predict_proba(stacked_features)[:, 1]
            else:
                # Fallback to weighted average
                return self.get_ensemble_predictions(X, mode='weighted')
        
        else:
            # Default to average
            return np.mean(list(predictions.values()), axis=0)
    
    def train_stacking_meta_model(self):
        """
        STACKING META-LEARNER
        Train a meta-model on base model predictions
        Uses Logistic Regression for interpretability
        """
        print("\n🎓 Training Stacking Meta-Model...")
        
        # Get base model predictions on validation set
        base_predictions_val = []
        for name, model in self.models.items():
            pred_proba = model.predict_proba(self.X_val_scaled)[:, 1]
            base_predictions_val.append(pred_proba)
        
        X_meta_val = np.column_stack(base_predictions_val)
        
        # Train meta-model with class weights
        fraud_ratio = np.sum(self.y_val == 0) / np.sum(self.y_val == 1)
        
        self.meta_model = LogisticRegression(
            class_weight={0: 1, 1: fraud_ratio},
            random_state=RANDOM_STATE,
            max_iter=1000
        )
        
        self.meta_model.fit(X_meta_val, self.y_val)
        
        # Evaluate meta-model
        y_meta_pred = self.meta_model.predict(X_meta_val)
        meta_f1 = f1_score(self.y_val, y_meta_pred) * 100
        
        print(f"   ✓ Meta-model trained")
        print(f"   Validation F1: {meta_f1:.2f}%")
        print(f"\n   Meta-model weights (how much each base model contributes):")
        for i, name in enumerate(self.models.keys()):
            coef = self.meta_model.coef_[0][i]
            print(f"      {name:12s}: {coef:+.4f}")
        
        return self.meta_model
    
    def evaluate_individual_models(self, X=None, y=None, dataset_name="Validation"):
        """
        INDIVIDUAL MODEL EVALUATION
        Print metrics for each model separately to understand their strengths
        """
        if X is None:
            X = self.X_val_scaled
            y = self.y_val
        
        print("\n" + "="*80)
        print(f"📊 INDIVIDUAL MODEL PERFORMANCE ({dataset_name} Set)")
        print("="*80)
        
        print(f"\n{'Model':<15} {'Acc%':>8} {'Prec%':>8} {'Rec%':>8} {'F1%':>8} {'F2%':>8} {'AUC%':>8}")
        print("-" * 80)
        
        model_results = {}
        
        for name, model in self.models.items():
            try:
                y_pred_proba = model.predict_proba(X)[:, 1]
                
                # Find optimal threshold for this model
                best_thresh = 0.5
                best_f1 = 0
                for thresh in np.linspace(0.3, 0.7, 41):
                    y_pred = (y_pred_proba > thresh).astype(int)
                    local_f1 = f1_score(y, y_pred)
                    if local_f1 > best_f1:
                        best_f1 = local_f1
                        best_thresh = thresh
                
                y_pred = (y_pred_proba > best_thresh).astype(int)
                
                acc = accuracy_score(y, y_pred) * 100
                prec = precision_score(y, y_pred, zero_division=0) * 100
                rec = recall_score(y, y_pred) * 100
                f1 = f1_score(y, y_pred) * 100
                f2 = fbeta_score(y, y_pred, beta=2) * 100
                auc = roc_auc_score(y, y_pred_proba) * 100
                
                model_results[name] = {
                    'accuracy': acc, 'precision': prec, 'recall': rec,
                    'f1': f1, 'f2': f2, 'auc': auc, 'threshold': best_thresh
                }
                
                print(f"{name:<15} {acc:>8.2f} {prec:>8.2f} {rec:>8.2f} {f1:>8.2f} {f2:>8.2f} {auc:>8.2f}")
                
            except Exception as e:
                print(f"{name:<15} - Failed: {str(e)}")
        
        # Add Autoencoder if available
        if hasattr(self, 'autoencoder') and self.autoencoder.is_fitted:
            try:
                ae_scores = self.autoencoder.predict(X)
                
                best_thresh = 0.5
                best_f1 = 0
                for thresh in np.linspace(0.3, 0.9, 31):
                    y_pred = (ae_scores > thresh).astype(int)
                    local_f1 = f1_score(y, y_pred)
                    if local_f1 > best_f1:
                        best_f1 = local_f1
                        best_thresh = thresh
                
                y_pred = (ae_scores > best_thresh).astype(int)
                
                acc = accuracy_score(y, y_pred) * 100
                prec = precision_score(y, y_pred, zero_division=0) * 100
                rec = recall_score(y, y_pred) * 100
                f1 = f1_score(y, y_pred) * 100
                f2 = fbeta_score(y, y_pred, beta=2) * 100
                auc = roc_auc_score(y, ae_scores) * 100
                
                model_results['autoencoder'] = {
                    'accuracy': acc, 'precision': prec, 'recall': rec,
                    'f1': f1, 'f2': f2, 'auc': auc, 'threshold': best_thresh
                }
                
                print(f"{'autoencoder':<15} {acc:>8.2f} {prec:>8.2f} {rec:>8.2f} {f1:>8.2f} {f2:>8.2f} {auc:>8.2f}")
                
            except Exception as e:
                print(f"{'autoencoder':<15} - Failed: {str(e)}")
        
        print("-" * 80)
        
        # Find best individual model
        best_model = max(model_results.items(), key=lambda x: x[1]['f1'])
        print(f"\n🏆 Best Individual Model: {best_model[0].upper()} (F1: {best_model[1]['f1']:.2f}%)")
        
        self.individual_model_results = model_results
        return model_results
    
    def _strategy_tiered_threshold(self, X, threshold=0.5, consensus_min=3, high_conf_thresh=0.9):
        """
        STRATEGY 1: Tiered Threshold Voting
        Layer 1: Rule Engine first
        Layer 2: Consensus (if ≥N models agree)
        Layer 3: High-confidence override (if ANY model is very sure)
        Layer 4: Weighted average fallback
        """
        predictions = {}
        
        # Get all model predictions
        for name, model in self.models.items():
            predictions[name] = model.predict_proba(X)[:, 1]
        
        if hasattr(self, 'autoencoder') and self.autoencoder.is_fitted:
            predictions['autoencoder'] = self.autoencoder.predict(X)
        
        n_samples = len(X)
        final_pred = np.zeros(n_samples)
        
        # Stack all predictions
        all_preds = np.array(list(predictions.values()))  # (n_models, n_samples)
        
        for i in range(n_samples):
            sample_preds = all_preds[:, i]
            
            # Layer 2: Consensus check
            votes_above_threshold = np.sum(sample_preds > threshold)
            
            if votes_above_threshold >= consensus_min:
                # Strong consensus -> high probability
                final_pred[i] = np.mean(sample_preds[sample_preds > threshold])
            
            # Layer 3: High-confidence override
            elif np.max(sample_preds) > high_conf_thresh:
                final_pred[i] = np.max(sample_preds)
            
            # Layer 4: Weighted average fallback
            else:
                if hasattr(self, 'optimal_weights'):
                    for name, weight in self.optimal_weights.items():
                        if name in predictions:
                            final_pred[i] += weight * predictions[name][i]
                else:
                    final_pred[i] = np.mean(sample_preds)
        
        return final_pred
    
    def _strategy_confidence_weighted(self, X):
        """
        STRATEGY 2: Confidence-Weighted Voting
        Models that are more "certain" (further from 0.5) get more weight
        """
        predictions = {}
        
        for name, model in self.models.items():
            predictions[name] = model.predict_proba(X)[:, 1]
        
        if hasattr(self, 'autoencoder') and self.autoencoder.is_fitted:
            predictions['autoencoder'] = self.autoencoder.predict(X)
        
        # Base weights from Bayesian optimization (or equal)
        base_weights = getattr(self, 'optimal_weights', {name: 1.0/len(predictions) for name in predictions})
        
        n_samples = len(X)
        final_pred = np.zeros(n_samples)
        
        for i in range(n_samples):
            weighted_sum = 0
            total_weight = 0
            
            for name, preds in predictions.items():
                p = preds[i]
                # Confidence = how far from 0.5 (uncertain)
                confidence = np.abs(p - 0.5) * 2  # 0 if p=0.5, 1 if p=0 or p=1
                
                base_w = base_weights.get(name, 1.0/len(predictions))
                effective_weight = base_w * (0.5 + confidence)  # At least 50% of base weight
                
                weighted_sum += effective_weight * p
                total_weight += effective_weight
            
            final_pred[i] = weighted_sum / (total_weight + 1e-9)
        
        return final_pred
    
    def _strategy_anomaly_boosted(self, X, alpha=0.3):
        """
        STRATEGY 3: Anomaly-Boosted Ensemble
        Use the Autoencoder as a MULTIPLIER, not a voter
        Increases scores for transactions the autoencoder finds "weird"
        """
        # Get ML model average
        ml_preds = []
        for name, model in self.models.items():
            ml_preds.append(model.predict_proba(X)[:, 1])
        
        ml_avg = np.mean(ml_preds, axis=0)
        
        # Get autoencoder boost
        if hasattr(self, 'autoencoder') and self.autoencoder.is_fitted:
            ae_scores = self.autoencoder.predict(X)
            ae_boost = 1 + alpha * ae_scores  # 1.0 to 1.3 multiplier
            
            final_pred = ml_avg * ae_boost
            # Clip to [0, 1]
            final_pred = np.clip(final_pred, 0, 1)
        else:
            final_pred = ml_avg
        
        return final_pred
    
    def _strategy_stacking_enhanced(self, X):
        """
        STRATEGY 4: Bayesian-Optimized Enhanced Stacking
        Uses base predictions (with optimized input weights) + summary statistics
        """
        if not hasattr(self, 'enhanced_meta_model'):
            # Train enhanced meta-model if not exists
            self._train_enhanced_meta_model()
        
        # Get base predictions
        base_preds = []
        for name, model in self.models.items():
            base_preds.append(model.predict_proba(X)[:, 1])
        
        if hasattr(self, 'autoencoder') and self.autoencoder.is_fitted:
            base_preds.append(self.autoencoder.predict(X))
        
        base_preds = np.array(base_preds).T  # (n_samples, n_models)
        n_base_models = base_preds.shape[1]
        
        # Apply Bayesian-optimized input weights
        if hasattr(self, 'meta_input_weights') and len(self.meta_input_weights) == n_base_models:
            for i in range(n_base_models):
                base_preds[:, i] *= self.meta_input_weights[i]
        
        # Add meta-features
        meta_features = np.column_stack([
            base_preds,
            np.max(base_preds, axis=1),      # Max across models
            np.min(base_preds, axis=1),      # Min across models
            np.std(base_preds, axis=1),      # Disagreement
            np.sum(base_preds > 0.5, axis=1) # Consensus count
        ])
        
        return self.enhanced_meta_model.predict_proba(meta_features)[:, 1]
    
    def _train_enhanced_meta_model(self):
        """
        DIRECT ENHANCED META-MODEL (NO BAYESIAN OPTIMIZATION)
        
        Uses optimal parameters from previous optimization Trial 296:
        - C: 0.08521596812289807
        - class_weight_ratio: 2.8291961956998812
        - input_weight_0 (xgboost): 0.35247629370537314
        - input_weight_1 (lightgbm): 0.5673794299635898
        - input_weight_2 (catboost): 1.894993092888986
        - threshold: 0.478701698309106
        
        Performance achieved: F2=72.32%
        """
        print("   🚀 Training enhanced stacking meta-model (Using Bayesian Optimization backed optimal weights)...")
        print("      Using optimal parameters from previous optimization")
        
        # Prepare base predictions
        base_preds_val = []
        model_names = []
        for name, model in self.models.items():
            base_preds_val.append(model.predict_proba(self.X_val_scaled)[:, 1])
            model_names.append(name)
        
        if hasattr(self, 'autoencoder') and self.autoencoder.is_fitted:
            base_preds_val.append(self.autoencoder.predict(self.X_val_scaled))
            model_names.append('autoencoder')
        
        base_preds_val = np.array(base_preds_val).T  # (n_samples, n_models)
        n_base_models = base_preds_val.shape[1]
        
        # Create meta-features
        meta_features = np.column_stack([
            base_preds_val,
            np.max(base_preds_val, axis=1),
            np.min(base_preds_val, axis=1),
            np.std(base_preds_val, axis=1),
            np.sum(base_preds_val > 0.5, axis=1)
        ])
        
        # Optimal parameters from Trial 296 (F2: 0.7232)
        best_params = {
            'C': 0.08521596812289807,
            'class_weight_ratio': 2.8291961956998812,
            'input_weight_0': 0.35247629370537314,   # xgboost
            'input_weight_1': 0.5673794299635898,    # lightgbm
            'input_weight_2': 1.894993092888986,     # catboost
            'threshold': 0.478701698309106
        }
        
        # Build input weights for current number of models
        input_weights = []
        for i in range(n_base_models):
            input_weights.append(best_params.get(f'input_weight_{i}', 1.0))
        
        # Apply input weights to base predictions
        meta_features_weighted = meta_features.copy()
        for i in range(n_base_models):
            meta_features_weighted[:, i] *= input_weights[i]
        
        # Train final meta-model with optimal parameters
        self.enhanced_meta_model = LogisticRegression(
            C=best_params['C'],
            class_weight={0: 1, 1: best_params['class_weight_ratio']},
            random_state=RANDOM_STATE,
            max_iter=1000,
            solver='lbfgs'
        )
        self.enhanced_meta_model.fit(meta_features_weighted, self.y_val)
        
        # Store optimized parameters for inference
        self.meta_input_weights = input_weights
        self.meta_threshold = best_params['threshold']
        
        print(f"      ✅ Meta-model configured with optimal parameters")
        print(f"      Meta-model input weights:")
        for i, name in enumerate(model_names):
            if i < len(input_weights):
                print(f"         {name:12s}: {input_weights[i]:.4f}")
        print(f"      Optimized threshold: {self.meta_threshold:.3f}")
        print("   ✓ Enhanced meta-model ready (Using Bayesian Optimization backed optimal weights)")
    
    
    def _strategy_delphi_consensus(self, X, n_rounds=3, convergence_threshold=0.01):
        """
        STRATEGY 5: Delphi Consensus Voting
        
        Inspired by the Delphi Method (expert panel technique):
        1. Each model predicts independently (Round 1)
        2. Calculate group consensus and disagreement
        3. Down-weight models that strongly disagree with majority
        4. Repeat until convergence or max rounds
        
        Key insight: When models disagree, trust the "wisdom of the crowd"
        and discount outlier predictions.
        """
        # Round 1: Get initial predictions from all models
        predictions = {}
        for name, model in self.models.items():
            predictions[name] = model.predict_proba(X)[:, 1]
        
        model_names = list(predictions.keys())
        n_models = len(model_names)
        n_samples = len(X)
        
        # Initialize weights equally
        weights = {name: 1.0 / n_models for name in model_names}
        
        prev_ensemble = None
        
        for round_num in range(n_rounds):
            # Calculate weighted ensemble prediction
            ensemble_pred = np.zeros(n_samples)
            for name in model_names:
                ensemble_pred += weights[name] * predictions[name]
            
            # Check convergence
            if prev_ensemble is not None:
                change = np.mean(np.abs(ensemble_pred - prev_ensemble))
                if change < convergence_threshold:
                    break
            
            prev_ensemble = ensemble_pred.copy()
            
            # Calculate per-sample disagreement for each model
            # Models that deviate from consensus get down-weighted
            new_weights = {}
            for name in model_names:
                # Disagreement = how far this model is from the ensemble
                disagreement = np.abs(predictions[name] - ensemble_pred)
                avg_disagreement = np.mean(disagreement)
                
                # More disagreement = lower weight (but never zero)
                # Agreement score: 1 = perfect agreement, 0 = max disagreement
                agreement_score = 1 - avg_disagreement
                
                # New weight: base weight adjusted by agreement
                # Models in agreement get boosted, outliers get penalized
                new_weights[name] = max(0.05, weights[name] * (0.5 + agreement_score))
            
            # Normalize weights to sum to 1
            total_weight = sum(new_weights.values())
            weights = {name: w / total_weight for name, w in new_weights.items()}
        
        # Final prediction with converged weights
        final_pred = np.zeros(n_samples)
        for name in model_names:
            final_pred += weights[name] * predictions[name]
        
        return final_pred
    
    def compare_all_strategies(self, X=None, y=None, dataset_name="Validation"):
        """
        COMPREHENSIVE STRATEGY COMPARISON
        Compare all voting strategies - selects based on BALANCED F1
        """
        if X is None:
            X = self.X_val_scaled
            y = self.y_val
        
        print("\n" + "="*80)
        print(f"🔬 COMPARING ALL VOTING STRATEGIES ({dataset_name} Set)")
        print("="*80)
        
        strategies = {
            'average': lambda x: self.get_ensemble_predictions(x, mode='average'),
            'weighted': lambda x: self.get_ensemble_predictions(x, mode='weighted'),
            'bayesian_opt': lambda x: self.get_ensemble_predictions(x, mode='optimized'),
            'tiered_vote': lambda x: self._strategy_tiered_threshold(x),
            'conf_weighted': lambda x: self._strategy_confidence_weighted(x),
            'delphi': lambda x: self._strategy_delphi_consensus(x),
            'stack_enhanced': lambda x: self._strategy_stacking_enhanced(x),
            'bouncer': lambda x: self.get_ensemble_predictions(x, mode='bouncer'),  # Rule Engine + ML Cascade
        }
        
        results = {}
        
        print(f"\n{'Strategy':<15} {'Acc%':>8} {'Prec%':>8} {'Rec%':>8} {'F1%':>8} {'F2%':>8} {'AUC%':>8}")
        print("-" * 80)
        
        best_strategy = None
        best_score = -float('inf')  # Score incorporating F2 + precision floor
        best_metrics = (0, 0, 0)    # (F2, AUC, Precision) for display
        
        PRECISION_FLOOR = 0.60
        
        for name, strategy_fn in strategies.items():
            try:
                y_pred_proba = strategy_fn(X)
                
                # Find optimal threshold based on F2 with precision constraint
                best_thresh = 0.5
                best_local_score = -float('inf')
                for thresh in np.linspace(0.3, 0.7, 41):
                    y_pred = (y_pred_proba > thresh).astype(int)
                    local_f2 = fbeta_score(y, y_pred, beta=2)
                    local_prec = precision_score(y, y_pred, zero_division=0)
                    
                    # Apply same F2 + precision floor scoring as Bayesian optimization
                    if local_prec < PRECISION_FLOOR:
                        penalty = (PRECISION_FLOOR - local_prec) * 2.0
                        local_score = local_f2 - penalty
                    else:
                        precision_bonus = min(0.05, (local_prec - PRECISION_FLOOR) * 0.1)
                        local_score = local_f2 + precision_bonus
                    
                    if local_score > best_local_score:
                        best_local_score = local_score
                        best_thresh = thresh
                
                y_pred = (y_pred_proba > best_thresh).astype(int)
                
                acc = accuracy_score(y, y_pred) * 100
                prec = precision_score(y, y_pred, zero_division=0) * 100
                rec = recall_score(y, y_pred) * 100
                f1 = f1_score(y, y_pred) * 100
                f2 = fbeta_score(y, y_pred, beta=2) * 100
                auc = roc_auc_score(y, y_pred_proba) * 100
                
                results[name] = {
                    'accuracy': acc, 'precision': prec, 'recall': rec,
                    'f1': f1, 'f2': f2, 'auc': auc, 'threshold': best_thresh,
                    'score': best_local_score
                }
                
                # Check if this strategy is better (using F2 + precision constraint score)
                is_better = best_local_score > best_score
                
                marker = "⭐" if is_better else "  "
                prec_marker = "✓" if prec >= 60 else "✗"
                print(f"{name:<15} {acc:>8.2f} {prec:>8.2f}{prec_marker} {rec:>8.2f} {f1:>8.2f} {f2:>8.2f} {auc:>8.2f} {marker}")
                
                if is_better:
                    best_score = best_local_score
                    best_metrics = (f2, auc, prec)
                    best_strategy = name
                    
            except Exception as e:
                print(f"{name:<15} - Failed: {str(e)}")
        
        print("-" * 80)
        print(f"\n🏆 BEST STRATEGY: {best_strategy.upper()} (F2 + Precision ≥ 60%)")
        print(f"   F2: {best_metrics[0]:.2f}%, AUC: {best_metrics[1]:.2f}%, Precision: {best_metrics[2]:.2f}%")
        print(f"   Optimal Threshold: {results[best_strategy]['threshold']:.3f}")
        print("="*80)
        
        self.best_voting_strategy = best_strategy
        self.strategy_comparison_results = results
        
        return best_strategy, results


    def find_optimal_threshold_multi_objective(self):
        """Multi-objective threshold optimization"""
        print("\n🎯 Multi-Objective Threshold Optimization...")
        
        # Use the best ensemble strategy found
        strategy = getattr(self, 'best_ensemble_strategy', 'weighted')
        y_pred_proba = self.get_ensemble_predictions(self.X_val_scaled, mode=strategy)
        
        thresholds = np.linspace(0.1, 0.9, 81)
        
        results = []
        print(f"\n{'Thresh':>8} {'Acc%':>8} {'Prec%':>8} {'Rec%':>8} {'F1%':>8} {'F2%':>8} {'MCC':>8}")
        print("-" * 70)
        
        best_balanced = None
        best_f2 = None
        
        for thresh in thresholds:
            y_pred = (y_pred_proba > thresh).astype(int)
            
            acc = accuracy_score(self.y_val, y_pred) * 100
            prec = precision_score(self.y_val, y_pred, zero_division=0) * 100
            rec = recall_score(self.y_val, y_pred) * 100
            f1 = f1_score(self.y_val, y_pred) * 100
            f2 = fbeta_score(self.y_val, y_pred, beta=2) * 100
            mcc = matthews_corrcoef(self.y_val, y_pred)
            
            balanced_score = (acc + prec + rec + f1) / 4
            
            res = {
                'threshold': float(thresh),
                'accuracy': float(acc),
                'precision': float(prec),
                'recall': float(rec),
                'f1': float(f1),
                'f2': float(f2),
                'mcc': float(mcc),
                'balanced_score': float(balanced_score)
            }
            results.append(res)
            
            # Track best candidates
            if prec >= 80.0 and rec >= 80.0:
                if best_balanced is None or balanced_score > best_balanced['balanced_score']:
                    best_balanced = res
            
            if f2 > 0:
                if best_f2 is None or f2 > best_f2['f2']:
                    best_f2 = res
            
            if thresh % 0.1 < 0.025:  # Print every 10th
                print(f"{thresh:>8.3f} {acc:>8.2f} {prec:>8.2f} {rec:>8.2f} {f1:>8.2f} {f2:>8.2f} {mcc:>8.4f}")
        
        # Selection logic
        if best_balanced:
            optimal = best_balanced
            print(f"\n🏆 SELECTED: Balanced Optimum (Prec≥80%, Rec≥80%)")
        elif best_f2:
            optimal = best_f2
            print(f"\n🥈 SELECTED: F2-Optimized (Recall Priority)")
        else:
            optimal = max(results, key=lambda x: x['balanced_score'])
            print(f"\n🥉 SELECTED: Max Balanced Score")
        
        print(f"\n✅ Optimal Threshold: {optimal['threshold']:.3f}")
        print(f"   Accuracy:  {optimal['accuracy']:.2f}%")
        print(f"   Precision: {optimal['precision']:.2f}%")
        print(f"   Recall:    {optimal['recall']:.2f}%")
        print(f"   F1-Score:  {optimal['f1']:.2f}%")
        print(f"   F2-Score:  {optimal['f2']:.2f}%")
        print(f"   MCC:       {optimal['mcc']:.4f}")
        
        self.optimal_threshold = optimal['threshold']
        return results, optimal
    
    def evaluate_final(self):
        """Comprehensive final evaluation using HYBRID: Bouncer + Best ML Strategy
        
        Uses same cascade logic as threshold analysis with best_voting_strategy
        """
        print("\n" + "="*80)
        print("📊 FINAL CASCADED SYSTEM EVALUATION ON TEST SET")
        print("="*80)
        
        # Use best_voting_strategy for consistency with threshold analysis
        ml_strategy = getattr(self, 'best_voting_strategy', 'average')
        if ml_strategy == 'bouncer':
            ml_strategy = 'average'  # Fallback if bouncer was selected
        
        print(f"   🚨 HYBRID MODE: Rule Engine (Bouncer) + {ml_strategy.upper()}")
        
        # ============= STEP 1: Rule Engine Evaluation (Hard Block) =============
        final_predictions = np.zeros(len(self.X_test_scaled))
        HARD_BLOCK_THRESHOLD = 0.85  # Scores >= 85 = definite fraud
        
        if hasattr(self, 'rule_engine') and hasattr(self, 'feature_names') and hasattr(self, 'X_test'):
            # CRITICAL: Use UNSCALED X_test for Rule Engine (thresholds need original values)
            rule_scores = self.rule_engine.predict_batch(self.X_test, self.feature_names)
            
            # Hard block obvious frauds
            hard_block_mask = rule_scores >= HARD_BLOCK_THRESHOLD
            n_hard_blocked = hard_block_mask.sum()
            final_predictions[hard_block_mask] = 1.0  # Flag as fraud
            
            print(f"   🚨 Rule Engine Hard Blocked: {n_hard_blocked}/{len(self.X_test_scaled)} ({n_hard_blocked/len(self.X_test_scaled)*100:.2f}%)")
            
            # ============= STEP 2: ML Strategy for Non-Blocked Transactions =============
            soft_mask = ~hard_block_mask
            n_soft = soft_mask.sum()
            
            if n_soft > 0:
                X_soft = self.X_test_scaled[soft_mask]
                
                # Get ML predictions using best strategy
                if ml_strategy == 'stack_enhanced':
                    ml_preds = self._strategy_stacking_enhanced(X_soft)
                elif ml_strategy == 'tiered_vote':
                    ml_preds = self._strategy_tiered_threshold(X_soft)
                elif ml_strategy == 'conf_weighted':
                    ml_preds = self._strategy_confidence_weighted(X_soft)
                elif ml_strategy == 'delphi':
                    ml_preds = self._strategy_delphi_consensus(X_soft)
                else:
                    mode_map = {'average': 'average', 'weighted': 'weighted', 'bayesian_opt': 'optimized'}
                    ml_preds = self.get_ensemble_predictions(X_soft, mode=mode_map.get(ml_strategy, 'average'))
                
                # Apply soft cascade: blend rule scores with ML predictions (BNN-Enhanced)
                soft_rule_scores = rule_scores[soft_mask]
                X_soft_unscaled = self.X_test[soft_mask] if hasattr(self, 'X_test') else None
                
                # Use BNN to identify high-risk transactions
                if hasattr(self, 'high_risk_identifier') and self.high_risk_identifier.is_fitted and X_soft_unscaled is not None:
                    bnn_result = self.high_risk_identifier.predict(
                        X_soft_unscaled,
                        self.feature_names,
                        self.rule_engine
                    )
                    high_risk_mask = bnn_result['is_high_risk']
                else:
                    # Fallback to hardcoded threshold
                    HIGH_RISK_THRESHOLD = 0.60
                    high_risk_mask = soft_rule_scores >= HIGH_RISK_THRESHOLD
                
                # FIX: Also use HIGH RISK blend when rule score >= 60%
                # This ensures strong rule signals (like velocity attacks) are not diluted by BNN
                strong_rule_mask = soft_rule_scores >= 0.60
                use_high_risk_blend = high_risk_mask | strong_rule_mask
                
                low_risk_mask = ~use_high_risk_blend
                blended = np.zeros(n_soft)
                
                # High risk (BNN OR strong rules): 35% rule + 65% ML
                if use_high_risk_blend.sum() > 0:
                    blended[use_high_risk_blend] = (
                        ml_preds[use_high_risk_blend] * 0.65 +
                        soft_rule_scores[use_high_risk_blend] * 0.35
                    )
                
                # Low risk (BNN says legit AND rules < 60%): 10% rule + 90% ML
                if low_risk_mask.sum() > 0:
                    blended[low_risk_mask] = (
                        ml_preds[low_risk_mask] * 0.90 +
                        soft_rule_scores[low_risk_mask] * 0.10
                    )
                
                final_predictions[soft_mask] = np.clip(blended, 0, 1)
                
                n_high_risk = use_high_risk_blend.sum()
                n_strong_rules_only = (strong_rule_mask & ~high_risk_mask).sum()
                bnn_status = "BNN" if (hasattr(self, 'high_risk_identifier') and self.high_risk_identifier.is_fitted) else "threshold"
                print(f"   ⚡ High Risk Blend (35% rule, {bnn_status}+Rules): {n_high_risk} ({n_strong_rules_only} by rules only)")
                print(f"   📊 Low Risk Blend (10% rule): {low_risk_mask.sum()}")
        else:
            # Fallback: No rule engine, use pure ML strategy
            print(f"   ⚠️ Rule Engine not available, using pure {ml_strategy.upper()}")
            if ml_strategy == 'stack_enhanced':
                final_predictions = self._strategy_stacking_enhanced(self.X_test_scaled)
            elif ml_strategy == 'tiered_vote':
                final_predictions = self._strategy_tiered_threshold(self.X_test_scaled)
            elif ml_strategy == 'conf_weighted':
                final_predictions = self._strategy_confidence_weighted(self.X_test_scaled)
            elif ml_strategy == 'delphi':
                final_predictions = self._strategy_delphi_consensus(self.X_test_scaled)
            else:
                mode_map = {'average': 'average', 'weighted': 'weighted', 'bayesian_opt': 'optimized'}
                final_predictions = self.get_ensemble_predictions(self.X_test_scaled, mode=mode_map.get(ml_strategy, 'average'))
        
        y_pred_proba = final_predictions
        y_pred = (y_pred_proba > self.optimal_threshold).astype(int)
        
        # Calculate all metrics
        acc = accuracy_score(self.y_test, y_pred) * 100
        prec = precision_score(self.y_test, y_pred, zero_division=0) * 100
        rec = recall_score(self.y_test, y_pred) * 100
        f1 = f1_score(self.y_test, y_pred) * 100
        f2 = fbeta_score(self.y_test, y_pred, beta=2) * 100
        roc_auc = roc_auc_score(self.y_test, y_pred_proba) * 100
        pr_auc = average_precision_score(self.y_test, y_pred_proba) * 100
        mcc = matthews_corrcoef(self.y_test, y_pred)
        kappa = cohen_kappa_score(self.y_test, y_pred)
        balanced_acc = balanced_accuracy_score(self.y_test, y_pred) * 100
        
        print(f"\n📈 METRICS (Threshold={self.optimal_threshold:.3f}):")
        print(f"   Accuracy:            {acc:.2f}% {'✅' if acc >= 80 else '⚠️'}")
        print(f"   Balanced Accuracy:   {balanced_acc:.2f}%")
        print(f"   Precision:           {prec:.2f}% {'✅' if prec >= 80 else '⚠️'}")
        print(f"   Recall:              {rec:.2f}% {'✅' if rec >= 80 else '⚠️'}")
        print(f"   F1-Score:            {f1:.2f}% {'✅' if f1 >= 80 else '⚠️'}")
        print(f"   F2-Score:            {f2:.2f}%")
        print(f"   ROC-AUC:             {roc_auc:.2f}%")
        print(f"   PR-AUC:              {pr_auc:.2f}%")
        print(f"   MCC:                 {mcc:.4f}")
        print(f"   Cohen's Kappa:       {kappa:.4f}")
        
        cm = confusion_matrix(self.y_test, y_pred)
        print(f"\n🔢 Confusion Matrix:")
        print(f"   TN: {cm[0,0]:6d} | FP: {cm[0,1]:6d}")
        print(f"   FN: {cm[1,0]:6d} | TP: {cm[1,1]:6d}")
        
        # Per-model performance
        print(f"\n📊 Individual Model Performance:")
        for name, model in self.models.items():
            model_proba = model.predict_proba(self.X_test_scaled)[:, 1]
            model_pred = (model_proba > self.optimal_threshold).astype(int)
            model_f1 = f1_score(self.y_test, model_pred) * 100
            model_auc = roc_auc_score(self.y_test, model_proba) * 100
            print(f"   {name:12s}: F1={model_f1:.2f}%, AUC={model_auc:.2f}%")
        
        # Compare ALL ensemble strategies on test set (matching compare_all_strategies)
        print(f"\n📊 Ensemble Strategy Comparison (Test Set):")
        strategies_to_compare = [
            ('average', 'average'),
            ('weighted', 'weighted'),
            ('bayesian_opt', 'optimized'),
            ('conf_weighted', None),  # Custom strategy
            ('stack_enhanced', None),  # Custom strategy (Bayesian stacking)
        ]
        
        for strategy_name, mode in strategies_to_compare:
            try:
                if strategy_name == 'stack_enhanced':
                    strat_proba = self._strategy_stacking_enhanced(self.X_test_scaled)
                elif strategy_name == 'conf_weighted':
                    strat_proba = self._strategy_confidence_weighted(self.X_test_scaled)
                else:
                    strat_proba = self.get_ensemble_predictions(self.X_test_scaled, mode=mode)
                
                strat_pred = (strat_proba > self.optimal_threshold).astype(int)
                strat_f1 = f1_score(self.y_test, strat_pred) * 100
                strat_f2 = fbeta_score(self.y_test, strat_pred, beta=2) * 100
                strat_auc = roc_auc_score(self.y_test, strat_proba) * 100
                strat_rec = recall_score(self.y_test, strat_pred) * 100
                
                # Show which strategy is being used
                marker = '🏆' if strategy_name == ml_strategy else '  '
                print(f"   {marker} {strategy_name:15s}: F2={strat_f2:.2f}%, F1={strat_f1:.2f}%, Rec={strat_rec:.2f}%, AUC={strat_auc:.2f}%")
            except Exception as e:
                print(f"   ⚠️ {strategy_name:15s}: Failed - {str(e)[:30]}")
        
        # Show optimized weights if available
        if hasattr(self, 'optimal_weights'):
            print(f"\n🎯 Optimized Ensemble Weights:")
            for name, weight in self.optimal_weights.items():
                print(f"   {name:12s}: {weight:.4f} ({weight*100:.2f}%)")
        
        return {
            'accuracy': acc, 'balanced_accuracy': balanced_acc,
            'precision': prec, 'recall': rec, 
            'f1': f1, 'f2': f2,
            'roc_auc': roc_auc, 'pr_auc': pr_auc, 
            'mcc': mcc, 'kappa': kappa,
            'ensemble_mode': ml_strategy
        }
    
    def evaluate_cascaded_pipeline(self, X_test=None, y_test=None):
        """
        END-TO-END EVALUATION OF CASCADED FRAUD DETECTION PIPELINE
        
        Matches inference_demo.py cascade logic:
        1. Rule Engine (≥85% → HARD BLOCK, skip ML)
        2. BNN High-Risk Identification
        3. ML Ensemble with weighted predictions
        4. Cascaded Blending:
           - HIGH RISK (BNN=True OR Rules≥60%): 65%ML + 35%Rule
           - LOW RISK: 90%ML + 10%Rule
        5. Final threshold comparison
        """
        print("\n" + "="*80)
        print("🔄 CASCADED PIPELINE EVALUATION ON TEST SET")
        print("   (Matches inference_demo.py pipeline)")
        print("="*80)
        
        # Use stored test data if not provided
        if X_test is None:
            X_test_scaled = self.X_test_scaled
            X_test_unscaled = self.X_test if hasattr(self, 'X_test') else None
        else:
            X_test_scaled = X_test
            X_test_unscaled = None
        if y_test is None:
            y_test = self.y_test
        
        n_total = len(y_test)
        n_actual_fraud = int(y_test.sum())
        n_actual_legit = n_total - n_actual_fraud
        
        print(f"\n📊 Test Set: {n_total} transactions ({n_actual_fraud} fraud, {n_actual_legit} legit)")
        
        # Thresholds matching inference_demo.py
        HARD_BLOCK_THRESHOLD = 0.85  # Rule score ≥85% = definite fraud
        HIGH_RISK_THRESHOLD = 0.60   # For blending decision
        
        # =========================
        # STAGE 1: RULE-BASED MODEL (HARD BLOCK)
        # =========================
        print("\n" + "-"*60)
        print("STAGE 1: RULE ENGINE (Hard Block ≥85%)")
        print("-"*60)
        
        final_predictions = np.zeros(n_total)
        rule_scores_normalized = np.zeros(n_total)
        
        if hasattr(self, 'rule_engine') and hasattr(self, 'feature_names') and X_test_unscaled is not None:
            # Use UNSCALED data for rule engine
            rule_scores = self.rule_engine.predict_batch(X_test_unscaled, self.feature_names)
            rule_scores_normalized = rule_scores / 100.0  # Normalize to 0-1
            
            # Hard block obvious frauds
            hard_block_mask = rule_scores_normalized >= HARD_BLOCK_THRESHOLD
            n_hard_blocked = hard_block_mask.sum()
            final_predictions[hard_block_mask] = 1.0
            
            # Metrics for hard-blocked
            rule_tp = ((hard_block_mask) & (y_test == 1)).sum()
            rule_fp = ((hard_block_mask) & (y_test == 0)).sum()
            
            print(f"   Hard Block Threshold: ≥{int(HARD_BLOCK_THRESHOLD*100)}%")
            print(f"   Transactions blocked: {n_hard_blocked}/{n_total} ({n_hard_blocked/n_total*100:.1f}%)")
            print(f"   ✓ True Positives (fraud caught): {rule_tp}")
            print(f"   ✗ False Positives (legit blocked): {rule_fp}")
            if n_actual_fraud > 0:
                print(f"   Stage 1 Recall: {rule_tp/n_actual_fraud*100:.2f}%")
        else:
            print("   ⚠️ Rule engine not available or X_test unscaled not found")
            hard_block_mask = np.zeros(n_total, dtype=bool)
            rule_tp = 0
            rule_fp = 0
            n_hard_blocked = 0
        
        # =========================
        # STAGE 2: BNN + ML ENSEMBLE (Non-blocked transactions)
        # =========================
        print("\n" + "-"*60)
        print("STAGE 2: BNN + ML ENSEMBLE (with Cascaded Blending)")
        print("-"*60)
        
        soft_mask = ~hard_block_mask
        n_to_ml = soft_mask.sum()
        
        ml_tp = 0
        ml_fp = 0
        n_high_risk_blend = 0
        n_low_risk_blend = 0
        
        if n_to_ml > 0:
            X_soft = X_test_scaled[soft_mask]
            y_soft = y_test[soft_mask] if isinstance(y_test, np.ndarray) else y_test.values[soft_mask]
            soft_rule_scores = rule_scores_normalized[soft_mask]
            
            # Get ML predictions using optimal weights
            models_to_use = getattr(self, 'calibrated_models', self.models)
            ml_probs = np.zeros(len(X_soft))
            
            if hasattr(self, 'optimal_weights') and self.optimal_weights:
                for name, model in models_to_use.items():
                    weight = self.optimal_weights.get(name, 0)
                    if weight > 0:
                        ml_probs += weight * model.predict_proba(X_soft)[:, 1]
            else:
                ml_preds_list = [model.predict_proba(X_soft)[:, 1] for model in models_to_use.values()]
                ml_probs = np.mean(ml_preds_list, axis=0)
            
            # BNN High-Risk Detection
            if hasattr(self, 'high_risk_identifier') and self.high_risk_identifier.is_fitted and X_test_unscaled is not None:
                X_soft_unscaled = X_test_unscaled[soft_mask]
                bnn_result = self.high_risk_identifier.predict(
                    X_soft_unscaled,
                    self.feature_names,
                    self.rule_engine
                )
                bnn_high_risk_mask = bnn_result['is_high_risk']
                bnn_status = "BNN"
            else:
                # Fallback: use rule threshold for high-risk classification
                bnn_high_risk_mask = np.zeros(len(X_soft), dtype=bool)
                bnn_status = "threshold-only"
            
            # CASCADED BLENDING (matching inference_demo.py)
            # High Risk = BNN says high risk OR strong rule signals (≥60%)
            strong_rule_mask = soft_rule_scores >= HIGH_RISK_THRESHOLD
            use_high_risk_blend = bnn_high_risk_mask | strong_rule_mask
            low_risk_mask = ~use_high_risk_blend
            
            blended_scores = np.zeros(len(X_soft))
            
            # High Risk Blend: 65% ML + 35% Rule
            if use_high_risk_blend.sum() > 0:
                blended_scores[use_high_risk_blend] = (
                    ml_probs[use_high_risk_blend] * 0.65 +
                    soft_rule_scores[use_high_risk_blend] * 0.35
                )
            
            # Low Risk Blend: 90% ML + 10% Rule
            if low_risk_mask.sum() > 0:
                blended_scores[low_risk_mask] = (
                    ml_probs[low_risk_mask] * 0.90 +
                    soft_rule_scores[low_risk_mask] * 0.10
                )
            
            blended_scores = np.clip(blended_scores, 0, 1)
            
            # Apply threshold
            ml_flags = (blended_scores >= self.optimal_threshold).astype(int)
            
            # Assign to final predictions
            final_predictions[soft_mask] = blended_scores
            
            # Metrics
            ml_tp = ((ml_flags == 1) & (y_soft == 1)).sum()
            ml_fp = ((ml_flags == 1) & (y_soft == 0)).sum()
            n_fraud_in_soft = (y_soft == 1).sum()
            n_high_risk_blend = use_high_risk_blend.sum()
            n_low_risk_blend = low_risk_mask.sum()
            
            print(f"   Transactions received: {n_to_ml}/{n_total} ({n_to_ml/n_total*100:.1f}%)")
            print(f"   High-Risk Classification: {bnn_status}")
            print(f"   ⚡ High Risk Blend (65%ML+35%Rule): {n_high_risk_blend} txns")
            print(f"   📊 Low Risk Blend (90%ML+10%Rule): {n_low_risk_blend} txns")
            print(f"   Fraud in soft zone: {n_fraud_in_soft}")
            print(f"   ✓ Additional fraud caught (ML): {ml_tp}")
            print(f"   ✗ False positives (ML): {ml_fp}")
            if n_fraud_in_soft > 0:
                print(f"   Stage 2 Recall (on soft zone): {ml_tp/n_fraud_in_soft*100:.2f}%")
        
        # =========================
        # FINAL COMBINED RESULTS
        # =========================
        print("\n" + "-"*60)
        print("FINAL CASCADED RESULTS")
        print("-"*60)
        
        # Convert probabilities to binary predictions
        y_pred = (final_predictions >= self.optimal_threshold).astype(int)
        
        # Calculate final metrics
        final_tp = ((y_pred == 1) & (y_test == 1)).sum()
        final_fp = ((y_pred == 1) & (y_test == 0)).sum()
        final_tn = ((y_pred == 0) & (y_test == 0)).sum()
        final_fn = ((y_pred == 0) & (y_test == 1)).sum()
        
        accuracy = (final_tp + final_tn) / n_total
        precision = final_tp / (final_tp + final_fp) if (final_tp + final_fp) > 0 else 0
        recall = final_tp / (final_tp + final_fn) if (final_tp + final_fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        f2 = 5 * precision * recall / (4 * precision + recall) if (4 * precision + recall) > 0 else 0
        
        print(f"\n   📈 FINAL CASCADED METRICS (threshold={self.optimal_threshold:.3f}):")
        print(f"   ┌{'─'*50}┐")
        print(f"   │ Accuracy:   {accuracy*100:6.2f}% {'✅' if accuracy >= 0.85 else '📊':<20}│")
        print(f"   │ Precision:  {precision*100:6.2f}% {'✅' if precision >= 0.50 else '📊':<20}│")
        print(f"   │ Recall:     {recall*100:6.2f}% {'✅' if recall >= 0.80 else '📊':<20}│")
        print(f"   │ F1-Score:   {f1*100:6.2f}% {'✅' if f1 >= 0.65 else '📊':<20}│")
        print(f"   │ F2-Score:   {f2*100:6.2f}% (recall-weighted)      │")
        print(f"   └{'─'*50}┘")
        
        print(f"\n   🔢 Confusion Matrix:")
        print(f"   ┌────────┬────────┬────────┐")
        print(f"   │        │ Pred 0 │ Pred 1 │")
        print(f"   ├────────┼────────┼────────┤")
        print(f"   │ Act 0  │ {final_tn:6d} │ {final_fp:6d} │")
        print(f"   │ Act 1  │ {final_fn:6d} │ {final_tp:6d} │")
        print(f"   └────────┴────────┴────────┘")
        
        print(f"\n   📊 FRAUD DETECTION BREAKDOWN:")
        print(f"   • Total fraud cases: {n_actual_fraud}")
        print(f"   • Caught by Rules (Stage 1, Hard Block): {rule_tp} ({rule_tp/n_actual_fraud*100:.1f}%)")
        print(f"   • Caught by ML (Stage 2, Blended): {ml_tp} ({ml_tp/n_actual_fraud*100:.1f}%)")
        print(f"   • Total caught: {final_tp} ({recall*100:.1f}%)")
        print(f"   • Missed: {final_fn} ({final_fn/n_actual_fraud*100:.1f}%)")
        
        print(f"\n   🔧 PIPELINE CONFIGURATION:")
        print(f"   • Hard Block Threshold: ≥{int(HARD_BLOCK_THRESHOLD*100)}%")
        print(f"   • High-Risk Blend Threshold: ≥{int(HIGH_RISK_THRESHOLD*100)}%")
        print(f"   • ML Decision Threshold: {self.optimal_threshold:.3f}")
        print(f"   • Blending: HIGH=65%ML+35%Rule, LOW=90%ML+10%Rule")
        
        return {
            'accuracy': accuracy * 100,
            'precision': precision * 100,
            'recall': recall * 100,
            'f1': f1 * 100,
            'f2': f2 * 100,
            'rule_catches': int(rule_tp),
            'ml_catches': int(ml_tp),
            'total_catches': int(final_tp),
            'missed': int(final_fn),
            'high_risk_blend_count': int(n_high_risk_blend),
            'low_risk_blend_count': int(n_low_risk_blend),
            'confusion_matrix': {
                'tp': int(final_tp), 'fp': int(final_fp),
                'tn': int(final_tn), 'fn': int(final_fn)
            }
        }
    
    def plot_comprehensive_analysis(self):
        """Generate comprehensive visualizations"""
        print("\n📊 Generating comprehensive analysis plots...")
        
        ensemble_mode = getattr(self, 'best_ensemble_strategy', 'optimized')
        
        fig = plt.figure(figsize=(20, 12))
        
        # 1. Feature Importance
        ax1 = plt.subplot(2, 3, 1)
        if self.feature_importance is not None:
            top_features = self.feature_importance.head(20)
            sns.barplot(data=top_features, x='importance', y='feature', palette='viridis', ax=ax1)
            ax1.set_title('Top 20 Feature Importance (XGBoost)', fontsize=14, fontweight='bold')
            ax1.set_xlabel('Importance')
        
        # 2. ROC Curve (Multiple Strategies)
        ax2 = plt.subplot(2, 3, 2)
        strategies = ['average', 'weighted', 'optimized', 'stacking']
        colors = ['blue', 'orange', 'green', 'red']
        
        for strategy, color in zip(strategies, colors):
            try:
                y_pred_proba = self.get_ensemble_predictions(self.X_test_scaled, mode=strategy)
                fpr, tpr, _ = roc_curve(self.y_test, y_pred_proba)
                roc_auc = roc_auc_score(self.y_test, y_pred_proba)
                label = f'{strategy.upper()}: AUC={roc_auc:.3f}'
                if strategy == ensemble_mode:
                    ax2.plot(fpr, tpr, color=color, linewidth=3, label=label + ' ★')
                else:
                    ax2.plot(fpr, tpr, color=color, linewidth=2, alpha=0.6, label=label)
            except:
                pass
        
        ax2.plot([0, 1], [0, 1], 'navy', linestyle='--', alpha=0.5)
        ax2.set_title('ROC Curves (Ensemble Strategies)', fontsize=14, fontweight='bold')
        ax2.set_xlabel('False Positive Rate')
        ax2.set_ylabel('True Positive Rate')
        ax2.legend(loc='lower right', fontsize=9)
        ax2.grid(True, alpha=0.3)
        
        # 3. Precision-Recall Curve
        ax3 = plt.subplot(2, 3, 3)
        y_pred_proba = self.get_ensemble_predictions(self.X_test_scaled, mode=ensemble_mode)
        precision, recall, _ = precision_recall_curve(self.y_test, y_pred_proba)
        pr_auc = average_precision_score(self.y_test, y_pred_proba)
        ax3.plot(recall, precision, 'green', linewidth=2, label=f'AP={pr_auc:.3f}')
        ax3.set_title(f'Precision-Recall ({ensemble_mode.upper()})', fontsize=14, fontweight='bold')
        ax3.set_xlabel('Recall')
        ax3.set_ylabel('Precision')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
        
        # 4. Confusion Matrix
        ax4 = plt.subplot(2, 3, 4)
        y_pred = (y_pred_proba > self.optimal_threshold).astype(int)
        cm = confusion_matrix(self.y_test, y_pred)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax4, 
                   xticklabels=['Legitimate', 'Fraud'],
                   yticklabels=['Legitimate', 'Fraud'])
        ax4.set_title('Confusion Matrix', fontsize=14, fontweight='bold')
        ax4.set_ylabel('True Label')
        ax4.set_xlabel('Predicted Label')
        
        # 5. Ensemble Weights Visualization
        ax5 = plt.subplot(2, 3, 5)
        if hasattr(self, 'optimal_weights'):
            model_names = list(self.optimal_weights.keys())
            weights = list(self.optimal_weights.values())
            colors_weights = ['#1f77b4', '#ff7f0e', '#2ca02c'][:len(weights)]
            
            bars = ax5.barh(model_names, weights, color=colors_weights, alpha=0.8)
            ax5.set_title('Bayesian-Optimized Ensemble Weights', fontsize=14, fontweight='bold')
            ax5.set_xlabel('Weight')
            ax5.set_xlim([0, 1])
            
            # Add percentage labels
            for bar, weight in zip(bars, weights):
                width = bar.get_width()
                ax5.text(width + 0.02, bar.get_y() + bar.get_height()/2.,
                        f'{weight*100:.1f}%', ha='left', va='center', fontweight='bold')
        else:
            ax5.text(0.5, 0.5, 'Weights not optimized', ha='center', va='center', 
                    transform=ax5.transAxes, fontsize=12)
            ax5.set_title('Ensemble Weights', fontsize=14, fontweight='bold')
        
        # 6. Model Comparison with Strategy Comparison
        ax6 = plt.subplot(2, 3, 6)
        model_scores = []
        model_labels = []
        
        # Individual models
        for name, model in self.models.items():
            model_proba = model.predict_proba(self.X_test_scaled)[:, 1]
            model_pred = (model_proba > self.optimal_threshold).astype(int)
            model_scores.append(f1_score(self.y_test, model_pred) * 100)
            model_labels.append(name.upper())
        
        # Ensemble strategies
        for strategy in ['average', 'weighted', 'optimized', 'stacking']:
            try:
                strat_proba = self.get_ensemble_predictions(self.X_test_scaled, mode=strategy)
                strat_pred = (strat_proba > self.optimal_threshold).astype(int)
                strat_f1 = f1_score(self.y_test, strat_pred) * 100
                model_scores.append(strat_f1)
                label = f'ENS-{strategy[:3].upper()}'
                if strategy == ensemble_mode:
                    label += ' ★'
                model_labels.append(label)
            except:
                pass
        
        colors_bars = ['#1f77b4', '#ff7f0e', '#2ca02c'] + ['#d62728', '#9467bd', '#8c564b', '#e377c2']
        colors_bars = colors_bars[:len(model_scores)]
        
        bars = ax6.bar(range(len(model_labels)), model_scores, color=colors_bars, alpha=0.8)
        ax6.set_xticks(range(len(model_labels)))
        ax6.set_xticklabels(model_labels, rotation=45, ha='right')
        ax6.set_title('F1-Score Comparison (Models + Ensembles)', fontsize=14, fontweight='bold')
        ax6.set_ylabel('F1-Score (%)')
        ax6.set_ylim([0, 100])
        ax6.axhline(y=80, color='red', linestyle='--', alpha=0.5, label='Target (80%)')
        ax6.legend()
        
        # Add value labels on bars
        for bar in bars:
            height = bar.get_height()
            ax6.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.1f}%', ha='center', va='bottom', fontsize=8, fontweight='bold')
        
        plt.tight_layout()
        plt.savefig('ultimate_fraud_analysis.png', dpi=300, bbox_inches='tight')
        print("   ✓ Saved: ultimate_fraud_analysis.png")
        plt.close()
    
    def save_complete_system(self, prefix='fraud_system'):
        """
        Save complete cascaded fraud detection system as 2 separate files:
        1. Stage 1 (Joblib) - Rule Engine + BNN High-Risk Identifier
        2. Stage 2 (Joblib) - ML Ensemble Models
        
        This matches the cascaded architecture:
        Transaction → [Stage 1: Rules + BNN] → [Stage 2: ML Ensemble]
        
        THRESHOLD SELECTION:
        - Rule Engine: ALGORITHM-CHOSEN (score >= 85 for hard block)
        - Ensemble ML: MANUALLY CHOSEN (0.390 for real-world compliance)
        """
        print(f"\n💾 Saving cascaded fraud detection system...")
        print("-" * 60)
        
        # =====================================================
        # FILE 1: STAGE 1 - BNN MODEL COMPONENTS (JOBLIB)
        # Saves BNN's sklearn model + threshold separately (not the class)
        # This allows inference to load without needing the class definition
        # =====================================================
        
        # Extract BNN components (model + threshold) if available
        bnn = getattr(self, 'high_risk_identifier', None)
        if bnn is not None and bnn.is_fitted:
            bnn_model = bnn.model  # sklearn MLPClassifier (can be pickled)
            bnn_threshold = bnn.threshold
            bnn_rule_names = bnn.rule_names
        else:
            bnn_model = None
            bnn_threshold = 0.5
            bnn_rule_names = []
        
        stage1_bundle = {
            # Rule Engine Configuration (for reference - engine is recreated at inference)
            'rule_weights': RULE_WEIGHTS,
            'hard_block_threshold': 85,  # Score >= 85 = definite fraud
            'high_risk_threshold': 60,   # Score >= 60 = high risk blend
            # Note: rule_engine NOT included (lambdas can't be pickled)
            # Recreate at inference: rule_engine = RuleBasedEngine()
            
            # BNN Components (NOT the class, just the model + config)
            'bnn_model': bnn_model,              # sklearn MLPClassifier
            'bnn_threshold': bnn_threshold,       # Decision threshold
            'bnn_rule_names': bnn_rule_names,     # Rule names for interpretability
            'bnn_is_fitted': bnn_model is not None,
            
            # Metadata
            'threshold_selection': 'ALGORITHM-CHOSEN',
            'threshold_justification': 'Optimized by Bayesian calibration for fraud pattern detection',
            'calibration_date': datetime.now().isoformat(),
            'version': '2.1',
            'description': 'Stage 1: BNN Model Components (sklearn MLP + threshold)'
        }
        
        stage1_file = f'{prefix}_stage1.joblib'
        joblib.dump(stage1_bundle, stage1_file)
        print(f"   ✅ Stage 1: {stage1_file}")
        print(f"      └── {len(RULE_WEIGHTS)} rules (config) + BNN model, hard_block≥85")
        
        # Also save JSON for quick reference (without BNN model)
        # NOW INCLUDES RULE_CONDITIONS for inference scripts to load rule logic
        rule_json = {
            'rule_weights': RULE_WEIGHTS,
            'rule_conditions': RULE_CONDITIONS,  # NEW: Declarative rule logic
            'hard_block_threshold': 85,
            'high_risk_threshold': 60,
            'threshold_selection': 'ALGORITHM-CHOSEN',
            'calibration_date': datetime.now().isoformat(),
            'version': '3.0',  # Bumped version for new format
            'description': 'Rule engine config with declarative conditions (can be loaded by inference scripts)'
        }
        rule_json_file = f'{prefix}_rules.json'
        with open(rule_json_file, 'w') as f:
            json.dump(convert_to_serializable(rule_json), f, indent=2)
        print(f"   ✅ Stage 1 Config (JSON): {rule_json_file}")
        print(f"      └── Includes {len(RULE_CONDITIONS)} declarative rule conditions")
        
        # =====================================================
        # FILE 2: STAGE 2 - ML ENSEMBLE + SCALERS (JOBLIB)
        # Contains: XGBoost, LightGBM, CatBoost, Scalers, Weights
        # =====================================================
        
        # Get quantile transformer if available
        quantile_transformer = getattr(self, 'quantile_transformer', None)
        
        # Get rule specs for BNN (rule names in exact order)
        rule_specs = [(r[0], r[2]) for r in self.rule_engine.rules] if hasattr(self, 'rule_engine') else []
        
        # Convert XGBoost to CPU for portable inference (avoids GPU warning at inference)
        if 'xgboost' in self.models:
            try:
                # For XGBClassifier, access the underlying booster to set params
                booster = self.models['xgboost'].get_booster()
                booster.set_param({'device': 'cpu', 'predictor': 'cpu_predictor'})
                print("   🔄 XGBoost converted to CPU for portable inference")
            except Exception as e:
                print(f"   ⚠️ XGBoost CPU conversion failed (will still work): {e}")
        
        stage2_bundle = {
            # ML Models
            'models': self.models,
            'optimal_weights': getattr(self, 'optimal_weights', None),
            'meta_model': getattr(self, 'meta_model', None),
            
            # Scalers (BOTH needed for proper inference)
            'scaler': self.scaler,  # RobustScaler
            'quantile_transformer': quantile_transformer,  # QuantileTransformer
            
            # Feature Engineering Config
            'feature_names': self.feature_names,
            
            # Rule specs for BNN input (names + weights in exact order)
            'rule_specs': rule_specs,  # [(rule_name, weight), ...]
            
            # Thresholds
            'optimal_threshold': self.optimal_threshold,
            'threshold_selection': 'MANUALLY-CHOSEN',
            'threshold_justification': 'Selected 0.390 for hackathon + bank compliance (Recall≥80%, Precision≥50%, Best F2)',
            'best_ensemble_strategy': getattr(self, 'best_ensemble_strategy', 'bouncer'),
            
            # Metadata
            'calibration_date': datetime.now().isoformat(),
            'version': '2.1',
            'description': 'Stage 2: ML Ensemble + Scalers (complete inference bundle)'
        }
        
        stage2_file = f'{prefix}_ensemble.joblib'
        joblib.dump(stage2_bundle, stage2_file)
        print(f"   ✅ Stage 2: {stage2_file}")
        print(f"      └── {len(self.models)} models + scalers, threshold={self.optimal_threshold:.3f}")
        
        # =====================================================
        # OPTIONAL: Feature importance CSV
        # =====================================================
        if self.feature_importance is not None:
            importance_file = f'{prefix}_feature_importance.csv'
            self.feature_importance.to_csv(importance_file, index=False)
            print(f"   ✅ Feature Importance: {importance_file}")
        
        # =====================================================
        # OPTIONAL: Ensemble weights JSON (for quick reference)
        # =====================================================
        if hasattr(self, 'optimal_weights') and self.optimal_weights:
            weights_file = f'{prefix}_ensemble_weights.json'
            with open(weights_file, 'w') as f:
                json.dump(convert_to_serializable(self.optimal_weights), f, indent=2)
            print(f"   ✅ Ensemble Weights: {weights_file}")
        
        # =====================================================
        # FILE 3: FEATURE ENGINEERING CONFIG (JSON + JOBLIB)
        # Required for FeatureEngineer class in inference
        # =====================================================
        
        # Compute user and merchant statistics from training data
        user_stats = {}
        merchant_stats = {}
        global_stats = {}
        
        if hasattr(self, 'df'):
            df = self.df
            
            # Global amount statistics
            if 'amount' in df.columns:
                global_stats['amount_mean'] = float(df['amount'].mean())
                global_stats['amount_std'] = float(df['amount'].std())
                global_stats['amount_q90'] = float(df['amount'].quantile(0.9))
            
            # User statistics (for user behavioral features)
            if 'user_id' in df.columns and 'amount' in df.columns:
                user_groups = df.groupby('user_id')['amount']
                for user_id, group in user_groups:
                    user_stats[user_id] = {
                        'mean': float(group.mean()),
                        'std': float(group.std()) if len(group) > 1 else 0,
                        'min': float(group.min()),
                        'max': float(group.max()),
                        'median': float(group.median()),
                        'sum': float(group.sum()),
                        'count': int(len(group))
                    }
            
            # Merchant statistics (for merchant features)
            if 'merchant_id' in df.columns and 'amount' in df.columns:
                merchant_groups = df.groupby('merchant_id')
                for merchant_id, group in merchant_groups:
                    fraud_rate = group['is_fraud'].mean() if 'is_fraud' in group.columns else 0
                    merchant_stats[merchant_id] = {
                        'mean': float(group['amount'].mean()),
                        'std': float(group['amount'].std()) if len(group) > 1 else 0,
                        'median': float(group['amount'].median()),
                        'count': int(len(group)),
                        'fraud_rate': float(fraud_rate)
                    }
        
        # Get categorical column values (for one-hot encoding)
        categorical_columns = {}
        low_card_categoricals = ['currency', 'transaction_type', 'channel', 'payment_method',
                                  'device_type', 'browser', 'merchant_category', 'kyc_status']
        if hasattr(self, 'df'):
            for col in low_card_categoricals:
                if col in self.df.columns:
                    categorical_columns[col] = list(self.df[col].dropna().unique())
        
        # Save feature config JSON
        feature_config = {
            'global_stats': global_stats,
            'categorical_columns': categorical_columns,
            'training_columns': self.feature_names,
            'high_risk_countries': ['BR', 'NG', 'RU', 'CN', 'VE', 'IR', 'KP', 'SD', 'SY', 'PK']
        }
        
        config_file = f'{prefix}_feature_config.json'
        with open(config_file, 'w') as f:
            json.dump(convert_to_serializable(feature_config), f, indent=2)
        print(f"   ✅ Feature Config: {config_file}")
        print(f"      └── {len(self.feature_names)} training columns, {len(categorical_columns)} categorical cols")
        
        # Save user/merchant stats (for behavioral features)
        stats_file = f'{prefix}_user_merchant_stats.joblib'
        stats_bundle = {
            'user_stats': user_stats,
            'merchant_stats': merchant_stats
        }
        joblib.dump(stats_bundle, stats_file)
        print(f"   ✅ User/Merchant Stats: {stats_file}")
        print(f"      └── {len(user_stats)} users, {len(merchant_stats)} merchants")
        
        print("-" * 60)
        print(f"🎉 CASCADED SYSTEM SAVED SUCCESSFULLY!")
        print(f"   📌 ARCHITECTURE:")
        print(f"      Stage 1: Rules + BNN → {stage1_file}")
        print(f"      Stage 2: ML Ensemble → {stage2_file}")
        print(f"      Feature Eng: {config_file} + {stats_file}")
        print(f"   📌 THRESHOLDS:")
        print(f"      Rule Engine:  hard_block≥85 (ALGORITHM-CHOSEN)")
        print(f"      Ensemble ML:  {self.optimal_threshold:.3f} (ALGORITHM-CHOSEN)")
        print(f"   To load in production:")
        print(f"   • Stage 1: joblib.load('{stage1_file}')")
        print(f"   • Stage 2: joblib.load('{stage2_file}')")
        print(f"   • Feature Eng: FeatureEngineer('{config_file}', '{stats_file}')")


class RecallOptimizedFraudDetectionEngine(UltimateFraudDetectionEngine):
    """
    ENHANCED VERSION WITH RECALL-FOCUSED OPTIMIZATION
    Now with option to use pre-tuned hyperparameters (skips Bayesian opt)
    """
    
    def __init__(self, data_path):
        super().__init__(data_path)
        self.optimization_history = {}
        self.recall_focused_weights = None
        self.recall_target = 0.80  # 80% recall target for fraud detection

    def train_with_bayesian_optimization(self):
        """
        BAYESIAN OPTIMIZED TRAINING with F2 + Precision >= 60% objective.
        
        Delegates to train_ensemble_models_with_practical_bayesian which performs:
        - XGBoost:  60 trials (GPU)
        - LightGBM: 60 trials (CPU)
        - CatBoost: 25 trials (CPU)
        
        Objective: Maximize F2 Score with Precision >= 60%
        (Aligned with problem statement: reduce false positives while detecting fraud)
        """
        
        # Call the real Bayesian optimization function
        self.train_ensemble_models_with_practical_bayesian()
        
        # Calculate feature importance
        if 'xgboost' in self.models:
            self.feature_importance = pd.DataFrame({
                'feature': self.feature_names,
                'importance': self.models['xgboost'].feature_importances_
            }).sort_values('importance', ascending=False)
    
    def train_framework_components(self):
        
        # === Framework Components ===
        print("\n🔧 Training Framework Components...")
        
        # 1. Rule-Based Engine
        print("   [1/2] Initializing Rule Engine (10 expert rules)...")
        self.rule_engine = RuleBasedEngine()
        print("   ✓ Rule Engine ready")
        
        # 2. Autoencoder - DISABLED BY USER REQUEST (23.81% precision = adds noise)
        # print("   [2/3] Training Autoencoder on normal transactions...")
        # normal_mask = self.y_train == 0
        # X_normal = self.X_train_scaled[normal_mask]
        # self.autoencoder = AutoencoderAnomalyDetector(input_dim=min(50, self.X_train_scaled.shape[1]))
        # self.autoencoder.fit(X_normal)
        # print(f"   ✓ Autoencoder trained on {len(X_normal)} normal samples")
        
        # 2. Bayesian High-Risk Identifier (learns gray zone patterns)
        print("   [2/2] Training Bayesian High-Risk Identifier...")
        

        self.high_risk_identifier = BayesianHighRiskIdentifier(n_rules=len(self.rule_engine.rules))
        self.high_risk_identifier.fit(
            self.X_train,  # Unscaled data
            self.feature_names,
            self.y_train,
            self.rule_engine
        )
        
        print("   ✓ Framework components ready")
 
    def _apply_self_training(self, confidence_threshold=0.95, max_iterations=3):
        """Self-training with pseudo-labels on high-confidence predictions"""
        for iteration in range(max_iterations):
            # Get ensemble predictions on validation set
            preds = {}
            for name, model in self.models.items():
                preds[name] = model.predict_proba(self.X_val_scaled)[:, 1]
            
            # Average predictions
            avg_pred = np.mean(list(preds.values()), axis=0)
            
            # Find high-confidence predictions
            high_conf_fraud = avg_pred >= confidence_threshold
            high_conf_legit = avg_pred <= (1 - confidence_threshold)
            high_conf_mask = high_conf_fraud | high_conf_legit
            
            n_high_conf = high_conf_mask.sum()
            if n_high_conf < 100:
                print(f"   Iteration {iteration+1}: Only {n_high_conf} high-confidence samples, stopping")
                break
            
            # Create pseudo-labels
            pseudo_labels = (avg_pred >= 0.5).astype(int)
            
            # Add high-confidence samples to training
            X_pseudo = self.X_val_scaled[high_conf_mask]
            y_pseudo = pseudo_labels[high_conf_mask]
            
            # Retrain only XGBoost with augmented data (fastest)
            X_augmented = np.vstack([self.X_train_scaled, X_pseudo])
            y_augmented = np.concatenate([self.y_train, y_pseudo])
            
            self.models['xgboost'].fit(
                X_augmented, y_augmented,
                eval_set=[(self.X_val_scaled, self.y_val)],
                early_stopping_rounds=30,
                verbose=False
            )
            
            print(f"   Iteration {iteration+1}: Added {n_high_conf} pseudo-labeled samples")
        
        print("   ✓ Self-training complete")
    
    def train_ensemble_models_with_practical_bayesian(self, n_trials_per_model=50):
        """
        DIRECT TRAINING WITH OPTIMAL HYPERPARAMETERS (NO BAYESIAN OPTIMIZATION)
        
        Uses best hyperparameters found from previous Bayesian optimization:
        - XGBoost:  Trial 58 (F2: 0.7085)
        - LightGBM: Trial 22 (F2: 0.7145)
        - CatBoost: Trial 23 (F2: 0.7096)
        
        This skips the time-consuming Bayesian search and directly trains with optimal params.
        """
        print("\n" + "="*80)
        print("🚀 TRAINING WITH OPTIMAL HYPERPARAMETERS (FOUND USING BAYESIAN OPTIMIZATION)")
        print("   Using best parameters from previous optimization runs")
        
        # === 1. XGBoost with optimal params from Trial 58 (F2: 0.7085) ===
        print(f"\n[1/3] Training XGBoost with optimal hyperparameters...")
        
        xgb_params = {
            'objective': 'binary:logistic',
            'eval_metric': 'aucpr',
            
            # Best params from Trial 58 (F2: 0.7085)
            'scale_pos_weight': 3.604751337893015,
            'n_estimators': 1600,
            'learning_rate': 0.01264923836032133,
            'max_depth': 8,
            'min_child_weight': 8,
            'subsample': 0.7971623302216809,
            'colsample_bytree': 0.7702707561076475,
            'gamma': 0.3182485411673384,
            'reg_alpha': 0.5721343259406985,
            'reg_lambda': 2.8676192049842437,
            'early_stopping_rounds': 50,  # Moved to constructor for XGBoost 2.0+ compatibility
            
            'random_state': RANDOM_STATE,
            'n_jobs': 2,
            
            # GPU configuration
            'tree_method': 'hist',
            'device': 'cuda',
            'predictor': 'gpu_predictor',
            'max_bin': 256,
            'grow_policy': 'lossguide',
            'sampling_method': 'gradient_based'
        }
        
        self.models['xgboost'] = xgb.XGBClassifier(**xgb_params)
        self.models['xgboost'].fit(
            self.X_train_scaled, self.y_train,
            eval_set=[(self.X_val_scaled, self.y_val)],
            verbose=False
        )
        print(f"   ✅ XGBoost trained (n_estimators={xgb_params['n_estimators']}, lr={xgb_params['learning_rate']:.4f})")
        
        # === 2. LightGBM with optimal params from Trial 22 (F2: 0.7145) ===
        if LIGHTGBM_AVAILABLE:
            print(f"\n[2/3] Training LightGBM with optimal hyperparameters...")
            
            lgb_params = {
                'objective': 'binary',
                'metric': 'auc',
                
                # Best params from Trial 22 (F2: 0.7145)
                'scale_pos_weight': 3.6979249328227004,
                'n_estimators': 1511,
                'learning_rate': 0.014462382942984102,
                'max_depth': 9,
                'num_leaves': 26,
                'subsample': 0.8661931380997505,
                'colsample_bytree': 0.7391783907773977,
                'reg_alpha': 0.2057923932601703,
                'reg_lambda': 0.5098277669321674,
                'min_child_samples': 18,
                
                'random_state': RANDOM_STATE,
                'n_jobs': -1,
                'verbose': -1
            }
            
            self.models['lightgbm'] = lgb.LGBMClassifier(**lgb_params)
            self.models['lightgbm'].fit(
                self.X_train_scaled, self.y_train,
                eval_set=[(self.X_val_scaled, self.y_val)],
                callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)]
            )
            print(f"   ✅ LightGBM trained (n_estimators={lgb_params['n_estimators']}, lr={lgb_params['learning_rate']:.4f})")
        
        # === 3. CatBoost with optimal params from Trial 23 (F2: 0.7096) ===
        if CATBOOST_AVAILABLE:
            print(f"\n[3/3] Training CatBoost with optimal hyperparameters...")
            
            cat_params = {
                'loss_function': 'Logloss',
                'eval_metric': 'AUC',
                
                # Best params from Trial 23 (F2: 0.7096)
                'scale_pos_weight': 3.6124435025354416,
                'iterations': 1481,
                'learning_rate': 0.0686923604666307,
                'depth': 6,
                'l2_leaf_reg': 1.4658666408948338,
                'border_count': 166,
                
                'random_seed': RANDOM_STATE,
                'verbose': 0,
                
                # GPU configuration
                'task_type': 'GPU',
                'devices': '0:0',
                'used_ram_limit': '10GB',
                'bootstrap_type': 'Bernoulli',
                'allow_writing_files': False
            }
            
            self.models['catboost'] = cb.CatBoostClassifier(**cat_params)
            self.models['catboost'].fit(
                self.X_train_scaled, self.y_train,
                eval_set=(self.X_val_scaled, self.y_val),
                early_stopping_rounds=50,
                verbose=False
            )
            print(f"   ✅ CatBoost trained (iterations={cat_params['iterations']}, lr={cat_params['learning_rate']:.4f})")
        
        # Calculate feature importance
        if 'xgboost' in self.models:
            self.feature_importance = pd.DataFrame({
                'feature': self.feature_names,
                'importance': self.models['xgboost'].feature_importances_
            }).sort_values('importance', ascending=False)
        
        print(f"\n✅ All models trained with optimal hyperparameters (Using Bayesian Optimization backed optimal weights)")
        print(f"   Total models: {len(self.models)}")
        
        return self.models
    
    
    def optimize_ensemble_weights_practical_recall_focused(self, n_trials=300):
        """
        DIRECT ENSEMBLE WEIGHT ASSIGNMENT (NO BAYESIAN OPTIMIZATION)
        
        Uses optimal weights from previous Bayesian optimization Trial 256:
        - XGBoost:  24.16%
        - LightGBM: 68.47%
        - CatBoost:  7.37%
        
        Performance achieved: F2=73.25%, Score=0.7325
        """
        print("\n" + "="*80)
        print("🚀 ENSEMBLE WEIGHT ASSIGNMENT (Using Bayesian Optimization backed optimal weights)")
        print("   Using optimal weights from previous optimization")
        print("="*80)
        
        # Get model names for weight mapping
        model_names = list(self.models.keys())
        print(f"   📊 Setting weights for {len(model_names)} models: {model_names}")
        
        # Optimal weights from Trial 256 (Score: 0.7325)
        # weight_0 (xgboost): 0.24161051662086008
        # weight_1 (lightgbm): 0.6846536032579759
        # catboost: 1 - 0.24161051662086008 - 0.6846536032579759 = 0.0737358801211640
        optimal_weight_map = {
            'xgboost': 0.24161051662086008,   # 24.16%
            'lightgbm': 0.6846536032579759,   # 68.47%
            'catboost': 0.0737358801211640    # 7.37%
        }
        
        # Build weights array in model order
        optimal_weights = []
        for name in model_names:
            weight = optimal_weight_map.get(name, 1.0 / len(model_names))
            optimal_weights.append(weight)
        
        # Normalize to ensure sum = 1
        optimal_weights = np.array(optimal_weights)
        optimal_weights = optimal_weights / optimal_weights.sum()
        
        print(f"\n🏆 OPTIMAL WEIGHTS (from Trial 256, Score=0.7325):")
        for name, weight in zip(model_names, optimal_weights):
            print(f"   {name:12s}: {weight:.4f} ({weight*100:.2f}%)")
        
        # Store weights
        self.balanced_weights = dict(zip(model_names, optimal_weights))
        self.optimal_weights = self.balanced_weights
        
        print(f"\n✅ Ensemble weights set (Using Bayesian Optimization backed optimal weights)")
        
        return self.balanced_weights
    
    def train_advanced_stacking_ensemble(self):
        """
        ADVANCED STACKING WITH MULTIPLE META-LEARNERS
        Trains and selects the best meta-model for stacking
        """
        print("\n🎓 TRAINING ADVANCED STACKING ENSEMBLE...")
        
        # Get base model predictions
        base_predictions_train = []
        base_predictions_val = []
        
        for name, model in self.models.items():
            train_pred = model.predict_proba(self.X_train_scaled)[:, 1]
            val_pred = model.predict_proba(self.X_val_scaled)[:, 1]
            
            base_predictions_train.append(train_pred)
            base_predictions_val.append(val_pred)
        
        X_meta_train = np.column_stack(base_predictions_train)
        X_meta_val = np.column_stack(base_predictions_val)
        
        # Define meta-model with optimal params from Trial 296
        # C: 0.08521596812289807, class_weight_ratio: 2.8291961956998812
        meta_models = {
            'logistic': LogisticRegression(
                class_weight={0: 1.0, 1: 2.8291961956998812},  # Optimal class weight ratio
                random_state=RANDOM_STATE,
                max_iter=1000,
                C=0.08521596812289807,  # Optimal C from Trial 296
                solver='liblinear'
            )
        }
        
        # Evaluate each meta-model
        best_meta_model = None
        best_meta_score = -float('inf')
        meta_results = {}
        
        print("\n   Meta-Model Evaluation:")
        print(f"   {'Model':<20} {'Recall%':>10} {'F1%':>10} {'F2%':>10}")
        print(f"   {'-'*20} {'-'*10} {'-'*10} {'-'*10}")
        
        for name, model in meta_models.items():
            try:
                model.fit(X_meta_train, self.y_train)
                y_meta_pred = model.predict(X_meta_val)
                
                recall = recall_score(self.y_val, y_meta_pred) * 100
                f1 = f1_score(self.y_val, y_meta_pred) * 100
                f2 = fbeta_score(self.y_val, y_meta_pred, beta=2) * 100
                
                # Score with emphasis on recall
                score = recall * 0.7 + f1 * 0.3
                
                meta_results[name] = {
                    'recall': recall,
                    'f1': f1,
                    'f2': f2,
                    'score': score
                }
                
                print(f"   {name:<20} {recall:>10.2f} {f1:>10.2f} {f2:>10.2f}")
                
                if score > best_meta_score:
                    best_meta_score = score
                    best_meta_model = (name, model)
                    
            except Exception as e:
                print(f"   {name:<20} Failed: {str(e)[:30]}...")
        
        # Train best meta-model on combined data
        if best_meta_model:
            meta_name, meta_model = best_meta_model
            
            # Combine train and val for final training
            X_meta_combined = np.vstack([X_meta_train, X_meta_val])
            y_combined = np.concatenate([self.y_train, self.y_val])
            
            meta_model.fit(X_meta_combined, y_combined)
            self.meta_model = meta_model
            
            print(f"\n   ✅ Selected meta-model: {meta_name}")
            print(f"   📊 Performance: Recall={meta_results[meta_name]['recall']:.2f}%, "
                  f"F1={meta_results[meta_name]['f1']:.2f}%")
        else:
            # Fallback to logistic regression
            print("\n   ⚠️  No meta-model selected, using logistic regression as fallback")
            self.meta_model = LogisticRegression(
                class_weight='balanced',
                random_state=RANDOM_STATE,
                max_iter=1000
            )
            X_meta_combined = np.vstack([X_meta_train, X_meta_val])
            y_combined = np.concatenate([self.y_train, self.y_val])
            self.meta_model.fit(X_meta_combined, y_combined)
        
        return self.meta_model
    
    def compare_ensemble_strategies_recall_focused(self):
        """
        ENSEMBLE STRATEGY COMPARISON WITH RECALL PRIORITY
        """
        print("\n" + "="*80)
        print("🔬 RECALL-FOCUSED ENSEMBLE STRATEGY COMPARISON")
        print("="*80)
        
        strategies = ['bouncer', 'enhanced', 'average', 'weighted', 'optimized', 'stacking']
        results = {}
        
        print(f"\n{'Strategy':>12} {'Acc%':>8} {'Prec%':>8} {'Rec%':>8} {'F1%':>8} {'F2%':>8} {'R-F Score':>10}")
        print("-" * 80)
        
        best_strategy = None
        best_recall_f1_score = 0
        
        for strategy in strategies:
            try:
                y_pred_proba = self.get_ensemble_predictions(self.X_val_scaled, mode=strategy)
                
                # Optimize threshold for recall-F1 balance
                best_thresh = 0.5
                best_recall_f1 = 0
                
                # Search thresholds with bias toward higher recall
                for thresh in np.linspace(0.2, 0.6, 41):  # Lower range for higher recall
                    y_pred = (y_pred_proba > thresh).astype(int)
                    recall = recall_score(self.y_val, y_pred)
                    f1 = f1_score(self.y_val, y_pred)
                    
                    # RECALL-F1 BALANCED SCORE (weights recall more)
                    recall_f1_score = recall * 0.6 + f1 * 0.4
                    
                    if recall_f1_score > best_recall_f1:
                        best_recall_f1 = recall_f1_score
                        best_thresh = thresh
                
                y_pred = (y_pred_proba > best_thresh).astype(int)
                
                # Calculate metrics
                acc = accuracy_score(self.y_val, y_pred) * 100
                prec = precision_score(self.y_val, y_pred, zero_division=0) * 100
                rec = recall_score(self.y_val, y_pred) * 100
                f1 = f1_score(self.y_val, y_pred) * 100
                f2 = fbeta_score(self.y_val, y_pred, beta=2) * 100
                auc = roc_auc_score(self.y_val, y_pred_proba) * 100
                
                # Combined recall-F1 score for ranking
                recall_f1_combined = rec * 0.6 + f1 * 0.4
                
                results[strategy] = {
                    'accuracy': acc, 'precision': prec, 'recall': rec,
                    'f1': f1, 'f2': f2, 'auc': auc, 'threshold': best_thresh,
                    'recall_f1_score': recall_f1_combined
                }
                
                # Mark stacking with special notation for high recall
                marker = "★" if strategy == 'stacking' else " "
                print(f"{marker}{strategy:>11} {acc:>8.2f} {prec:>8.2f} {rec:>8.2f} {f1:>8.2f} {f2:>8.2f} {recall_f1_combined:>10.2f}")
                
                if recall_f1_combined > best_recall_f1_score:
                    best_recall_f1_score = recall_f1_combined
                    best_strategy = strategy
                    
            except Exception as e:
                print(f"{strategy:>12} - Failed: {str(e)}")
        
        print("\n" + "="*80)
        
        # DYNAMIC STRATEGY SELECTION BASED ON RECALL
        stacking_recall = results.get('stacking', {}).get('recall', 0)
        weighted_recall = results.get('weighted', {}).get('recall', 0)
        stacking_f1 = results.get('stacking', {}).get('f1', 0)
        weighted_f1 = results.get('weighted', {}).get('f1', 0)
        
        # Select stacking if it has significantly higher recall with acceptable F1 loss
        if (stacking_recall - weighted_recall > 5 and 
            weighted_f1 - stacking_f1 < 5):
            print(f"🏆 SELECTING STACKING (Superior Recall: +{stacking_recall-weighted_recall:.1f}%)")
            best_strategy = 'stacking'
            selection_reason = "Stacking selected for superior recall"
        else:
            print(f"🏆 BEST STRATEGY: {best_strategy.upper()}")
            selection_reason = f"{best_strategy.upper()} selected based on recall-F1 score"
        
        print(f"   {selection_reason}")
        print(f"   Recall-F1 Score: {best_recall_f1_score:.2f}")
        print("="*80)
        
        self.best_ensemble_strategy = best_strategy
        self.ensemble_comparison_results = results
        
        return best_strategy, results
    
    def find_optimal_threshold_recall_focused(self):
        """
        CASCADE-BASED THRESHOLD ANALYSIS
        
        Uses full cascade predictions (Rules→BNN→ML blend) on validation set
        to find optimal threshold that:
        - Maximizes Recall
        - With Precision in 50-55% range (optimal for fraud detection)
        """
        print("\n" + "="*100)
        print("🎯 CASCADE-BASED THRESHOLD ANALYSIS")
        ml_strat_display = getattr(self, 'best_voting_strategy', 'average')
        print(f"   Using full pipeline: Rules (≥85% hard block) → BNN → {ml_strat_display.upper()} → Blend")
        print("   Constraint: Max Recall for Precision ≈ 50-55%")
        print("="*100)
        
        # === STEP 1: Get CASCADE predictions on validation set ===
        # This mimics evaluate_final() logic but on validation data
        
        HARD_BLOCK_THRESHOLD = 0.85
        HIGH_RISK_THRESHOLD = 0.60
        
        n_val = len(self.y_val)
        cascade_predictions = np.zeros(n_val)
        
        # Get rule scores on unscaled data
        if hasattr(self, 'rule_engine') and hasattr(self, 'feature_names') and hasattr(self, 'X_val'):
            rule_scores = self.rule_engine.predict_batch(self.X_val, self.feature_names)
            rule_scores_norm = rule_scores / 100.0
            
            # Hard block
            hard_block_mask = rule_scores_norm >= HARD_BLOCK_THRESHOLD
            cascade_predictions[hard_block_mask] = 1.0
            
            # Soft zone processing
            soft_mask = ~hard_block_mask
            if soft_mask.sum() > 0:
                X_soft = self.X_val_scaled[soft_mask]
                soft_rule_scores = rule_scores_norm[soft_mask]
                
                # Get ML predictions using best_voting_strategy (same as evaluate_final)
                ml_strategy = getattr(self, 'best_voting_strategy', 'average')
                if ml_strategy == 'bouncer':
                    ml_strategy = 'average'
                
                if ml_strategy == 'stack_enhanced':
                    ml_probs = self._strategy_stacking_enhanced(X_soft)
                elif ml_strategy == 'tiered_vote':
                    ml_probs = self._strategy_tiered_threshold(X_soft)
                elif ml_strategy == 'conf_weighted':
                    ml_probs = self._strategy_confidence_weighted(X_soft)
                elif ml_strategy == 'delphi':
                    ml_probs = self._strategy_delphi_consensus(X_soft)
                else:
                    mode_map = {'average': 'average', 'weighted': 'weighted', 'bayesian_opt': 'optimized'}
                    ml_probs = self.get_ensemble_predictions(X_soft, mode=mode_map.get(ml_strategy, 'average'))
                
                # BNN High-Risk Detection
                if hasattr(self, 'high_risk_identifier') and self.high_risk_identifier.is_fitted:
                    X_soft_unscaled = self.X_val[soft_mask]
                    bnn_result = self.high_risk_identifier.predict(
                        X_soft_unscaled, self.feature_names, self.rule_engine
                    )
                    bnn_high_risk = bnn_result['is_high_risk']
                else:
                    bnn_high_risk = np.zeros(len(X_soft), dtype=bool)
                
                # Cascade Blending
                strong_rule_mask = soft_rule_scores >= HIGH_RISK_THRESHOLD
                use_high_risk = bnn_high_risk | strong_rule_mask
                
                blended = np.zeros(len(X_soft))
                blended[use_high_risk] = ml_probs[use_high_risk] * 0.65 + soft_rule_scores[use_high_risk] * 0.35
                blended[~use_high_risk] = ml_probs[~use_high_risk] * 0.90 + soft_rule_scores[~use_high_risk] * 0.10
                
                cascade_predictions[soft_mask] = np.clip(blended, 0, 1)
        else:
            # Fallback: use ML ensemble only
            print("   ⚠️ Rule engine not available, using ML ensemble only")
            strategy = getattr(self, 'best_ensemble_strategy', 'weighted')
            cascade_predictions = self.get_ensemble_predictions(self.X_val_scaled, mode=strategy)
        
        # === STEP 2: Threshold Analysis with CASCADE predictions ===
        thresholds = np.linspace(0.10, 0.60, 51)
        
        print(f"\n{'Thresh':>7} │ {'Recall%':>8} {'Prec%':>8} {'F1%':>8} {'F2%':>8} │ {'TP':>6} {'FP':>6} {'FN':>6} │ Notes")
        print("─"*95)
        
        all_results = []
        
        for thresh in thresholds:
            y_pred = (cascade_predictions > thresh).astype(int)
            
            recall = recall_score(self.y_val, y_pred)
            precision = precision_score(self.y_val, y_pred, zero_division=0)
            f1 = f1_score(self.y_val, y_pred)
            f2 = fbeta_score(self.y_val, y_pred, beta=2)
            
            tn, fp, fn, tp = confusion_matrix(self.y_val, y_pred).ravel()
            
            result = {
                'threshold': thresh, 'recall': recall, 'precision': precision,
                'f1': f1, 'f2': f2, 'tp': tp, 'fp': fp, 'fn': fn
            }
            all_results.append(result)
            
            # Notes
            notes = []
            if recall >= 0.80:
                notes.append("✓Rec≥80")
            if 0.50 <= precision <= 0.55:
                notes.append("★P:50-55%")
            elif precision >= 0.50:
                notes.append("✓Prec≥50")
            
            notes_str = " ".join(notes) if notes else ""
            print(f"{thresh:>7.3f} │ {recall*100:>8.2f} {precision*100:>8.2f} {f1*100:>8.2f} {f2*100:>8.2f} │ {tp:>6} {fp:>6} {fn:>6} │ {notes_str}")
        
        print("─"*95)
        
        # === STEP 3: AUTOMATIC Threshold Selection ===
        # Constraint: Max Recall for Precision in 50-55%
        PREC_MIN = 0.50
        PREC_MAX = 0.55
        
        viable = [r for r in all_results if PREC_MIN <= r['precision'] <= PREC_MAX]
        
        print(f"\n🎯 AUTOMATIC THRESHOLD SELECTION:")
        print(f"   Constraint: Best F2 with Precision in {PREC_MIN*100:.0f}%-{PREC_MAX*100:.0f}%")
        
        if viable:
            # Best = max F2 within precision constraint (optimal for hackathon + bank compliance)
            best = max(viable, key=lambda x: x['f2'])
            print(f"\n   ✅ OPTIMAL THRESHOLD (Algorithm-Chosen): {best['threshold']:.3f}")
            print(f"      → F2-Score: {best['f2']*100:.2f}% (MAXIMIZED)")
            print(f"      → Recall: {best['recall']*100:.2f}%")
            print(f"      → Precision: {best['precision']*100:.2f}% (in target range)")
        else:
            # Fallback: find threshold with precision closest to 52.5% (midpoint)
            midpoint = (PREC_MIN + PREC_MAX) / 2
            best = min(all_results, key=lambda x: abs(x['precision'] - midpoint))
            print(f"\n   ⚠️ No threshold in exact range, using closest to {midpoint*100:.1f}%")
            print(f"   SELECTED THRESHOLD: {best['threshold']:.3f}")
            print(f"      → Recall: {best['recall']*100:.2f}%")
            print(f"      → Precision: {best['precision']*100:.2f}%")
        
        # Also show other reference points
        print(f"\n📊 REFERENCE THRESHOLDS:")
        
        # Best F2 unconstrained
        best_f2 = max(all_results, key=lambda x: x['f2'])
        print(f"   Best F2:       {best_f2['threshold']:.3f} → F2={best_f2['f2']*100:.2f}%, Rec={best_f2['recall']*100:.2f}%, Prec={best_f2['precision']*100:.2f}%")
        
        # Best F1 (balanced)
        best_f1 = max(all_results, key=lambda x: x['f1'])
        print(f"   Best F1:       {best_f1['threshold']:.3f} → F1={best_f1['f1']*100:.2f}%, Rec={best_f1['recall']*100:.2f}%, Prec={best_f1['precision']*100:.2f}%")
        
        # Max recall with P >= 50%
        viable_50 = [r for r in all_results if r['precision'] >= 0.50]
        if viable_50:
            best_rec_50 = max(viable_50, key=lambda x: x['recall'])
            print(f"   Max Rec@P≥50%: {best_rec_50['threshold']:.3f} → Rec={best_rec_50['recall']*100:.2f}%, Prec={best_rec_50['precision']*100:.2f}%")
        
        self.optimal_threshold = best['threshold']
        self.threshold_analysis = all_results
        
        print(f"\n✅ Selected threshold: {self.optimal_threshold:.3f}")
        
        return best
    
    def evaluate_final_recall_optimized(self):
        """
        FINAL EVALUATION WITH BALANCE FOCUS
        Enhanced to show precision-recall balance metrics
        """
        # Header is printed by evaluate_final() - no duplicate needed
        
        # Get results from parent class
        results = self.evaluate_final()
        
        # === NEW: Balance Analysis ===
        print(f"\n🎯 BALANCE ANALYSIS:")
        
        recall = results['recall']
        precision = results['precision']
        f1 = results['f1']
        
        # Precision-Recall Gap
        pr_gap = abs(precision - recall)
        gap_status = '✅' if pr_gap < 20 else '📊' if pr_gap < 30 else '📊'
        print(f"   Precision-Recall Gap: {pr_gap:.1f}% {gap_status}")
        
        # Balance Score (min/max ratio)
        balance_score = min(precision, recall) / (max(precision, recall) + 1e-6)
        balance_status = '✅' if balance_score > 0.70 else '📊' if balance_score > 0.50 else '📊'
        print(f"   Balance Score: {balance_score:.3f} {balance_status}")
        
        # F1 Score Assessment
        f1_status = '✅' if f1 > 65 else '📊' if f1 > 55 else '📊'
        print(f"   F1 Score: {f1:.2f}% {f1_status}")
        
        # Overall balance grade
        if balance_score > 0.70 and f1 > 65:
            grade = "EXCELLENT"
            grade_emoji = "🏆"
        elif balance_score > 0.60 and f1 > 60:
            grade = "GOOD"
            grade_emoji = "✅"
        elif balance_score > 0.50:
            grade = "MODERATE"
            grade_emoji = "📊"
        else:
            grade = "NEEDS IMPROVEMENT"
            grade_emoji = "📊"
        
        print(f"   Overall Balance: {grade_emoji} {grade}")
        
        # Precision improvement check
        baseline_precision = 36.93  # Original system
        if precision > baseline_precision:
            improvement = precision - baseline_precision
            print(f"\n   ✅ PRECISION IMPROVED: {precision:.1f}% (+{improvement:.1f}% from baseline)")
        else:
            print(f"   📊 Precision: {precision:.1f}%")
        
        # Show optimization summary
        if hasattr(self, 'optimization_history'):
            total_trials = 0
            for model_name, study in self.optimization_history.items():
                if study:
                    total_trials += len(study.trials)
            
            print(f"\n🧬 OPTIMIZATION SUMMARY:")
            print(f"   • Mode: FAST (Hardcoded Optimal Parameters)")
            print(f"   • Bayesian Optimization: SKIPPED for all models")
            print(f"   • Cost Savings: ~600+ trials avoided")
            # print(f"   • Total optimization trials: {total_trials}")
            # print(f"   • Models optimized: {', '.join([k for k, v in self.optimization_history.items() if v])}")
        
        return results


# ============================================================================
# MAIN EXECUTION WITH BALANCED PRECISION-RECALL OPTIMIZATION
# ============================================================================

# In the main execution section:

if __name__ == "__main__":
    DATA_PATH = r"C:\Users\ADMIN\Documents\TechFiesta_2026\data\realistic_transaction_dataset_821.csv"
    
    print("="*80)
    print("🚀 ULTIMATE FRAUD DETECTION ENGINE ")
    print("   🎯 Target: Maximize F2 Score with balanced Precision-Recall")
    print("   ⚡ 3 ML Models: XGBoost, LightGBM, CatBoost")
    print("="*80)

    try:
        # Initialize enhanced engine
        engine = RecallOptimizedFraudDetectionEngine(DATA_PATH)
        
        # Step 1: Data Preparation & Feature Engineering
        print("\n[Step 1/8] Data Preparation & Feature Engineering...")
        engine.prepare_data_with_time_aware_split()
        
        # Step 2: Verify Class Distribution
        print("\n[Step 2/8] Verifying Class Distribution...")
        engine.verify_class_distribution()

        # Step 3: Initialising Rule-Based Engine and Training BNN
        print("\n[Step 3/8] Initializing Rule-Based Engine and Training BNN")
        engine.train_framework_components()
        
        # Step 4: Training Models with Optimal Hyperparameters
        print("\n[Step 4/8] Training ML Models (pre-tuned hyperparameters)...")
        engine.train_with_bayesian_optimization()
        
        # Step 5: Ensemble Weights (Direct Assignment)
        print("\n[Step 5/8] Assigning Optimized Ensemble Weights...")
        engine.optimize_ensemble_weights_practical_recall_focused(n_trials=0)
        
        # Step 6: Comparing ALL Voting Strategies (includes stack_enhanced with meta-model)
        print("\n[Step 6/8] Comparing ALL Voting Strategies...")
        best_strategy, strategy_results = engine.compare_all_strategies()
        
        # Step 7: Threshold Analysis
        print("\n[Step 7/8] Threshold Analysis (ALL METRICS)...")
        engine.find_optimal_threshold_recall_focused()
        
        # Step 8: Final Cascaded Evaluation on Test Set
        print("\n[Step 8/8] Final Cascaded Evaluation on Test Set...")
        results = engine.evaluate_final_recall_optimized()
        
        engine.plot_comprehensive_analysis()
        
        # Save system
        engine.save_complete_system('ultimate_fraud_system_fast.joblib')
        
        # Save results
        with open('ultimate_evaluation_results_fast.json', 'w') as f:
            enhanced_results = {
                **results,
                'recall_target': engine.recall_target,
                'mode': 'fast_pretrained_params'
            }
            json.dump(convert_to_serializable(enhanced_results), f, indent=4)
        
        print("\n" + "="*80)
        print("✅ FAST MODE ENGINE - COMPLETE!")
        print("="*80)
        
        print(f"\n📊 SUMMARY:")
        print(f"   • Models trained: {len(engine.models)} (with pre-tuned hyperparameters)")
        print(f"   • Bayesian optimization: SKIPPED (using best params from previous run)")
        print(f"   • Threshold table: Printed above for manual selection")
        
        print(f"\n🎯 PERFORMANCE:")
        print(f"   • Recall: {results['recall']:.2f}%")
        print(f"   • Precision: {results['precision']:.2f}%")
        print(f"   • F1-Score: {results['f1']:.2f}%")
        
        print(f"\n📌 TO OVERRIDE THRESHOLD:")
        print(f"   results = engine.evaluate_final_recall_optimized()")
        
        print(f"\n💾 SAVED:")
        print(f"   - ultimate_fraud_system_fast.joblib")
        print(f"   - ultimate_evaluation_results_fast.json")
        
    except FileNotFoundError:
        print(f"\n❌ ERROR: File not found at {DATA_PATH}")
        print("   Please update DATA_PATH with your dataset location.")
    except Exception as e:
        print(f"\n❌ ERROR: {str(e)}")
        import traceback
        traceback.print_exc()

🖥️  Compute device: GPU (NVIDIA)
🔧 MAX_THREADS: 8
🚀 ULTIMATE FRAUD DETECTION ENGINE 
   🎯 Target: Maximize F2 Score with balanced Precision-Recall
   ⚡ 3 ML Models: XGBoost, LightGBM, CatBoost
📂 Loading data from C:\Users\ADMIN\Documents\TechFiesta_2026\data\realistic_transaction_dataset_821.csv
   ✓ Converted timestamp to datetime
   ✓ Converted user_signup_date to datetime
✅ Loaded: (150000, 38)
💰 Fraud rate: 23.67%

[Step 1/8] Data Preparation & Feature Engineering...

📊 Preparing time-aware data splits (70:15:15)...

🔧 ULTIMATE Feature Engineering (150+ features)...
   ⏰ Temporal features...
   💵 Amount features...
   👤 User behavioral features...
   🌍 Location features...
   🏪 Merchant features...
   📱 Device & Channel features...
   🕸️ Creating Graph Network features...
   ⚠️ Risk flags & composite scores...
   🎯 Enhanced balance features...
   🔗 Feature interactions...
   🎯 Pattern-aligned features (matching fraud injection logic)...
   📊 Creating composite pattern scores...
   

Default metric period is 5 because AUC is/are not implemented for GPU


   ✅ CatBoost trained (iterations=1481, lr=0.0687)

✅ All models trained with optimal hyperparameters (Using Bayesian Optimization backed optimal weights)
   Total models: 3

[Step 5/8] Assigning Optimized Ensemble Weights...

🚀 ENSEMBLE WEIGHT ASSIGNMENT (Using Bayesian Optimization backed optimal weights)
   Using optimal weights from previous optimization
   📊 Setting weights for 3 models: ['xgboost', 'lightgbm', 'catboost']

🏆 OPTIMAL WEIGHTS (from Trial 256, Score=0.7325):
   xgboost     : 0.2416 (24.16%)
   lightgbm    : 0.6847 (68.47%)
   catboost    : 0.0737 (7.37%)

✅ Ensemble weights set (Using Bayesian Optimization backed optimal weights)

[Step 6/8] Comparing ALL Voting Strategies...

🔬 COMPARING ALL VOTING STRATEGIES (Validation Set)

Strategy            Acc%    Prec%     Rec%      F1%      F2%     AUC%
--------------------------------------------------------------------------------
average            83.16    60.17✓    77.39    67.70    73.20    89.16 ⭐
weighted          